# Imports

In [1]:
import math
import os
import sys
import pandas as pd
sys.path.append("/home/weber/PycharmProjects/EXOTIC/src")
from statannot import add_stat_annotation

pd.options.mode.chained_assignment = None  # default='warn'
import multiprocessing
import parmap
import numpy as np
import collections
from tqdm import tqdm
import pandarallel
from pandarallel import pandarallel

pandarallel.initialize(nb_workers=20, progress_bar=True)
# tqdm.pandas()
from pprint import pprint
from scipy.stats import zscore
from scipy import stats

import requests
import re
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.colors as mc
from matplotlib.colors import LogNorm
import json
import matplotlib.ticker as mtick
from utils.utils import load_config_file

## YAML FILES CONFIG
yaml = load_config_file(config_file="/home/weber/PycharmProjects/EXOTIC/src/config.yaml")
exotic_files = yaml

import matplotlib.font_manager as font_manager

font_dirs = ['/home/weber/Fonts', ]
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)
font_list = font_manager.createFontList(font_files)
font_manager.fontManager.ttflist.extend(font_list)

from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']
rcParams['font.weight'] = 'light'

INFO: Pandarallel will run on 20 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


/home/weber/.conda/envs/ExoCarto/lib/python3.7/site-packages/ipykernel_launcher.py:42: MatplotlibDeprecationWarning: 
The createFontList function was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use FontManager.addfont instead.


# RefSeq GRCh38 parsing to retrieve exon-intron boundaries

In [135]:
import pandas as pd
pd.set_option('display.width', 1000)
refseq_grch38_gff = "/gstock/biolo_datasets/refseq/2020/GRCh38_latest_genomic.gff.gz"
refseq_38_df = pd.read_csv(refseq_grch38_gff, compression='gzip', sep='\t', skiprows=9, names=['NC', 'RefSeq_validation', 'Region_type', 'Start', 'End', 'Score', 'Strand', 'Phase', 'Attributes'])
refseq_38_df = refseq_38_df.dropna(subset=['Start', 'End'])
refseq_38_df['Start'] = refseq_38_df['Start'].astype(int)
refseq_38_df['End'] = refseq_38_df['End'].astype(int)
# refseq_38_df['Attributes']
refseq_38_df

,NC,RefSeq_validation,Region_type,Start,End,Score,Strand,Phase,Attributes
0,NC_000001.11,RefSeq,region,1,248956422,.,+,.,ID=NC_000001.11:1..248956422;Dbxref=taxon:9606;Name=1;chromosome=1;gbkey=Src;genome=chromosome;mol_type=genomic DNA
1,NC_000001.11,BestRefSeq,pseudogene,11874,14409,.,+,.,"ID=gene-DDX11L1;Dbxref=GeneID:100287102,HGNC:HGNC:37102;Name=DDX11L1;description=DEAD/H-box helicase 11 like 1 (pseudogene);gbkey=Gene;gene=DDX11L1;gene_biotype=transcribed_pseudogene;pseudo=true"
2,NC_000001.11,BestRefSeq,transcript,11874,14409,.,+,.,"ID=rna-NR_046018.2;Parent=gene-DDX11L1;Dbxref=GeneID:100287102,Genbank:NR_046018.2,HGNC:HGNC:37102;Name=NR_046018.2;gbkey=misc_RNA;gene=DDX11L1;product=DEAD/H-box helicase 11 like 1 (pseudogene);p..."
3,NC_000001.11,BestRefSeq,exon,11874,12227,.,+,.,"ID=exon-NR_046018.2-1;Parent=rna-NR_046018.2;Dbxref=GeneID:100287102,Genbank:NR_046018.2,HGNC:HGNC:37102;gbkey=misc_RNA;gene=DDX11L1;product=DEAD/H-box helicase 11 like 1 (pseudogene);pseudo=true;..."
4,NC_000001.11,BestRefSeq,exon,12613,12721,.,+,.,"ID=exon-NR_046018.2-2;Parent=rna-NR_046018.2;Dbxref=GeneID:100287102,Genbank:NR_046018.2,HGNC:HGNC:37102;gbkey=misc_RNA;gene=DDX11L1;product=DEAD/H-box helicase 11 like 1 (pseudogene);pseudo=true;..."
...,...,...,...,...,...,...,...,...,...
3781987,NC_012920.1,RefSeq,exon,15888,15953,.,+,.,"ID=exon-TRNT-1;Parent=rna-TRNT;Dbxref=GeneID:4576,HGNC:HGNC:7499,MIM:590090;Note=NAR: 1797;anticodon=(pos:15919..15921);codons=38;gbkey=tRNA;gene=TRNT;product=tRNA-Thr"
3781988,NC_012920.1,RefSeq,gene,15956,16023,.,-,.,"ID=gene-TRNP;Dbxref=GeneID:4571,HGNC:HGNC:7494,MIM:590075;Name=TRNP;gbkey=Gene;gene=TRNP;gene_biotype=tRNA;gene_synonym=MTTP"
3781989,NC_012920.1,RefSeq,tRNA,15956,16023,.,-,.,"ID=rna-TRNP;Parent=gene-TRNP;Dbxref=GeneID:4571,HGNC:HGNC:7494,MIM:590075;Note=NAR: 1597;anticodon=(pos:complement(15990..15992));codons=22;gbkey=tRNA;gene=TRNP;product=tRNA-Pro"
3781990,NC_012920.1,RefSeq,exon,15956,16023,.,-,.,"ID=exon-TRNP-1;Parent=rna-TRNP;Dbxref=GeneID:4571,HGNC:HGNC:7494,MIM:590075;Note=NAR: 1597;anticodon=(pos:complement(15990..15992));codons=22;gbkey=tRNA;gene=TRNP;product=tRNA-Pro"


In [136]:
refseq_38_df_chroms = refseq_38_df.loc[refseq_38_df['Region_type'] == 'region']
index_list = list(refseq_38_df_chroms.index)
index_list

chroms = [(i,index_list[j+1]-1) for j,i in enumerate(index_list) if j < (len(index_list)-1)]
# chroms = [i for i,j in enumerate(index_list)]
refseq_38_df_chroms

,NC,RefSeq_validation,Region_type,Start,End,Score,Strand,Phase,Attributes
0,NC_000001.11,RefSeq,region,1,248956422,.,+,.,ID=NC_000001.11:1..248956422;Dbxref=taxon:9606;Name=1;chromosome=1;gbkey=Src;genome=chromosome;mol_type=genomic DNA
345657,NT_187361.1,RefSeq,region,1,175055,.,+,.,ID=NT_187361.1:1..175055;Dbxref=taxon:9606;Name=1;chromosome=1;gbkey=Src;genome=genomic;map=unlocalized;mol_type=genomic DNA
345685,NT_187362.1,RefSeq,region,1,32032,.,+,.,ID=NT_187362.1:1..32032;Dbxref=taxon:9606;Name=1;chromosome=1;gbkey=Src;genome=genomic;map=unlocalized;mol_type=genomic DNA
345688,NT_187363.1,RefSeq,region,1,127682,.,+,.,ID=NT_187363.1:1..127682;Dbxref=taxon:9606;Name=1;chromosome=1;gbkey=Src;genome=genomic;map=unlocalized;mol_type=genomic DNA
345696,NT_187364.1,RefSeq,region,1,66860,.,+,.,ID=NT_187364.1:1..66860;Dbxref=taxon:9606;Name=1;chromosome=1;gbkey=Src;genome=genomic;map=unlocalized;mol_type=genomic DNA
...,...,...,...,...,...,...,...,...,...
3780867,NT_187685.1,RefSeq,region,1,170148,.,+,.,ID=NT_187685.1:1..170148;Dbxref=taxon:9606;Name=19;chromosome=19;gbkey=Src;genome=genomic;map=19q13.42;mol_type=genomic DNA
3781063,NT_187686.1,RefSeq,region,1,215732,.,+,.,ID=NT_187686.1:1..215732;Dbxref=taxon:9606;Name=19;chromosome=19;gbkey=Src;genome=genomic;map=19q13.42;mol_type=genomic DNA
3781390,NT_187687.1,RefSeq,region,1,170537,.,+,.,ID=NT_187687.1:1..170537;Dbxref=taxon:9606;Name=19;chromosome=19;gbkey=Src;genome=genomic;map=19q13.42;mol_type=genomic DNA
3781593,NT_113949.2,RefSeq,region,1,177381,.,+,.,ID=NT_113949.2:1..177381;Dbxref=taxon:9606;Name=19;chromosome=19;gbkey=Src;genome=genomic;map=19q13.42;mol_type=genomic DNA


In [137]:
pd.options.display.max_colwidth = 200
refseq_38_df_pc_genes = refseq_38_df.loc[(refseq_38_df['Attributes'].str.contains('gene_biotype=protein_coding')) & (refseq_38_df['NC'].str.contains('NC_'))]
refseq_38_df_pc_genes

,NC,RefSeq_validation,Region_type,Start,End,Score,Strand,Phase,Attributes
43,NC_000001.11,BestRefSeq,gene,65419,71585,.,+,.,"ID=gene-OR4F5;Dbxref=GeneID:79501,HGNC:HGNC:14825;Name=OR4F5;description=olfactory receptor family 4 subfamily F member 5;gbkey=Gene;gene=OR4F5;gene_biotype=protein_coding"
181,NC_000001.11,Gnomon,gene,350706,476822,.,-,.,ID=gene-LOC112268260;Dbxref=GeneID:112268260;Name=LOC112268260;gbkey=Gene;gene=LOC112268260;gene_biotype=protein_coding
243,NC_000001.11,BestRefSeq,gene,450740,451678,.,-,.,"ID=gene-OR4F29;Dbxref=GeneID:729759,HGNC:HGNC:31275;Name=OR4F29;description=olfactory receptor family 4 subfamily F member 29;gbkey=Gene;gene=OR4F29;gene_biotype=protein_coding;gene_synonym=OR7-21"
255,NC_000001.11,Gnomon,gene,586287,611297,.,-,.,ID=gene-LOC105378947;Dbxref=GeneID:105378947;Name=LOC105378947;gbkey=Gene;gene=LOC105378947;gene_biotype=protein_coding
287,NC_000001.11,BestRefSeq%2CGnomon,gene,683910,720115,.,-,.,"ID=gene-OR4F16;Dbxref=GeneID:81399,HGNC:HGNC:15079;Name=OR4F16;description=olfactory receptor family 4 subfamily F member 16;gbkey=Gene;gene=OR4F16;gene_biotype=protein_coding;gene_synonym=OR1-1,O..."
...,...,...,...,...,...,...,...,...,...
3781963,NC_012920.1,RefSeq,gene,10470,10766,.,+,.,"ID=gene-ND4L;Dbxref=GeneID:4539,HGNC:HGNC:7460,MIM:516004;Name=ND4L;gbkey=Gene;gene=ND4L;gene_biotype=protein_coding;gene_synonym=MTND4L"
3781965,NC_012920.1,RefSeq,gene,10760,12137,.,+,.,"ID=gene-ND4;Dbxref=GeneID:4538,HGNC:HGNC:7459,MIM:516003;Name=ND4;gbkey=Gene;gene=ND4;gene_biotype=protein_coding;gene_synonym=MTND4"
3781976,NC_012920.1,RefSeq,gene,12337,14148,.,+,.,"ID=gene-ND5;Dbxref=GeneID:4540,HGNC:HGNC:7461,MIM:516005;Name=ND5;gbkey=Gene;gene=ND5;gene_biotype=protein_coding;gene_synonym=MTND5"
3781978,NC_012920.1,RefSeq,gene,14149,14673,.,-,.,"ID=gene-ND6;Dbxref=GeneID:4541,HGNC:HGNC:7462,MIM:516006;Name=ND6;gbkey=Gene;gene=ND6;gene_biotype=protein_coding;gene_synonym=MTND6"


In [19]:
refseq_38_df_pc_genes.loc[refseq_38_df_pc_genes['Attributes'].str.contains('LOC105378947')]

,NC,RefSeq_validation,Region_type,Start,End,Score,Strand,Phase,Attributes
255,NC_000001.11,Gnomon,gene,586287,611297,.,-,.,ID=gene-LOC105378947;Dbxref=GeneID:105378947;Name=LOC105378947;gbkey=Gene;gene=LOC105378947;gene_biotype=protein_coding


In [138]:
pd.options.display.max_colwidth = 200
refseq_38_df_mrna = refseq_38_df.loc[(refseq_38_df['Attributes'].str.contains('NM_'))  & (refseq_38_df['Region_type'] == 'mRNA') & (refseq_38_df['NC'].str.contains('NC_'))]
refseq_38_df_mrna

,NC,RefSeq_validation,Region_type,Start,End,Score,Strand,Phase,Attributes
44,NC_000001.11,BestRefSeq,mRNA,65419,71585,.,+,.,"ID=rna-NM_001005484.2;Parent=gene-OR4F5;Dbxref=GeneID:79501,Genbank:NM_001005484.2,HGNC:HGNC:14825;Name=NM_001005484.2;gbkey=mRNA;gene=OR4F5;product=olfactory receptor family 4 subfamily F member ..."
244,NC_000001.11,BestRefSeq,mRNA,450740,451678,.,-,.,"ID=rna-NM_001005221.2;Parent=gene-OR4F29;Dbxref=GeneID:729759,Genbank:NM_001005221.2,HGNC:HGNC:31275;Name=NM_001005221.2;gbkey=mRNA;gene=OR4F29;product=olfactory receptor family 4 subfamily F memb..."
321,NC_000001.11,BestRefSeq,mRNA,685716,686654,.,-,.,"ID=rna-NM_001005277.1;Parent=gene-OR4F16;Dbxref=GeneID:81399,Genbank:NM_001005277.1,HGNC:HGNC:15079;Name=NM_001005277.1;gbkey=mRNA;gene=OR4F16;product=olfactory receptor family 4 subfamily F membe..."
453,NC_000001.11,BestRefSeq,mRNA,923923,944574,.,+,.,"ID=rna-NM_001385640.1;Parent=gene-SAMD11;Dbxref=GeneID:148398,Genbank:NM_001385640.1,HGNC:HGNC:28706,MIM:616765;Name=NM_001385640.1;gbkey=mRNA;gene=SAMD11;product=sterile alpha motif domain contai..."
482,NC_000001.11,BestRefSeq,mRNA,923923,944574,.,+,.,"ID=rna-NM_001385641.1;Parent=gene-SAMD11;Dbxref=GeneID:148398,Genbank:NM_001385641.1,HGNC:HGNC:28706,MIM:616765;Name=NM_001385641.1;gbkey=mRNA;gene=SAMD11;product=sterile alpha motif domain contai..."
...,...,...,...,...,...,...,...,...,...
3601877,NC_000024.10,BestRefSeq,mRNA,57067865,57130289,.,+,.,"ID=rna-NM_005638.6-2;Parent=gene-VAMP7-2;Dbxref=GeneID:6845,Genbank:NM_005638.6,HGNC:HGNC:11486,MIM:300053;Name=NM_005638.6;gbkey=mRNA;gene=VAMP7;product=vesicle associated membrane protein 7%2C t..."
3601893,NC_000024.10,BestRefSeq,mRNA,57067865,57130289,.,+,.,"ID=rna-NM_001185183.2-2;Parent=gene-VAMP7-2;Dbxref=GeneID:6845,Genbank:NM_001185183.2,HGNC:HGNC:11486,MIM:300053;Name=NM_001185183.2;gbkey=mRNA;gene=VAMP7;product=vesicle associated membrane prote..."
3601907,NC_000024.10,BestRefSeq,mRNA,57067865,57130289,.,+,.,"ID=rna-NM_001145149.3-2;Parent=gene-VAMP7-2;Dbxref=GeneID:6845,Genbank:NM_001145149.3,HGNC:HGNC:11486,MIM:300053;Name=NM_001145149.3;gbkey=mRNA;gene=VAMP7;product=vesicle associated membrane prote..."
3602194,NC_000024.10,BestRefSeq,mRNA,57184216,57197337,.,+,.,"ID=rna-NM_176786.2-2;Parent=gene-IL9R-2;Dbxref=GeneID:3581,Genbank:NM_176786.2,HGNC:HGNC:6030,MIM:300007;Name=NM_176786.2;gbkey=mRNA;gene=IL9R;product=interleukin 9 receptor%2C transcript variant ..."


In [139]:
pd.options.display.max_colwidth = 200
pd.options.display.max_rows = 20
refseq_38_df_exons = refseq_38_df.loc[(refseq_38_df['Attributes'].str.contains('exon-NM'))  & (refseq_38_df['Region_type'] == 'exon')  & (refseq_38_df['NC'].str.contains('NC_'))]
refseq_38_df_exons

,NC,RefSeq_validation,Region_type,Start,End,Score,Strand,Phase,Attributes
45,NC_000001.11,BestRefSeq,exon,65419,65433,.,+,.,"ID=exon-NM_001005484.2-1;Parent=rna-NM_001005484.2;Dbxref=GeneID:79501,Genbank:NM_001005484.2,HGNC:HGNC:14825;gbkey=mRNA;gene=OR4F5;product=olfactory receptor family 4 subfamily F member 5;tag=Ref..."
46,NC_000001.11,BestRefSeq,exon,65520,65573,.,+,.,"ID=exon-NM_001005484.2-2;Parent=rna-NM_001005484.2;Dbxref=GeneID:79501,Genbank:NM_001005484.2,HGNC:HGNC:14825;gbkey=mRNA;gene=OR4F5;product=olfactory receptor family 4 subfamily F member 5;tag=Ref..."
47,NC_000001.11,BestRefSeq,exon,69037,71585,.,+,.,"ID=exon-NM_001005484.2-3;Parent=rna-NM_001005484.2;Dbxref=GeneID:79501,Genbank:NM_001005484.2,HGNC:HGNC:14825;gbkey=mRNA;gene=OR4F5;product=olfactory receptor family 4 subfamily F member 5;tag=Ref..."
245,NC_000001.11,BestRefSeq,exon,450740,451678,.,-,.,"ID=exon-NM_001005221.2-1;Parent=rna-NM_001005221.2;Dbxref=GeneID:729759,Genbank:NM_001005221.2,HGNC:HGNC:31275;gbkey=mRNA;gene=OR4F29;product=olfactory receptor family 4 subfamily F member 29;tag=..."
322,NC_000001.11,BestRefSeq,exon,685716,686654,.,-,.,"ID=exon-NM_001005277.1-1;Parent=rna-NM_001005277.1;Dbxref=GeneID:81399,Genbank:NM_001005277.1,HGNC:HGNC:15079;gbkey=mRNA;gene=OR4F16;product=olfactory receptor family 4 subfamily F member 16;tag=R..."
...,...,...,...,...,...,...,...,...,...
3602218,NC_000024.10,BestRefSeq,exon,57190940,57191085,.,+,.,"ID=exon-NM_002186.3-2-5;Parent=rna-NM_002186.3-2;Dbxref=GeneID:3581,Genbank:NM_002186.3,HGNC:HGNC:6030,MIM:300007;gbkey=mRNA;gene=IL9R;product=interleukin 9 receptor%2C transcript variant 1;transc..."
3602219,NC_000024.10,BestRefSeq,exon,57191798,57191999,.,+,.,"ID=exon-NM_002186.3-2-6;Parent=rna-NM_002186.3-2;Dbxref=GeneID:3581,Genbank:NM_002186.3,HGNC:HGNC:6030,MIM:300007;gbkey=mRNA;gene=IL9R;product=interleukin 9 receptor%2C transcript variant 1;transc..."
3602220,NC_000024.10,BestRefSeq,exon,57192603,57192708,.,+,.,"ID=exon-NM_002186.3-2-7;Parent=rna-NM_002186.3-2;Dbxref=GeneID:3581,Genbank:NM_002186.3,HGNC:HGNC:6030,MIM:300007;gbkey=mRNA;gene=IL9R;product=interleukin 9 receptor%2C transcript variant 1;transc..."
3602221,NC_000024.10,BestRefSeq,exon,57194043,57194127,.,+,.,"ID=exon-NM_002186.3-2-8;Parent=rna-NM_002186.3-2;Dbxref=GeneID:3581,Genbank:NM_002186.3,HGNC:HGNC:6030,MIM:300007;gbkey=mRNA;gene=IL9R;product=interleukin 9 receptor%2C transcript variant 1;transc..."


In [140]:
pd.options.display.max_colwidth = 200
refseq_38_df_cds = refseq_38_df.loc[(refseq_38_df['Attributes'].str.contains('NP_'))  & (refseq_38_df['Region_type'] == 'CDS')  & (refseq_38_df['NC'].str.contains('NC_'))]
refseq_38_df_cds

,NC,RefSeq_validation,Region_type,Start,End,Score,Strand,Phase,Attributes
48,NC_000001.11,BestRefSeq,CDS,65565,65573,.,+,0,"ID=cds-NP_001005484.2;Parent=rna-NM_001005484.2;Dbxref=CCDS:CCDS30547.1,GeneID:79501,Genbank:NP_001005484.2,HGNC:HGNC:14825;Name=NP_001005484.2;gbkey=CDS;gene=OR4F5;product=olfactory receptor 4F5;..."
49,NC_000001.11,BestRefSeq,CDS,69037,70008,.,+,0,"ID=cds-NP_001005484.2;Parent=rna-NM_001005484.2;Dbxref=CCDS:CCDS30547.1,GeneID:79501,Genbank:NP_001005484.2,HGNC:HGNC:14825;Name=NP_001005484.2;gbkey=CDS;gene=OR4F5;product=olfactory receptor 4F5;..."
246,NC_000001.11,BestRefSeq,CDS,450740,451678,.,-,0,"ID=cds-NP_001005221.2;Parent=rna-NM_001005221.2;Dbxref=CCDS:CCDS72675.1,GeneID:729759,Genbank:NP_001005221.2,HGNC:HGNC:31275;Name=NP_001005221.2;gbkey=CDS;gene=OR4F29;product=olfactory receptor 4F..."
323,NC_000001.11,BestRefSeq,CDS,685716,686654,.,-,0,"ID=cds-NP_001005277.1;Parent=rna-NM_001005277.1;Dbxref=CCDS:CCDS41221.1,GeneID:81399,Genbank:NP_001005277.1,HGNC:HGNC:15079;Name=NP_001005277.1;gbkey=CDS;gene=OR4F16;product=olfactory receptor 4F3..."
468,NC_000001.11,BestRefSeq,CDS,924432,924948,.,+,0,"ID=cds-NP_001372569.1;Parent=rna-NM_001385640.1;Dbxref=GeneID:148398,Genbank:NP_001372569.1,HGNC:HGNC:28706,MIM:616765;Name=NP_001372569.1;Note=isoform 2 is encoded by transcript variant 2;gbkey=C..."
...,...,...,...,...,...,...,...,...,...
3602227,NC_000024.10,BestRefSeq,CDS,57190940,57191085,.,+,2,"ID=cds-NP_002177.2-2;Parent=rna-NM_002186.3-2;Dbxref=CCDS:CCDS14771.4,GeneID:3581,Genbank:NP_002177.2,HGNC:HGNC:6030,MIM:300007;Name=NP_002177.2;Note=isoform 1 precursor is encoded by transcript v..."
3602228,NC_000024.10,BestRefSeq,CDS,57191798,57191999,.,+,0,"ID=cds-NP_002177.2-2;Parent=rna-NM_002186.3-2;Dbxref=CCDS:CCDS14771.4,GeneID:3581,Genbank:NP_002177.2,HGNC:HGNC:6030,MIM:300007;Name=NP_002177.2;Note=isoform 1 precursor is encoded by transcript v..."
3602229,NC_000024.10,BestRefSeq,CDS,57192603,57192708,.,+,2,"ID=cds-NP_002177.2-2;Parent=rna-NM_002186.3-2;Dbxref=CCDS:CCDS14771.4,GeneID:3581,Genbank:NP_002177.2,HGNC:HGNC:6030,MIM:300007;Name=NP_002177.2;Note=isoform 1 precursor is encoded by transcript v..."
3602230,NC_000024.10,BestRefSeq,CDS,57194043,57194127,.,+,1,"ID=cds-NP_002177.2-2;Parent=rna-NM_002186.3-2;Dbxref=CCDS:CCDS14771.4,GeneID:3581,Genbank:NP_002177.2,HGNC:HGNC:6030,MIM:300007;Name=NP_002177.2;Note=isoform 1 precursor is encoded by transcript v..."


In [141]:
from tqdm import tqdm
l = list()


# for j, index in tqdm(enumerate(list(refseq_38_df_pc_genes.index))):

def test_mp(t, l):
    j, index = t[0], t[1]
#     if j == 10:
#         break

    if j < len(list(refseq_38_df_pc_genes.index)) - 1 :
        start_index, stop_index = index, list(refseq_38_df_pc_genes.index)[j+1]

        current_gene = [e.replace('Name=', '') for e in refseq_38_df_pc_genes.loc[start_index]['Attributes'].split(';') if 'Name' in e][0]
        for e, cds in enumerate(list(refseq_38_df_exons.index)):
            if cds > start_index and cds < stop_index:

                current_gene_cds = [e.replace('gene=', '') for e in refseq_38_df_exons.loc[cds]['Attributes'].split(';') if 'gene' in e][0]
                parent_mrna = [e.replace('Parent=rna-', '') for e in refseq_38_df_exons.loc[cds]['Attributes'].split(';') if 'Parent' in e][0].split('.')[0]
                mrna_index = refseq_38_df_mrna.loc[refseq_38_df_mrna['Attributes'].str.contains(parent_mrna)].index[0]
                
                

#                 print(j, start_index, stop_index, cds, current_gene, current_gene_cds)

                if current_gene == current_gene_cds:
                    l.append(
                        {
                            'Gene' : current_gene,
                            'Gene_start' : refseq_38_df_pc_genes.loc[start_index]['Start'],
                            'Gene_stop' : refseq_38_df_pc_genes.loc[start_index]['End'],
                            'Exon_start' : refseq_38_df_exons.loc[cds]['Start'],
                            'Exon_stop' : refseq_38_df_exons.loc[cds]['End'],
                            'mRNA' : parent_mrna,
                            'mRNA_start' : refseq_38_df_mrna.loc[mrna_index]['Start'],
                            'mRNA_stop' : refseq_38_df_mrna.loc[mrna_index]['End'],
                            'Exon_type' : 'Exon',
                        }
                    )

m = multiprocessing.Manager()
l = m.list()
parmap.starmap(test_mp, list(zip(enumerate(list(refseq_38_df_pc_genes.index)))), l, pm_pbar=True)
            
            
pd.options.display.max_rows = 20 
df = pd.DataFrame(list(l)).sort_values(by=['Gene', 'Exon_start', 'Exon_stop']).drop_duplicates()
df['Length'] = df['Exon_stop'] - df['Exon_start']
df = df.sort_values(by=['Gene', 'Exon_start', 'Exon_stop', 'Length'], ascending=[True, True, True, False])
# df = df.drop_duplicates(subset=['Gene', 'Exon_start'], keep='last').drop_duplicates(subset=['Gene', 'Exon_stop'], keep='first')
df.to_parquet('/gstock/EXOTIC/data/GENOMICS/refseq_38_tmp_exons_part.parquet')
df


19712it [27:39, 11.88it/s]                            


,Gene,Gene_start,Gene_stop,Exon_start,Exon_stop,mRNA,mRNA_start,mRNA_stop,Exon_type,Length
584613,A1BG,58345183,58353492,58345183,58347029,NM_130786,58345183,58353492,Exon,1846
584535,A1BG,58345183,58353492,58347353,58347640,NM_130786,58345183,58353492,Exon,287
584489,A1BG,58345183,58353492,58350370,58350651,NM_130786,58345183,58353492,Exon,281
584410,A1BG,58345183,58353492,58351391,58351687,NM_130786,58345183,58353492,Exon,296
584322,A1BG,58345183,58353492,58352283,58352555,NM_130786,58345183,58353492,Exon,272
...,...,...,...,...,...,...,...,...,...,...
65476,ZZZ3,77561526,77683440,77683262,77683398,NM_001376147,77562416,77683398,Exon,136
66281,ZZZ3,77561526,77683440,77683262,77683398,NM_001376156,77562416,77683398,Exon,136
67143,ZZZ3,77561526,77683440,77683262,77683398,NM_001308237,77562416,77683398,Exon,136
67897,ZZZ3,77561526,77683440,77683262,77683398,NM_001376148,77562416,77683398,Exon,136


In [142]:
from tqdm import tqdm
l = list()


# for j, index in tqdm(enumerate(list(refseq_38_df_pc_genes.index))):

def test_mp(t, l):
    j, index = t[0], t[1]
#     if j == 10:
#         break

    if j < len(list(refseq_38_df_pc_genes.index)) - 1 :
        start_index, stop_index = index, list(refseq_38_df_pc_genes.index)[j+1]

        current_gene = [e.replace('Name=', '') for e in refseq_38_df_pc_genes.loc[start_index]['Attributes'].split(';') if 'Name' in e][0]
        for cds in list(refseq_38_df_cds.index):
            if cds > start_index and cds < stop_index:

                current_gene_cds = [e.replace('gene=', '') for e in refseq_38_df_cds.loc[cds]['Attributes'].split(';') if 'gene' in e][0]
                parent_mrna = [e.replace('Parent=rna-', '') for e in refseq_38_df_cds.loc[cds]['Attributes'].split(';') if 'Parent' in e][0].split('.')[0]
                mrna_index = refseq_38_df_mrna.loc[refseq_38_df_mrna['Attributes'].str.contains(parent_mrna)].index[0]
                
                

#                 print(j, start_index, stop_index, cds, current_gene, current_gene_cds)

                if current_gene == current_gene_cds:
                    l.append(
                        {
                            'Gene' : current_gene,
                            'Gene_start' : refseq_38_df_pc_genes.loc[start_index]['Start'],
                            'Gene_stop' : refseq_38_df_pc_genes.loc[start_index]['End'],
                            'Exon_start' : refseq_38_df_cds.loc[cds]['Start'],
                            'Exon_stop' : refseq_38_df_cds.loc[cds]['End'],
                            'mRNA' : parent_mrna,
                            'mRNA_start' : refseq_38_df_mrna.loc[mrna_index]['Start'],
                            'mRNA_stop' : refseq_38_df_mrna.loc[mrna_index]['End'],
                            'Exon_type' : 'CDS',
                        }
                    )

m = multiprocessing.Manager()
l = m.list()
parmap.starmap(test_mp, list(zip(enumerate(list(refseq_38_df_pc_genes.index)))), l, pm_pbar=True)
            
            
pd.options.display.max_rows = 20
df = pd.DataFrame(list(l)).sort_values(by=['Gene', 'Exon_start', 'Exon_stop']).drop_duplicates()
df['Length'] = df['Exon_stop'] - df['Exon_start']
df = df.sort_values(by=['Gene', 'Exon_start', 'Exon_stop', 'Length'], ascending=[True, True, True, False])
# df = df.drop_duplicates(subset=['Gene', 'Exon_start'], keep='last').drop_duplicates(subset=['Gene', 'Exon_stop'], keep='first')
df.to_parquet('/gstock/EXOTIC/data/GENOMICS/refseq_38_tmp_cds_part.parquet')
df


19712it [24:26, 13.44it/s]                           


,Gene,Gene_start,Gene_stop,Exon_start,Exon_stop,mRNA,mRNA_start,mRNA_stop,Exon_type,Length
522102,A1BG,58345183,58353492,58347022,58347029,NM_130786,58345183,58353492,CDS,7
522020,A1BG,58345183,58353492,58347353,58347640,NM_130786,58345183,58353492,CDS,287
521971,A1BG,58345183,58353492,58350370,58350651,NM_130786,58345183,58353492,CDS,281
521888,A1BG,58345183,58353492,58351391,58351687,NM_130786,58345183,58353492,CDS,296
521791,A1BG,58345183,58353492,58352283,58352555,NM_130786,58345183,58353492,CDS,272
...,...,...,...,...,...,...,...,...,...,...
57500,ZZZ3,77561526,77683440,77631850,77633354,NM_001376154,77562416,77682658,CDS,1504
58190,ZZZ3,77561526,77683440,77631850,77633354,NM_015534,77562416,77682658,CDS,1504
54318,ZZZ3,77561526,77683440,77639449,77639471,NM_001376156,77562416,77683398,CDS,22
54733,ZZZ3,77561526,77683440,77639449,77639471,NM_001308237,77562416,77683398,CDS,22


In [8]:
concat_df_exons_cds = pd.concat(
    [
        pd.read_parquet('/gstock/EXOTIC/data/GENOMICS/refseq_38_tmp_exons_part.parquet'),
        pd.read_parquet('/gstock/EXOTIC/data/GENOMICS/refseq_38_tmp_cds_part.parquet')
    ]    
).sort_values(by=['Gene', 'Exon_start', 'Exon_stop'])
concat_df_exons_cds

,Gene,Gene_start,Gene_stop,Exon_start,Exon_stop,mRNA,mRNA_start,mRNA_stop,Exon_type,Length
584613,A1BG,58345183,58353492,58345183,58347029,NM_130786,58345183,58353492,Exon,1846
522102,A1BG,58345183,58353492,58347022,58347029,NM_130786,58345183,58353492,CDS,7
584535,A1BG,58345183,58353492,58347353,58347640,NM_130786,58345183,58353492,Exon,287
522020,A1BG,58345183,58353492,58347353,58347640,NM_130786,58345183,58353492,CDS,287
584489,A1BG,58345183,58353492,58350370,58350651,NM_130786,58345183,58353492,Exon,281
...,...,...,...,...,...,...,...,...,...,...
65476,ZZZ3,77561526,77683440,77683262,77683398,NM_001376147,77562416,77683398,Exon,136
66281,ZZZ3,77561526,77683440,77683262,77683398,NM_001376156,77562416,77683398,Exon,136
67143,ZZZ3,77561526,77683440,77683262,77683398,NM_001308237,77562416,77683398,Exon,136
67897,ZZZ3,77561526,77683440,77683262,77683398,NM_001376148,77562416,77683398,Exon,136


In [145]:
concat_df_exons_cds.to_parquet('/gstock/EXOTIC/data/GENOMICS/refseq_38_tmp_exons_cds.parquet')

In [2]:
concat_df_exons_cds = pd.read_parquet('/gstock/EXOTIC/data/GENOMICS/refseq_38_tmp_exons_cds.parquet')

In [9]:
concat_df_exons_cds

,Gene,Gene_start,Gene_stop,Exon_start,Exon_stop,mRNA,mRNA_start,mRNA_stop,Exon_type,Length
584613,A1BG,58345183,58353492,58345183,58347029,NM_130786,58345183,58353492,Exon,1846
522102,A1BG,58345183,58353492,58347022,58347029,NM_130786,58345183,58353492,CDS,7
584535,A1BG,58345183,58353492,58347353,58347640,NM_130786,58345183,58353492,Exon,287
522020,A1BG,58345183,58353492,58347353,58347640,NM_130786,58345183,58353492,CDS,287
584489,A1BG,58345183,58353492,58350370,58350651,NM_130786,58345183,58353492,Exon,281
...,...,...,...,...,...,...,...,...,...,...
65476,ZZZ3,77561526,77683440,77683262,77683398,NM_001376147,77562416,77683398,Exon,136
66281,ZZZ3,77561526,77683440,77683262,77683398,NM_001376156,77562416,77683398,Exon,136
67143,ZZZ3,77561526,77683440,77683262,77683398,NM_001308237,77562416,77683398,Exon,136
67897,ZZZ3,77561526,77683440,77683262,77683398,NM_001376148,77562416,77683398,Exon,136


In [65]:
df.to_parquet('/gstock/EXOTIC/data/GENOMICS/refseq_38_tmp.parquet')

In [3]:
df = pd.read_parquet('/gstock/EXOTIC/data/GENOMICS/refseq_38_tmp.parquet')
df['Exon_start'] = df['Exon_start'].astype(str)
df['Exon_stop'] = df['Exon_stop'].astype(str)

In [7]:
df.sort_values(by=['Gene', 'Gene_start', 'Exon_start'])

,Gene,Gene_start,Gene_stop,Exon_start,Exon_stop,mRNA,mRNA_start,mRNA_stop,Length
519971,A1BG,58345183,58353492,58347022,58347029,NM_130786,58345183,58353492,7
519915,A1BG,58345183,58353492,58347353,58347640,NM_130786,58345183,58353492,287
519829,A1BG,58345183,58353492,58350370,58350651,NM_130786,58345183,58353492,281
519757,A1BG,58345183,58353492,58351391,58351687,NM_130786,58345183,58353492,296
519689,A1BG,58345183,58353492,58352283,58352555,NM_130786,58345183,58353492,272
...,...,...,...,...,...,...,...,...,...
68870,ZZZ3,77561526,77683440,77631850,77633354,NM_001376154,77562416,77682658,1504
69447,ZZZ3,77561526,77683440,77631850,77633354,NM_015534,77562416,77682658,1504
64538,ZZZ3,77561526,77683440,77639449,77639471,NM_001376156,77562416,77683398,22
65286,ZZZ3,77561526,77683440,77639449,77639471,NM_001308237,77562416,77683398,22


In [22]:
df.groupby(['Gene', 'mRNA'])['Exon_start'].apply(lambda r: ','.join(r))

Gene    mRNA        
A1BG    NM_130786       58347022,58347353,58350370,58351391,58352283,5...
A1CF    NM_001198818                                             50859842
        NM_001198819    50806729,50809894,50811040,50813857,50816006,5...
A2M     NM_000014                                 9112377,9113360,9115764
        NM_001347424    9067823,9068183,9068740,9069745,9070488,907235...
                                              ...                        
ZYG11B  NM_024646       52726654,52756458,52771020,52779853,52784877,5...
ZYX     NM_001010972    143381572,143382248,143382593,143382801,143388...
ZZEF1   NM_015113       4006890,4008883,4009604,4013449,4014090,401434...
ZZZ3    NM_001376155                                             77639449
        NM_015534       77565640,77566081,77568332,77576068,77578774,7...
Name: Exon_start, Length: 31980, dtype: object

In [24]:
df.groupby(['Gene', 'mRNA'])['Exon_stop'].apply(lambda r: ','.join(r))

Gene    mRNA        
A1BG    NM_130786       58347029,58347640,58350651,58351687,58352555,5...
A1CF    NM_001198818                                             50859940
        NM_001198819    50806880,50810042,50811176,50814062,50816279,5...
A2M     NM_000014                                 9112536,9113543,9115849
        NM_001347424    9067839,9068224,9068842,9069813,9070578,907248...
                                              ...                        
ZYG11B  NM_024646       52726683,52756623,52771774,52779993,52785053,5...
ZYX     NM_001010972    143381779,143382447,143382685,143383322,143388...
ZZEF1   NM_015113       4006970,4008954,4009757,4013614,4014188,401451...
ZZZ3    NM_001376155                                             77639471
        NM_015534       77565784,77566181,77568466,77576220,77578869,7...
Name: Exon_stop, Length: 31980, dtype: object

In [11]:
concat_exons.reset_index().mRNA.nunique()

59150

In [4]:
concat_exons = pd.concat([df.groupby(['Gene', 'mRNA'])['Exon_start'].apply(lambda r: ','.join(r)), df.groupby(['Gene', 'mRNA'])['Exon_stop'].apply(lambda r: ','.join(r))], axis=1)
concat_exons['Exon_list'] = concat_exons[['Exon_start', 'Exon_stop']].apply(lambda r: [start + '_' + stop for start, stop in zip(r['Exon_start'].split(','), r['Exon_stop'].split(','))], axis=1)
concat_exons 



Exon_start  \
Gene mRNA                                                              
A1BG NM_130786     58347022,58347353,58350370,58351391,58352283,5...   
A1CF NM_001198818  50806729,50809894,50811040,50813857,50816006,5...   
     NM_001198819  50806729,50809894,50811040,50813857,50816006,5...   
     NM_001198820  50806729,50809894,50811040,50813857,50816006,5...   
     NM_001370130  50806729,50809894,50811040,50813857,50816006,5...   
...                                                              ...   
ZZZ3 NM_001376153  77565640,77566081,77568332,77576068,77578774,7...   
     NM_001376154  77565640,77566081,77568332,77576068,77578774,7...   
     NM_001376155  77565640,77566081,77568332,77576068,77578774,7...   
     NM_001376156  77565640,77566081,77568332,77576068,77578774,7...   
     NM_015534     77565640,77566081,77568332,77576068,77578774,7...   

                                                           Exon_stop  \
Gene mRNA                                                              
A1BG NM_130786     58347029,58347640,58350651,58351687,58352555,5...   
A1CF NM_001198818  50806880,50810042,50811176,50814038,50816279,5...   
     NM_001198819  50806880,50810042,50811176,50814062,50816279,5...   
     NM_001198820  50806880,50810042,50811176,50814038,50816279,5...   
     NM_001370130  50806880,50810042,50811176,50814038,50816279,5...   
...                                                              ...   
ZZZ3 NM_001376153  77565784,77566181,77568466,77576220,77578869,7...   
     NM_001376154  77565784,77566181,77568466,77576220,77578869,7...   
     NM_001376155  77565784,77566181,77568466,77576220,77578869,7...   
     NM_001376156  77565784,77566181,77568466,77576220,77578869,7...   
     NM_015534     77565784,77566181,77568466,77576220,77578869,7...   

                                                           Exon_list  
Gene mRNA                                                             
A1BG NM_130786     [58347022_58347029, 58347353_58347640, 5835037...  
A1CF NM_001198818  [50806729_50806880, 50809894_50810042, 5081104...  
     NM_001198819  [50806729_50806880, 50809894_50810042, 5081104...  
     NM_001198820  [50806729_50806880, 50809894_50810042, 5081104...  
     NM_001370130  [50806729_50806880, 50809894_50810042, 5081104...  
...                                                              ...  
ZZZ3 NM_001376153  [77565640_77565784, 77566081_77566181, 7756833...  
     NM_001376154  [77565640_77565784, 77566081_77566181, 7756833...  
     NM_001376155  [77565640_77565784, 77566081_77566181, 7756833...  
     NM_001376156  [77565640_77565784, 77566081_77566181, 7756833...  
     NM_015534     [77565640_77565784, 77566081_77566181, 7756833...  

[59150 rows x 3 columns]

In [48]:
concat_exons.reset_index().groupby('Gene')['mRNA'].apply(list)


Gene
A1BG                        [NM_130786]
A1CF       [NM_001198818, NM_001198819]
A2M           [NM_000014, NM_001347424]
A2ML1         [NM_001282424, NM_144670]
A3GALT2                  [NM_001080438]
                       ...             
ZYG11A     [NM_001004339, NM_001307931]
ZYG11B                      [NM_024646]
ZYX                      [NM_001010972]
ZZEF1                       [NM_015113]
ZZZ3          [NM_001376155, NM_015534]
Name: mRNA, Length: 19344, dtype: object

In [49]:
concat_exons.reset_index().groupby('Gene')['Exon_list'].apply(list)

Gene
A1BG       [[58347022_58347029, 58347353_58347640, 583503...
A1CF       [[50859842_50859940], [50806729_50806880, 5080...
A2M        [[9112377_9112536, 9113360_9113543, 9115764_91...
A2ML1      [[8845172_8845174, 8845442_8845502, 8846077_88...
A3GALT2    [[33306766_33307453, 33312052_33312189, 333125...
                                 ...                        
ZYG11A     [[52842884_52842973, 52854465_52854630, 528569...
ZYG11B     [[52726654_52726683, 52756458_52756623, 527710...
ZYX        [[143381572_143381779, 143382248_143382447, 14...
ZZEF1      [[4006890_4006970, 4008883_4008954, 4009604_40...
ZZZ3       [[77639449_77639471], [77565640_77565784, 7756...
Name: Exon_list, Length: 19344, dtype: object

In [14]:
concat_exons_gene_level = pd.concat(
    [
        concat_exons.reset_index().groupby('Gene')['mRNA'].apply(list),
        concat_exons.reset_index().groupby('Gene')['Exon_list'].apply(list)
    ],
    axis=1
)
concat_exons_gene_level = concat_exons_gene_level.reset_index()
concat_exons_gene_level

,Gene,mRNA,Exon_list
0,A1BG,[NM_130786],"[[58347022_58347029, 58347353_58347640, 583503..."
1,A1CF,"[NM_001198818, NM_001198819, NM_001198820, NM_...","[[50806729_50806880, 50809894_50810042, 508110..."
2,A2M,"[NM_000014, NM_001347423, NM_001347424, NM_001...","[[9067823_9067839, 9068183_9068224, 9068740_90..."
3,A2ML1,"[NM_001282424, NM_144670]","[[8845172_8845174, 8845442_8845502, 8846077_88..."
4,A3GALT2,[NM_001080438],"[[33306766_33307453, 33312052_33312189, 333125..."
...,...,...,...
19339,ZYG11A,"[NM_001004339, NM_001307931]","[[52842884_52842973, 52854465_52854630, 528569..."
19340,ZYG11B,[NM_024646],"[[52726654_52726683, 52756458_52756623, 527710..."
19341,ZYX,"[NM_001010972, NM_001362783, NM_003461]","[[143381572_143381779, 143382248_143382447, 14..."
19342,ZZEF1,[NM_015113],"[[4006890_4006970, 4008883_4008954, 4009604_40..."


In [111]:
concat_exons_gene_level.head()

,Gene,mRNA,Exon_list,Intersection,mRNA_nb,Exon_list_nb,Exon_list_set,Check_at_least_two,Check_common_nb
0,A1BG,[NM_130786],"[[58347022_58347029, 58347353_58347640, 58350370_58350651, 58351391_58351687, 58352283_58352555, 58352928_58353197, 58353292_58353327, 58353404_58353437]]","{58353404_58353437, 58352928_58353197, 58350370_58350651, 58347353_58347640, 58347022_58347029, 58351391_58351687, 58353292_58353327, 58352283_58352555}",1,[8],1,False,False
1,A1CF,"[NM_001198818, NM_001198819, NM_001198820, NM_001370130, NM_001370131, NM_014576, NM_138932, NM_138933]","[[50806729_50806880, 50809894_50810042, 50811040_50811176, 50813857_50814038, 50816006_50816279, 50820552_50820649, 50828131_50828295, 50836074_50836312, 50841862_50841992, 50843988_50844122, 5085...","{50809894_50810042, 50828131_50828295, 50811040_50811176, 50841862_50841992, 50820552_50820649, 50816006_50816279, 50843988_50844122, 50836074_50836312, 50806729_50806880}",8,"[11, 11, 11, 11, 11, 11, 11, 11]",1,True,True
2,A2M,"[NM_000014, NM_001347423, NM_001347424, NM_001347425]","[[9067823_9067839, 9068183_9068224, 9068740_9068842, 9069745_9069813, 9070488_9070578, 9072359_9072486, 9072653_9072871, 9074560_9074783, 9076756_9076936, 9077346_9077420, 9077701_9077857, 9079244...","{9109321_9109405, 9069745_9069813, 9107524_9107644, 9110314_9110334, 9093465_9093579, 9101447_9101674, 9080094_9080177, 9077701_9077857, 9106491_9106605, 9068183_9068224, 9079639_9079815, 9072359_...",4,"[36, 36, 34, 33]",3,True,True
3,A2ML1,"[NM_001282424, NM_144670]","[[8845172_8845174, 8845442_8845502, 8846077_8846222, 8847549_8847698, 8848720_8848914, 8849669_8849759, 8850160_8850274, 8851784_8852012, 8852210_8852336, 8854128_8854249, 8854780_8854831, 8855509...","{8874971_8875011, 8857164_8857340, 8849669_8849759, 8846077_8846222, 8847549_8847698, 8874425_8874527, 8857507_8857588, 8850160_8850274, 8855509_8855592, 8857946_8858102, 8848720_8848914, 8851784_...",2,"[24, 35]",2,False,False
4,A3GALT2,[NM_001080438],"[[33306766_33307453, 33312052_33312189, 33312501_33312590, 33312807_33312890, 33321076_33321098]]","{33312807_33312890, 33306766_33307453, 33312501_33312590, 33321076_33321098, 33312052_33312189}",1,[5],1,False,False


In [20]:
concat_exons_gene_level

,Gene,mRNA,Exon_list,Intersection,mRNA_nb,Exon_list_nb,Exon_list_set,Check_at_least_two,Check_common_nb
0,A1BG,[NM_130786],"[[58347022_58347029, 58347353_58347640, 583503...","{58353292_58353327, 58347022_58347029, 5835228...",1,[8],1,False,False
1,A1CF,"[NM_001198818, NM_001198819, NM_001198820, NM_...","[[50806729_50806880, 50809894_50810042, 508110...","{50809894_50810042, 50828131_50828295, 5082055...",8,"[11, 11, 11, 11, 11, 11, 11, 11]",1,True,True
2,A2M,"[NM_000014, NM_001347423, NM_001347424, NM_001...","[[9067823_9067839, 9068183_9068224, 9068740_90...","{9099381_9099523, 9068740_9068842, 9079639_907...",4,"[36, 36, 34, 33]",3,True,True
3,A2ML1,"[NM_001282424, NM_144670]","[[8845172_8845174, 8845442_8845502, 8846077_88...","{8868537_8868627, 8845442_8845502, 8857507_885...",2,"[24, 35]",2,False,False
4,A3GALT2,[NM_001080438],"[[33306766_33307453, 33312052_33312189, 333125...","{33306766_33307453, 33321076_33321098, 3331205...",1,[5],1,False,False
...,...,...,...,...,...,...,...,...,...
19339,ZYG11A,"[NM_001004339, NM_001307931]","[[52842884_52842973, 52854465_52854630, 528569...","{52885833_52885894, 52877925_52877969, 5286753...",2,"[14, 11]",2,False,False
19340,ZYG11B,[NM_024646],"[[52726654_52726683, 52756458_52756623, 527710...","{52726654_52726683, 52796292_52796391, 5279673...",1,[14],1,False,False
19341,ZYX,"[NM_001010972, NM_001362783, NM_003461]","[[143381572_143381779, 143382248_143382447, 14...","{143388767_143388945, 143390578_143390682, 143...",3,"[9, 8, 9]",2,True,True
19342,ZZEF1,[NM_015113],"[[4006890_4006970, 4008883_4008954, 4009604_40...","{4049708_4049859, 4076868_4076989, 4112609_411...",1,[55],1,False,False


In [24]:
concat_exons_gene_level = pd.concat(
    [
        concat_.reset_index().groupby('Gene')['mRNA'].apply(list),
        concat_exons.reset_index().groupby('Gene')['Exon_list'].apply(list)
    ],
    axis=1
)
concat_exons_gene_level = concat_exons_gene_level.reset_index()
concat_exons_gene_level

,Gene,Gene_start,Gene_stop,Exon_start,Exon_stop,mRNA,mRNA_start,mRNA_stop,Length
519971,A1BG,58345183,58353492,58347022,58347029,NM_130786,58345183,58353492,7
519915,A1BG,58345183,58353492,58347353,58347640,NM_130786,58345183,58353492,287
519829,A1BG,58345183,58353492,58350370,58350651,NM_130786,58345183,58353492,281
519757,A1BG,58345183,58353492,58351391,58351687,NM_130786,58345183,58353492,296
519689,A1BG,58345183,58353492,58352283,58352555,NM_130786,58345183,58353492,272
...,...,...,...,...,...,...,...,...,...
68870,ZZZ3,77561526,77683440,77631850,77633354,NM_001376154,77562416,77682658,1504
69447,ZZZ3,77561526,77683440,77631850,77633354,NM_015534,77562416,77682658,1504
64538,ZZZ3,77561526,77683440,77639449,77639471,NM_001376156,77562416,77683398,22
65286,ZZZ3,77561526,77683440,77639449,77639471,NM_001308237,77562416,77683398,22


In [3]:
concat_df_exons_cds['Range'] = concat_df_exons_cds['Exon_start'].astype(str) + '_' + concat_df_exons_cds['Exon_stop'].astype(str) 

In [4]:
mrna_count = pd.DataFrame(concat_df_exons_cds.groupby('Gene')['mRNA'].nunique())
mrna_count = mrna_count.loc[mrna_count['mRNA'] > 1]
mrna_count

,mRNA
Gene,
A1CF,8
A2M,4
A2ML1,2
A4GALT,2
AAAS,2
...,...
ZWINT,3
ZXDC,2
ZYG11A,2


In [9]:
concat_df_exons_cds.loc[(concat_df_exons_cds['Gene'] == 'A1CF') & (concat_df_exons_cds['Exon_type'] == 'CDS')]

,Gene,Gene_start,Gene_stop,Exon_start,Exon_stop,mRNA,mRNA_start,mRNA_stop,Exon_type,Length,Range
404040,A1CF,50799409,50885666,50806729,50806880,NM_001370130,50799409,50885627,CDS,151,50806729_50806880
404547,A1CF,50799409,50885666,50806729,50806880,NM_001370131,50799409,50885627,CDS,151,50806729_50806880
405195,A1CF,50799409,50885666,50806729,50806880,NM_138933,50799409,50885627,CDS,151,50806729_50806880
405835,A1CF,50799409,50885666,50806729,50806880,NM_014576,50799409,50885627,CDS,151,50806729_50806880
406486,A1CF,50799409,50885666,50806729,50806880,NM_138932,50799409,50885627,CDS,151,50806729_50806880
...,...,...,...,...,...,...,...,...,...,...,...
403438,A1CF,50799409,50885666,50859842,50859940,NM_001370130,50799409,50885627,CDS,98,50859842_50859940
404112,A1CF,50799409,50885666,50859842,50859940,NM_001370131,50799409,50885627,CDS,98,50859842_50859940
405225,A1CF,50799409,50885666,50859842,50859940,NM_014576,50799409,50885627,CDS,98,50859842_50859940
405858,A1CF,50799409,50885666,50859842,50859940,NM_138932,50799409,50885627,CDS,98,50859842_50859940


In [30]:
tmp_df = concat_df_exons_cds.loc[(concat_df_exons_cds['Gene'].isin(mrna_count.index.tolist())) & (concat_df_exons_cds['Exon_type'] == 'CDS')]
cds_join = pd.DataFrame(tmp_df.groupby(['Gene', 'mRNA'])['Range'].apply(",".join))
cds_join.columns = ['CDS_ranges']
cds_join

CDS_ranges
Gene mRNA                                                           
A1CF NM_001198818  50806729_50806880,50809894_50810042,50811040_5...
     NM_001198819  50806729_50806880,50809894_50810042,50811040_5...
     NM_001198820  50806729_50806880,50809894_50810042,50811040_5...
     NM_001370130  50806729_50806880,50809894_50810042,50811040_5...
     NM_001370131  50806729_50806880,50809894_50810042,50811040_5...
...                                                              ...
ZZZ3 NM_001376153  77565640_77565784,77566081_77566181,77568332_7...
     NM_001376154  77565640_77565784,77566081_77566181,77568332_7...
     NM_001376155  77565640_77565784,77566081_77566181,77568332_7...
     NM_001376156  77565640_77565784,77566081_77566181,77568332_7...
     NM_015534     77565640_77565784,77566081_77566181,77568332_7...

[51017 rows x 1 columns]

In [31]:
tmp_df = concat_df_exons_cds.loc[(concat_df_exons_cds['Gene'].isin(mrna_count.index.tolist())) & (concat_df_exons_cds['Exon_type'] == 'Exon')]
exon_join = pd.DataFrame(tmp_df.groupby(['Gene', 'mRNA'])['Range'].apply(",".join))
exon_join.columns = ['Exons_ranges']
exon_join

Exons_ranges
Gene mRNA                                                           
A1CF NM_001198818  50799409_50806880,50809894_50810042,50811040_5...
     NM_001198819  50799409_50806880,50809894_50810042,50811040_5...
     NM_001198820  50799409_50806880,50809894_50810042,50811040_5...
     NM_001370130  50799409_50806880,50809894_50810042,50811040_5...
     NM_001370131  50799409_50806880,50809894_50810042,50811040_5...
...                                                              ...
ZZZ3 NM_001376153  77562416_77565784,77566081_77566181,77568332_7...
     NM_001376154  77562416_77565784,77566081_77566181,77568332_7...
     NM_001376155  77562416_77565784,77566081_77566181,77568332_7...
     NM_001376156  77562416_77565784,77566081_77566181,77568332_7...
     NM_015534     77562416_77565784,77566081_77566181,77568332_7...

[51017 rows x 1 columns]

In [33]:
concat_cds_exon_join = pd.concat([exon_join, cds_join], axis=1)
concat_cds_exon_join

Exons_ranges  \
Gene mRNA                                                              
A1CF NM_001198818  50799409_50806880,50809894_50810042,50811040_5...   
     NM_001198819  50799409_50806880,50809894_50810042,50811040_5...   
     NM_001198820  50799409_50806880,50809894_50810042,50811040_5...   
     NM_001370130  50799409_50806880,50809894_50810042,50811040_5...   
     NM_001370131  50799409_50806880,50809894_50810042,50811040_5...   
...                                                              ...   
ZZZ3 NM_001376153  77562416_77565784,77566081_77566181,77568332_7...   
     NM_001376154  77562416_77565784,77566081_77566181,77568332_7...   
     NM_001376155  77562416_77565784,77566081_77566181,77568332_7...   
     NM_001376156  77562416_77565784,77566081_77566181,77568332_7...   
     NM_015534     77562416_77565784,77566081_77566181,77568332_7...   

                                                          CDS_ranges  
Gene mRNA                                                             
A1CF NM_001198818  50806729_50806880,50809894_50810042,50811040_5...  
     NM_001198819  50806729_50806880,50809894_50810042,50811040_5...  
     NM_001198820  50806729_50806880,50809894_50810042,50811040_5...  
     NM_001370130  50806729_50806880,50809894_50810042,50811040_5...  
     NM_001370131  50806729_50806880,50809894_50810042,50811040_5...  
...                                                              ...  
ZZZ3 NM_001376153  77565640_77565784,77566081_77566181,77568332_7...  
     NM_001376154  77565640_77565784,77566081_77566181,77568332_7...  
     NM_001376155  77565640_77565784,77566081_77566181,77568332_7...  
     NM_001376156  77565640_77565784,77566081_77566181,77568332_7...  
     NM_015534     77565640_77565784,77566081_77566181,77568332_7...  

[51017 rows x 2 columns]

In [114]:
merge_with_strand = pd.merge(biomart_raw[['RefSeq mRNA ID', 'Strand']].dropna(), concat_cds_exon_join.reset_index(), left_on='RefSeq mRNA ID', right_on='mRNA').drop_duplicates()
merge_with_strand

,RefSeq mRNA ID,Strand,Gene,mRNA,Exons_ranges,CDS_ranges
0,NM_001318854,1,OPRL1,NM_001318854,"64092688_64092938,64097802_64098157,64098276_6...","64092721_64092938,64097802_64098157,64098276_6..."
2,NM_001318855,1,OPRL1,NM_001318855,"64092688_64092981,64097802_64098157,64098276_6...","64092836_64092981,64097802_64098157,64098276_6..."
4,NM_001104925,1,PCMTD2,NM_001104925,"64255748_64255870,64259942_64260272,64264429_6...","64259966_64260272,64264429_64264531,64265258_6..."
6,NM_018257,1,PCMTD2,NM_018257,"64255748_64255870,64259942_64260272,64264429_6...","64259966_64260272,64264429_64264531,64265258_6..."
8,NM_023015,1,INTS3,NM_023015,"153728050_153728784,153740651_153740734,153741...","153728635_153728784,153740651_153740734,153741..."
...,...,...,...,...,...,...
54918,NM_001193328,-1,ZNF692,NM_001193328,"248850008_248850516,248850682_248850781,248853...","248850210_248850516,248850682_248850781,248853..."
54919,NM_001350073,-1,ZNF692,NM_001350073,"248850008_248850516,248850682_248850781,248853...","248850210_248850516,248850682_248850781,248853..."
54920,NM_001136036,-1,ZNF692,NM_001136036,"248850008_248850516,248850682_248850781,248853...","248850210_248850516,248850682_248850781,248853..."
54921,NM_001017434,1,PGBD2,NM_001017434,"248906235_248906342,248913816_248913879,248917...",248917338_248918363


In [57]:
cds_join

CDS_ranges
Gene mRNA                                                           
A1CF NM_001198818  50806729_50806880,50809894_50810042,50811040_5...
     NM_001198819  50806729_50806880,50809894_50810042,50811040_5...
     NM_001198820  50806729_50806880,50809894_50810042,50811040_5...
     NM_001370130  50806729_50806880,50809894_50810042,50811040_5...
     NM_001370131  50806729_50806880,50809894_50810042,50811040_5...
...                                                              ...
ZZZ3 NM_001376153  77565640_77565784,77566081_77566181,77568332_7...
     NM_001376154  77565640_77565784,77566081_77566181,77568332_7...
     NM_001376155  77565640_77565784,77566081_77566181,77568332_7...
     NM_001376156  77565640_77565784,77566081_77566181,77568332_7...
     NM_015534     77565640_77565784,77566081_77566181,77568332_7...

[51017 rows x 1 columns]

In [122]:
test = pd.DataFrame(cds_join.groupby('Gene')['CDS_ranges'].apply(lambda r: list(r))).reset_index()
test

,Gene,CDS_ranges
0,A1CF,"[50806729_50806880,50809894_50810042,50811040_..."
1,A2M,"[9067823_9067839,9068183_9068224,9068740_90688..."
2,A2ML1,"[8845172_8845174,8845442_8845502,8846077_88462..."
3,A4GALT,"[42692890_42693951, 42692890_42693951]"
4,AAAS,"[53307489_53307713,53307845_53307929,53308052_..."
...,...,...
11206,ZWINT,"[56358418_56358459,56358556_56358687,56358909_..."
11207,ZXDC,"[126461529_126462220,126466155_126466325,12647..."
11208,ZYG11A,"[52842884_52842973,52854465_52854630,52856998_..."
11209,ZYX,"[143381572_143381779,143382248_143382447,14338..."


In [26]:
test.loc['A1CF'].values.tolist()[0]

['50806729_50806880,50809894_50810042,50811040_50811176,50813857_50814038,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50859842_50859940',
 '50806729_50806880,50809894_50810042,50811040_50811176,50813857_50814062,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50850665_50850787',
 '50806729_50806880,50809894_50810042,50811040_50811176,50813857_50814038,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50850665_50850787',
 '50806729_50806880,50809894_50810042,50811040_50811176,50813857_50814038,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50859842_50859940',
 '50806729_50806880,50809894_50810042,50811040_50811176,50813857_50814062,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50859842_5

In [85]:
collections.Counter(test.loc[test['Gene'] == 'A1CF'].CDS_ranges.values.tolist()[0])

Counter({'50806729_50806880,50809894_50810042,50811040_50811176,50813857_50814038,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50859842_50859940': 3,
         '50806729_50806880,50809894_50810042,50811040_50811176,50813857_50814062,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50850665_50850787': 1,
         '50806729_50806880,50809894_50810042,50811040_50811176,50813857_50814038,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50850665_50850787': 2,
         '50806729_50806880,50809894_50810042,50811040_50811176,50813857_50814062,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50859842_50859940': 2})

In [102]:
k = "50806729_50806880,50809894_50810042,50811040_50811176,50813857_50814062,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50859842_50859940"
# pprint([set(sorted(elem.split(','))) for elem in concat_cds_exon_join.loc[concat_cds_exon_join['CDS_ranges'] == k, 'Exons_ranges'].tolist()])
exons_including_utrs = concat_cds_exon_join.loc[concat_cds_exon_join['CDS_ranges'] == k, 'Exons_ranges'].tolist()
tmp_l = [k for k,v in dict(collections.Counter([sub_e for elem in exons_including_utrs for sub_e in elem.split(',')])).items() if v < len(exons_including_utrs)]
# tmp_l[1].difference(tmp_l[0])
tmp_l
# print(set.difference(*[set(elem.split(',')) for elem in concat_cds_exon_join.loc[concat_cds_exon_join['CDS_ranges'] == k, 'Exons_ranges'].tolist()]))


['50864033_50864080']

In [93]:
tmp_l[1]

{'50799409_50806880',
 '50809894_50810042',
 '50811040_50811176',
 '50813857_50814062',
 '50816006_50816279',
 '50820552_50820649',
 '50828131_50828295',
 '50836074_50836312',
 '50841862_50841992',
 '50843988_50844122',
 '50859842_50859985',
 '50864033_50864080',
 '50885581_50885627'}

In [38]:
concat_cds_exon_join.loc[concat_cds_exon_join['CDS_ranges'] == '50806729_50806880,50809894_50810042,50811040_50811176,50813857_50814038,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50859842_50859940', 'Exons_ranges'].tolist()

['50799409_50806880,50809894_50810042,50811040_50811176,50813857_50814038,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50859842_50859985,50862889_50862981,50864033_50864080,50885581_50885627',
 '50799409_50806880,50809894_50810042,50811040_50811176,50813857_50814038,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50859842_50859985,50885581_50885627',
 '50799409_50806880,50809894_50810042,50811040_50811176,50813857_50814038,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50859842_50859985,50864033_50864080,50885581_50885627']

In [89]:
concat_cds_exon_join.loc[concat_cds_exon_join['CDS_ranges'] == '50806729_50806880,50809894_50810042,50811040_50811176,50813857_50814062,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50859842_50859940', 'Exons_ranges'].tolist()

['50799409_50806880,50809894_50810042,50811040_50811176,50813857_50814062,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50859842_50859985,50885581_50885627',
 '50799409_50806880,50809894_50810042,50811040_50811176,50813857_50814062,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50859842_50859985,50864033_50864080,50885581_50885627']

In [55]:

for diff_exon in set.difference(*[set(elem.split(',')) for elem in concat_cds_exon_join.loc[concat_cds_exon_join['CDS_ranges'] == '50806729_50806880,50809894_50810042,50811040_50811176,50813857_50814038,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50850665_50850787', 'Exons_ranges'].tolist()]):
    print(diff_exon)
    print(sorted(list(set([sub_e for elem in concat_cds_exon_join.loc[concat_cds_exon_join['CDS_ranges'] == '50806729_50806880,50809894_50810042,50811040_50811176,50813857_50814038,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50850665_50850787', 'Exons_ranges'].tolist() for sub_e in elem.split(',') ]))))
    print(list(sorted(list(set([sub_e for elem in concat_cds_exon_join.loc[concat_cds_exon_join['CDS_ranges'] == '50806729_50806880,50809894_50810042,50811040_50811176,50813857_50814038,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50850665_50850787', 'Exons_ranges'].tolist() for sub_e in elem.split(',') ])))).index(diff_exon))


50864033_50864080
['50799409_50806880', '50809894_50810042', '50811040_50811176', '50813857_50814038', '50816006_50816279', '50820552_50820649', '50828131_50828295', '50836074_50836312', '50841862_50841992', '50843988_50844122', '50850665_50850807', '50859842_50859985', '50864033_50864080', '50885581_50885627']
12


In [71]:
concat_cds_exon_join.loc[concat_cds_exon_join['CDS_ranges'] == '50806729_50806880,50809894_50810042,50811040_50811176,50813857_50814062,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50859842_50859940', 'Exons_ranges'].tolist()

['50799409_50806880,50809894_50810042,50811040_50811176,50813857_50814062,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50859842_50859985,50885581_50885627',
 '50799409_50806880,50809894_50810042,50811040_50811176,50813857_50814062,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50859842_50859985,50864033_50864080,50885581_50885627']

In [68]:
len(test.head(1).CDS_ranges.values.tolist()[0])

8

In [135]:
pprint(test.loc[test['Gene'] == 'A2M', 'CDS_ranges'].values.tolist())

[['9067823_9067839,9068183_9068224,9068740_9068842,9069745_9069813,9070488_9070578,9072359_9072486,9072653_9072871,9074560_9074783,9076756_9076936,9077346_9077420,9077701_9077857,9079244_9079331,9079639_9079815,9080094_9080177,9089200_9089251,9089902_9090023,9090356_9090482,9091201_9091429,9093465_9093579,9094973_9095084,9095539_9095700,9098607_9098756,9099381_9099523,9101144_9101207,9101447_9101674,9104239_9104400,9106236_9106345,9106491_9106605,9107524_9107644,9109321_9109405,9109867_9110035,9110314_9110334,9112159_9112211,9112377_9112536,9113360_9113543,9115764_9115849',
  '9067823_9067839,9068183_9068224,9068740_9068842,9069745_9069813,9070488_9070578,9072359_9072486,9072653_9072871,9074560_9074783,9076756_9076936,9077346_9077420,9077701_9077857,9079244_9079331,9079639_9079815,9080094_9080177,9089200_9089251,9089902_9090023,9090356_9090482,9091201_9091429,9093465_9093579,9094973_9095084,9095539_9095700,9098607_9098756,9099381_9099523,9101144_9101207,9101447_9101674,9104239_9104400,

In [187]:
def counter(r):
    
    three_prime_counter = 0
    five_prime_counter = 0
    
    counter_dict = dict(collections.Counter(r))
    for k,v in counter_dict.items():
        if v > 1:
#             print(k, v)
            exons_including_utrs = merge_with_strand.loc[merge_with_strand['CDS_ranges'] == k, 'Exons_ranges'].tolist()
#             print(exons_including_utrs)
            exons_including_utrs_split = list(sorted(set([sub_e for e in exons_including_utrs for sub_e in e.split(',')])))
#             print(exons_including_utrs_split)
            strand_l = list(set(merge_with_strand.loc[merge_with_strand['CDS_ranges'] == k, 'Strand'].values))
            
            gene = list(set(merge_with_strand.loc[merge_with_strand['CDS_ranges'] == k, 'Gene'].values))
#             print(gene)

            if len(strand_l) > 1:
#                 print('ERROR : ' + k)
                strand = 1
            elif len(strand_l) == 0:
                strand = 1
            else:
                strand = strand_l[0]
            
            tmp_l = [k for k,v in dict(collections.Counter([sub_e for elem in exons_including_utrs for sub_e in elem.split(',')])).items() if v < len(exons_including_utrs)]
#             print(k, v, strand)
            
            
            for utr in tmp_l:
#                 print(utr)
                index = exons_including_utrs_split.index(utr)
                exons_nb = len(exons_including_utrs_split)
                if index > round(exons_nb/2, 0):
                    check = True
                else:
                    check = False
                                
                if check is True and int(strand) == -1:
                    check = False
                elif check is False and int(strand) == -1:
                    check = True
                
                t = '3_prime_UTR' if check is True else '5_prime_UTR'
                if t == '3_prime_UTR':
                    three_prime_counter += 1
                elif t == '5_prime_UTR':
                    five_prime_counter += 1
#     print([five_prime_counter, three_prime_counter])
    return pd.Series([five_prime_counter, three_prime_counter])
                    
#             print(k, v, tmp_l)
tqdm.pandas()
test[['5_prime_modif', '3_prime_modif']] = test.CDS_ranges.progress_apply(counter)
test

/home/weber/.conda/envs/ExoCarto/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 11211/11211 [09:16<00:00, 20.15it/s]


,Gene,CDS_ranges,5_prime_modif,3_prime_modif
0,A1CF,"[50806729_50806880,50809894_50810042,50811040_...",4,0
1,A2M,"[9067823_9067839,9068183_9068224,9068740_90688...",3,0
2,A2ML1,"[8845172_8845174,8845442_8845502,8846077_88462...",0,0
3,A4GALT,"[42692890_42693951, 42692890_42693951]",1,1
4,AAAS,"[53307489_53307713,53307845_53307929,53308052_...",0,0
...,...,...,...,...
11206,ZWINT,"[56358418_56358459,56358556_56358687,56358909_...",0,3
11207,ZXDC,"[126461529_126462220,126466155_126466325,12647...",0,0
11208,ZYG11A,"[52842884_52842973,52854465_52854630,52856998_...",0,0
11209,ZYX,"[143381572_143381779,143382248_143382447,14338...",2,0


In [195]:
test.loc[(test['5_prime_modif'] > 0)]

,Gene,CDS_ranges,5_prime_modif,3_prime_modif
0,A1CF,"[50806729_50806880,50809894_50810042,50811040_...",4,0
1,A2M,"[9067823_9067839,9068183_9068224,9068740_90688...",3,0
3,A4GALT,"[42692890_42693951, 42692890_42693951]",1,1
6,AADAT,"[170060928_170060969,170061892_170061993,17006...",4,0
7,AAGAB,"[67202821_67202898,67203548_67203597,67204044_...",2,0
...,...,...,...,...
11200,ZSCAN9,"[28227085_28227504,28227690_28227837,28230336_...",2,0
11204,ZUP1,"[116635832_116635879,116645714_116645934,11664...",2,0
11205,ZWILCH,"[66518901_66519078,66520590_66520660,66521050_...",2,0
11209,ZYX,"[143381572_143381779,143382248_143382447,14338...",2,0


In [196]:
test.loc[(test['5_prime_modif'] > 0) & (test['3_prime_modif'] > 0)]

,Gene,CDS_ranges,5_prime_modif,3_prime_modif
3,A4GALT,"[42692890_42693951, 42692890_42693951]",1,1
15,ABAT,"[8735740_8735809,8746001_8746098,8748108_87481...",3,2
42,ABCG2,"[88092356_88092381,88094577_88094659,88095520_...",7,2
43,ABCG4,"[119149966_119150203,119154026_119154143,11915...",3,2
49,ABHD14B,"[51969426_51969605,51969943_51970184,51971460_...",1,1
...,...,...,...,...
11193,ZSCAN29,"[43361073_43361941,43363915_43364382,43366110_...",2,1
11195,ZSCAN31,"[28326166_28326854,28327383_28327533,28329303_...",8,7
11196,ZSCAN32,"[3382852_3383711,3384459_3384941,3390010_33901...",4,5
11198,ZSCAN5A,"[56221575_56222323,56222591_56222741,56223631_...",10,12


In [179]:
test.loc[(test['3_prime_modif'] != 0)]

,Gene,CDS_ranges,5_prime_modif,3_prime_modif
15,ABAT,"[8735740_8735809,8746001_8746098,8748108_87481...",3,2
42,ABCG2,"[88092356_88092381,88094577_88094659,88095520_...",7,2
43,ABCG4,"[119149966_119150203,119154026_119154143,11915...",3,2
56,ABHD5,"[43690993_43691039,43699276_43699361,43702215_...",0,3
66,ABLIM3,"[149142096_149142108,149183452_149183589,14919...",12,8
...,...,...,...,...
11195,ZSCAN31,"[28326166_28326854,28327383_28327533,28329303_...",10,5
11196,ZSCAN32,"[3382852_3383711,3384459_3384941,3390010_33901...",6,3
11198,ZSCAN5A,"[56221575_56222323,56222591_56222741,56223631_...",12,10
11202,ZSWIM7,"[15978047_15978163,15981040_15981144,15987266_...",0,3


In [191]:
test.loc[(test['3_prime_modif'] == 0) & (test['5_prime_modif'] != 0)]

,Gene,CDS_ranges,5_prime_modif,3_prime_modif
0,A1CF,"[50806729_50806880,50809894_50810042,50811040_...",4,0
1,A2M,"[9067823_9067839,9068183_9068224,9068740_90688...",3,0
6,AADAT,"[170060928_170060969,170061892_170061993,17006...",4,0
7,AAGAB,"[67202821_67202898,67203548_67203597,67204044_...",2,0
9,AAMDC,"[77842497_77842628,77869722_77869817,77876950_...",4,0
...,...,...,...,...
11200,ZSCAN9,"[28227085_28227504,28227690_28227837,28230336_...",2,0
11204,ZUP1,"[116635832_116635879,116645714_116645934,11664...",2,0
11205,ZWILCH,"[66518901_66519078,66520590_66520660,66521050_...",2,0
11209,ZYX,"[143381572_143381779,143382248_143382447,14338...",2,0


In [176]:
merge_with_strand

,RefSeq mRNA ID,Strand,Gene,mRNA,Exons_ranges,CDS_ranges
0,NM_001318854,1,OPRL1,NM_001318854,"64092688_64092938,64097802_64098157,64098276_6...","64092721_64092938,64097802_64098157,64098276_6..."
2,NM_001318855,1,OPRL1,NM_001318855,"64092688_64092981,64097802_64098157,64098276_6...","64092836_64092981,64097802_64098157,64098276_6..."
4,NM_001104925,1,PCMTD2,NM_001104925,"64255748_64255870,64259942_64260272,64264429_6...","64259966_64260272,64264429_64264531,64265258_6..."
6,NM_018257,1,PCMTD2,NM_018257,"64255748_64255870,64259942_64260272,64264429_6...","64259966_64260272,64264429_64264531,64265258_6..."
8,NM_023015,1,INTS3,NM_023015,"153728050_153728784,153740651_153740734,153741...","153728635_153728784,153740651_153740734,153741..."
...,...,...,...,...,...,...
54918,NM_001193328,-1,ZNF692,NM_001193328,"248850008_248850516,248850682_248850781,248853...","248850210_248850516,248850682_248850781,248853..."
54919,NM_001350073,-1,ZNF692,NM_001350073,"248850008_248850516,248850682_248850781,248853...","248850210_248850516,248850682_248850781,248853..."
54920,NM_001136036,-1,ZNF692,NM_001136036,"248850008_248850516,248850682_248850781,248853...","248850210_248850516,248850682_248850781,248853..."
54921,NM_001017434,1,PGBD2,NM_001017434,"248906235_248906342,248913816_248913879,248917...",248917338_248918363


In [172]:
merge_with_strand.loc[merge_with_strand['Gene'] == 'A2M']

,RefSeq mRNA ID,Strand,Gene,mRNA,Exons_ranges,CDS_ranges
23912,NM_001347423,-1,A2M,NM_001347423,"9067708_9067839,9068183_9068224,9068740_906884...","9067823_9067839,9068183_9068224,9068740_906884..."
23913,NM_001347425,-1,A2M,NM_001347425,"9067708_9067839,9068183_9068224,9068740_906884...","9067823_9067839,9068183_9068224,9068740_906884..."
23914,NM_000014,-1,A2M,NM_000014,"9067708_9067839,9068183_9068224,9068740_906884...","9067823_9067839,9068183_9068224,9068740_906884..."
23915,NM_001347424,-1,A2M,NM_001347424,"9067708_9067839,9068183_9068224,9068740_906884...","9067823_9067839,9068183_9068224,9068740_906884..."


In [171]:
biomart_raw.loc[biomart_raw['Gene name'] == 'A1CF']

,Gene stable ID,Transcript stable ID,Strand,Gene name,Transcript type,Gene type,Gene % GC content,APPRIS annotation,Transcript support level (TSL),Transcript length (including UTRs and CDS),Transcription start site (TSS),Transcript end (bp),Transcript start (bp),Gene end (bp),Gene start (bp),Transcript count,RefSeq mRNA ID,HGNC ID
78017,ENSG00000148584,ENST00000374001,-1,A1CF,protein_coding,protein_coding,36.24,alternative1,tsl5,9221,50885675,50885675,50799409,50885675,50799409,10,NM_001370130,HGNC:24086
78027,ENSG00000148584,ENST00000373993,-1,A1CF,protein_coding,protein_coding,36.24,principal3,tsl1,1997,50859985,50859985,50806562,50885675,50799409,10,NM_001370131,HGNC:24086
78028,ENSG00000148584,ENST00000373993,-1,A1CF,protein_coding,protein_coding,36.24,principal3,tsl1,1997,50859985,50859985,50806562,50885675,50799409,10,NM_138932,HGNC:24086
78037,ENSG00000148584,ENST00000373997,-1,A1CF,protein_coding,protein_coding,36.24,alternative1,tsl1 (assigned to previous version 7),9221,50885627,50885627,50799409,50885675,50799409,10,NM_014576,HGNC:24086
78047,ENSG00000148584,ENST00000373995,-1,A1CF,protein_coding,protein_coding,36.24,NaN,tsl2,2211,50885675,50885675,50806562,50885675,50799409,10,NaN,HGNC:24086
78054,ENSG00000148584,ENST00000493415,-1,A1CF,processed_transcript,protein_coding,36.24,NaN,tsl5,926,50828461,50828461,50811044,50885675,50799409,10,NaN,HGNC:24086
78057,ENSG00000148584,ENST00000473480,-1,A1CF,processed_transcript,protein_coding,36.24,NaN,tsl3,813,50885675,50885675,50828131,50885675,50799409,10,NaN,HGNC:24086
78060,ENSG00000148584,ENST00000414883,-1,A1CF,protein_coding,protein_coding,36.24,NaN,tsl5,885,50885675,50885675,50836074,50885675,50799409,10,NaN,HGNC:24086
78065,ENSG00000148584,ENST00000395489,-1,A1CF,protein_coding,protein_coding,36.24,NaN,tsl2,9517,50885675,50885675,50799421,50885675,50799409,10,NM_001198819,HGNC:24086
78070,ENSG00000148584,ENST00000282641,-1,A1CF,protein_coding,protein_coding,36.24,alternative1,tsl2,9350,50885675,50885675,50799421,50885675,50799409,10,NM_001198818,HGNC:24086


In [163]:
test.loc['Total'] = test.sum(axis=0)
test

,Gene,CDS_ranges,5_prime_modif,3_prime_modif
0,A1CF,"[50806729_50806880,50809894_50810042,50811040_...",4,0
1,A2M,"[9067823_9067839,9068183_9068224,9068740_90688...",3,0
2,A2ML1,"[8845172_8845174,8845442_8845502,8846077_88462...",0,0
3,A4GALT,"[42692890_42693951, 42692890_42693951]",2,0
4,AAAS,"[53307489_53307713,53307845_53307929,53308052_...",0,0
...,...,...,...,...
11207,ZXDC,"[126461529_126462220,126466155_126466325,12647...",0,0
11208,ZYG11A,"[52842884_52842973,52854465_52854630,52856998_...",0,0
11209,ZYX,"[143381572_143381779,143382248_143382447,14338...",2,0
11210,ZZZ3,"[77565640_77565784,77566081_77566181,77568332_...",9,0


In [164]:
test = test.drop('Total')


In [190]:
pd.concat([
    pd.DataFrame(test.loc[test['5_prime_modif'] != 0, '5_prime_modif'].describe()),
    pd.DataFrame(test.loc[test['3_prime_modif'] != 0, '3_prime_modif'].describe())
], axis=1
)

,5_prime_modif,3_prime_modif
count,4220.000000,1584.000000
mean,4.291232,2.641414
std,4.514155,2.227481
min,1.000000,1.000000
25%,2.000000,2.000000
50%,3.000000,2.000000
75%,5.000000,3.000000
max,55.000000,50.000000


In [183]:
test.loc[test['3_prime_modif'] != 0, '3_prime_modif'].describe()

count    1562.000000
mean        2.452625
std         2.097098
min         1.000000
25%         1.000000
50%         2.000000
75%         3.000000
max        50.000000
Name: 3_prime_modif, dtype: float64

In [27]:
collections.Counter(test.loc['A1CF'].values.tolist()[0])

Counter({'50806729_50806880,50809894_50810042,50811040_50811176,50813857_50814038,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50859842_50859940': 3,
         '50806729_50806880,50809894_50810042,50811040_50811176,50813857_50814062,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50850665_50850787': 1,
         '50806729_50806880,50809894_50810042,50811040_50811176,50813857_50814038,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50850665_50850787': 2,
         '50806729_50806880,50809894_50810042,50811040_50811176,50813857_50814062,50816006_50816279,50820552_50820649,50828131_50828295,50836074_50836312,50841862_50841992,50843988_50844122,50859842_50859940': 2})

In [23]:
def return_common_nb(r):
    if r['mRNA_nb'] > 1:
        print(r['Gene'], len(set(r['Exon_list_nb'])), len(r['Exon_list_nb']), r['Exon_list'], [e for e in r['Exon_list']])




A1CF 1 8 [['50806729_50806880', '50809894_50810042', '50811040_50811176', '50813857_50814038', '50816006_50816279', '50820552_50820649', '50828131_50828295', '50836074_50836312', '50841862_50841992', '50843988_50844122', '50859842_50859940'], ['50806729_50806880', '50809894_50810042', '50811040_50811176', '50813857_50814062', '50816006_50816279', '50820552_50820649', '50828131_50828295', '50836074_50836312', '50841862_50841992', '50843988_50844122', '50850665_50850787'], ['50806729_50806880', '50809894_50810042', '50811040_50811176', '50813857_50814038', '50816006_50816279', '50820552_50820649', '50828131_50828295', '50836074_50836312', '50841862_50841992', '50843988_50844122', '50850665_50850787'], ['50806729_50806880', '50809894_50810042', '50811040_50811176', '50813857_50814038', '50816006_50816279', '50820552_50820649', '50828131_50828295', '50836074_50836312', '50841862_50841992', '50843988_50844122', '50859842_50859940'], ['50806729_50806880', '50809894_50810042', '50811040_50811

0    None
1    None
2    None
3    None
4    None
dtype: object

In [21]:
def return_common_nb(r):
    if r['mRNA_nb'] > 0:
        print(r['Gene'], len(set(r['Exon_list_nb'])), len(r['Exon_list_nb']), r['Exon_list'])
        if len(set(r['Exon_list_nb'])) < len(r['Exon_list_nb']):
            counter = dict(collections.Counter(r['Exon_list_nb']))
            print(counter)
            for mrna, count in counter.items():
                if count > 1:
                    print(mrna, counter[mrna])
#                     index_positions = [i for i, e in enumerate(r['Exon_list_nb']) if e == mrna]
#                     print(index_positions)
#                     intersect_exons_list = [set(r['Exon_list'][i]) for i in index_positions]
#                     print(len(set.intersection(*intersect_exons_list)), r['Exon_list_nb'][index_positions[0]])
#                     if len(set.intersection(*intersect_exons_list)) == r['Exon_list_nb'][index_positions[0]]:
#                         print(len(set.intersection(*intersect_exons_list)), r['Exon_list_nb'][index_positions[0]])
                

# concat_exons_gene_level['Intersection'] = concat_exons_gene_level['Exon_list'].apply(lambda r: set.intersection(*[set(e) for e in r]))
# concat_exons_gene_level['mRNA_nb'] = concat_exons_gene_level['mRNA'].apply(len)
# concat_exons_gene_level['Exon_list_nb'] = concat_exons_gene_level['Exon_list'].apply(lambda r: [len(e) for e in r])
# concat_exons_gene_level['Exon_list_set'] = concat_exons_gene_level['Exon_list_nb'].apply(lambda r: len(set(r)))
# concat_exons_gene_level['Check_at_least_two'] = concat_exons_gene_level.apply(lambda r: True if len(set(r['Exon_list_nb'])) < r['mRNA_nb'] else False, axis=1)
# concat_exons_gene_level['Check_common_nb'] = concat_exons_gene_level.apply(lambda r: True if len(set(r['Exon_list_nb'])) < r['mRNA_nb'] else False, axis=1)
concat_exons_gene_level.head().apply(lambda r: return_common_nb(r), axis=1)
# concat_exons_gene_level



A1BG 1 1 [['58347022_58347029', '58347353_58347640', '58350370_58350651', '58351391_58351687', '58352283_58352555', '58352928_58353197', '58353292_58353327', '58353404_58353437']]
A1CF 1 8 [['50806729_50806880', '50809894_50810042', '50811040_50811176', '50813857_50814038', '50816006_50816279', '50820552_50820649', '50828131_50828295', '50836074_50836312', '50841862_50841992', '50843988_50844122', '50859842_50859940'], ['50806729_50806880', '50809894_50810042', '50811040_50811176', '50813857_50814062', '50816006_50816279', '50820552_50820649', '50828131_50828295', '50836074_50836312', '50841862_50841992', '50843988_50844122', '50850665_50850787'], ['50806729_50806880', '50809894_50810042', '50811040_50811176', '50813857_50814038', '50816006_50816279', '50820552_50820649', '50828131_50828295', '50836074_50836312', '50841862_50841992', '50843988_50844122', '50850665_50850787'], ['50806729_50806880', '50809894_50810042', '50811040_50811176', '50813857_50814038', '50816006_50816279', '5082

0    None
1    None
2    None
3    None
4    None
dtype: object

In [108]:
print(len(set(concat_exons_gene_level.loc[concat_exons_gene_level['Gene'] == 'A2M', 'Exon_list'].values.tolist()[0][0]).intersection(set(concat_exons_gene_level.loc[concat_exons_gene_level['Gene'] == 'A2M', 'Exon_list'].values.tolist()[0][1]))))
print(len(concat_exons_gene_level.loc[concat_exons_gene_level['Gene'] == 'A2M', 'Exon_list'].values.tolist()[0][0]))
# pprint(concat_exons_gene_level.loc[concat_exons_gene_level['Gene'] == 'A2M'].values.tolist())

36
36


In [12]:
concat_exons_gene_level.loc[(concat_exons_gene_level['mRNA_nb'] > 1)].explode('mRNA').mRNA.nunique()

KeyError: 'mRNA_nb'

In [82]:
concat_exons_gene_level.loc[(concat_exons_gene_level['mRNA_nb'] > 1)]

,mRNA,Exon_list,Intersection,mRNA_nb,Exon_list_nb,Exon_list_set
Gene,,,,,,
A1CF,"[NM_001198818, NM_001198819, NM_001198820, NM_001370130, NM_001370131, NM_014576, NM_138932, NM_138933]","[[50806729_50806880, 50809894_50810042, 50811040_50811176, 50813857_50814038, 50816006_50816279, 50820552_50820649, 50828131_50828295, 50836074_50836312, 50841862_50841992, 50843988_50844122, 5085...","{50809894_50810042, 50828131_50828295, 50811040_50811176, 50841862_50841992, 50820552_50820649, 50816006_50816279, 50843988_50844122, 50836074_50836312, 50806729_50806880}",8,"[11, 11, 11, 11, 11, 11, 11, 11]",1
A2M,"[NM_000014, NM_001347423, NM_001347424, NM_001347425]","[[9067823_9067839, 9068183_9068224, 9068740_9068842, 9069745_9069813, 9070488_9070578, 9072359_9072486, 9072653_9072871, 9074560_9074783, 9076756_9076936, 9077346_9077420, 9077701_9077857, 9079244...","{9109321_9109405, 9069745_9069813, 9107524_9107644, 9110314_9110334, 9093465_9093579, 9101447_9101674, 9080094_9080177, 9077701_9077857, 9106491_9106605, 9068183_9068224, 9079639_9079815, 9072359_...",4,"[36, 36, 34, 33]",3
A2ML1,"[NM_001282424, NM_144670]","[[8845172_8845174, 8845442_8845502, 8846077_8846222, 8847549_8847698, 8848720_8848914, 8849669_8849759, 8850160_8850274, 8851784_8852012, 8852210_8852336, 8854128_8854249, 8854780_8854831, 8855509...","{8874971_8875011, 8857164_8857340, 8849669_8849759, 8846077_8846222, 8847549_8847698, 8874425_8874527, 8857507_8857588, 8850160_8850274, 8855509_8855592, 8857946_8858102, 8848720_8848914, 8851784_...",2,"[24, 35]",2
A4GALT,"[NM_001318038, NM_017436]","[[42692890_42693951], [42692890_42693951]]",{42692890_42693951},2,"[1, 1]",1
AAAS,"[NM_001173466, NM_015665]","[[53307489_53307713, 53307845_53307929, 53308052_53308133, 53308282_53308349, 53308435_53308528, 53308725_53308815, 53308960_53309020, 53309157_53309281, 53309601_53309721, 53314298_53314441, 5331...","{53309157_53309281, 53308435_53308528, 53315094_53315140, 53321343_53321465, 53307845_53307929, 53308282_53308349, 53307489_53307713, 53309601_53309721, 53314298_53314441, 53315727_53315782, 53320...",2,"[15, 16]",2
...,...,...,...,...,...,...
ZWINT,"[NM_001005413, NM_007057, NM_032997]","[[56358418_56358459, 56358556_56358687, 56358909_56358947, 56359476_56359532, 56359687_56359853, 56360018_56360141, 56360293_56360383, 56361196_56361236], [56358418_56358459, 56358556_56358724, 56...","{56359687_56359853, 56360293_56360383, 56361196_56361236, 56358418_56358459, 56359476_56359532, 56360018_56360141}",3,"[8, 8, 8]",1
ZXDC,"[NM_001040653, NM_025112]","[[126461529_126462220, 126466155_126466325, 126470895_126471025, 126471973_126472051, 126472153_126472305, 126474959_126475865], [126438375_126438461, 126439632_126439727, 126441765_126441946, 126...","{126470895_126471025, 126472153_126472305, 126466155_126466325, 126471973_126472051, 126474959_126475865}",2,"[6, 10]",2
ZYG11A,"[NM_001004339, NM_001307931]","[[52842884_52842973, 52854465_52854630, 52856998_52857749, 52860731_52860871, 52863981_52864157, 52866503_52866567, 52867539_52867638, 52867727_52867777, 52877682_52877843, 52877925_52877969, 5288...","{52866503_52866567, 52885833_52885894, 52863981_52864157, 52877925_52877969, 52867727_52867777, 52892782_52892957, 52881471_52881665, 52877682_52877843, 52886956_52887053, 52867539_52867638}",2,"[14, 11]",2


In [109]:
concat_exons_gene_level.loc[(concat_exons_gene_level['mRNA_nb'] > 1) & (concat_exons_gene_level['Check_at_least_two'] == True)]

,Gene,mRNA,Exon_list,Intersection,mRNA_nb,Exon_list_nb,Exon_list_set,Check_at_least_two
1,A1CF,"[NM_001198818, NM_001198819, NM_001198820, NM_001370130, NM_001370131, NM_014576, NM_138932, NM_138933]","[[50806729_50806880, 50809894_50810042, 50811040_50811176, 50813857_50814038, 50816006_50816279, 50820552_50820649, 50828131_50828295, 50836074_50836312, 50841862_50841992, 50843988_50844122, 5085...","{50809894_50810042, 50828131_50828295, 50811040_50811176, 50841862_50841992, 50820552_50820649, 50816006_50816279, 50843988_50844122, 50836074_50836312, 50806729_50806880}",8,"[11, 11, 11, 11, 11, 11, 11, 11]",1,True
2,A2M,"[NM_000014, NM_001347423, NM_001347424, NM_001347425]","[[9067823_9067839, 9068183_9068224, 9068740_9068842, 9069745_9069813, 9070488_9070578, 9072359_9072486, 9072653_9072871, 9074560_9074783, 9076756_9076936, 9077346_9077420, 9077701_9077857, 9079244...","{9109321_9109405, 9069745_9069813, 9107524_9107644, 9110314_9110334, 9093465_9093579, 9101447_9101674, 9080094_9080177, 9077701_9077857, 9106491_9106605, 9068183_9068224, 9079639_9079815, 9072359_...",4,"[36, 36, 34, 33]",3,True
5,A4GALT,"[NM_001318038, NM_017436]","[[42692890_42693951], [42692890_42693951]]",{42692890_42693951},2,"[1, 1]",1,True
8,AACS,"[NM_001319839, NM_001319840, NM_023928]","[[125065585_125065717, 125073876_125073979, 125076491_125076611, 125086330_125086443, 125091426_125091523, 125102679_125102793, 125103000_125103081, 125107121_125107268, 125114477_125114557, 12511...","{125073876_125073979, 125124705_125124769, 125118641_125118765, 125086330_125086443, 125129335_125129460, 125107121_125107268, 125102679_125102793, 125134003_125134072, 125114477_125114557, 125076...",3,"[17, 17, 18]",2,True
13,AADAT,"[NM_001286682, NM_001286683, NM_016228, NM_182662]","[[170060928_170060969, 170061892_170061993, 170064719_170064825, 170066414_170066478, 170067327_170067388, 170068591_170068687, 170069148_170069230, 170070587_170070652, 170073136_170073345, 17007...","{170087116_170087248, 170068591_170068687, 170064719_170064825, 170078509_170078583, 170060928_170060969, 170067327_170067388, 170088396_170088564, 170061892_170061993, 170073136_170073345, 170070...",4,"[13, 13, 13, 13]",1,True
...,...,...,...,...,...,...,...,...
19332,ZUP1,"[NM_001361189, NM_001361190, NM_001361191, NM_145062]","[[116635832_116635879, 116645714_116645934, 116647459_116647610, 116651572_116651737, 116652004_116652192, 116656684_116656852, 116658803_116658924, 116660736_116660846, 116666634_116667192], [116...","{116652004_116652192, 116651572_116651737, 116658803_116658924, 116647459_116647610, 116635832_116635879, 116645714_116645934, 116656684_116656852}",4,"[9, 9, 8, 9]",2,True
19334,ZWILCH,"[NM_001287821, NM_001287822, NM_001287823, NM_017975]","[[66518901_66519078, 66520590_66520660, 66521050_66521205, 66523677_66523748, 66527290_66527383, 66527857_66527912, 66528852_66528957, 66529494_66529573, 66532247_66532403, 66532985_66533013, 6653...","{66532985_66533013, 66529494_66529573, 66527857_66527912, 66520590_66520660, 66528852_66528957, 66532247_66532403, 66546591_66546679, 66521050_66521205, 66527290_66527383, 66540098_66540210, 66535...",4,"[14, 14, 14, 18]",2,True
19335,ZWINT,"[NM_001005413, NM_007057, NM_032997]","[[56358418_56358459, 56358556_56358687, 56358909_56358947, 56359476_56359532, 56359687_56359853, 56360018_56360141, 56360293_56360383, 56361196_56361236], [56358418_56358459, 56358556_56358724, 56...","{56359687_56359853, 56360293_56360383, 56361196_56361236, 56358418_56358459, 56359476_56359532, 56360018_56360141}",3,"[8, 8, 8]",1,True
19341,ZYX,"[NM_001010972, NM_001362783, NM_003461]","[[143381572_143381779, 143382248_143382447, 143382593_143382685, 143382801_143383322, 143388219_143388339, 143388489_143388658, 143388767_143388945, 143389857_143389977, 143390578_143390682], [143...","{143390578_143390682, 143389857_143389977, 143388489_143388658, 143388767_143388945, 143382248_143382447, 143388219_143388339, 143381572_143381779, 143382801_143383322}

In [93]:
concat_exons_gene_level.loc[(concat_exons_gene_level['mRNA_nb'] > 1) & (concat_exons_gene_level['Exon_list_set'] == 1)]

,Gene,mRNA,Exon_list,Intersection,mRNA_nb,Exon_list_nb,Exon_list_set
1,A1CF,"[NM_001198818, NM_001198819, NM_001198820, NM_001370130, NM_001370131, NM_014576, NM_138932, NM_138933]","[[50806729_50806880, 50809894_50810042, 50811040_50811176, 50813857_50814038, 50816006_50816279, 50820552_50820649, 50828131_50828295, 50836074_50836312, 50841862_50841992, 50843988_50844122, 5085...","{50809894_50810042, 50828131_50828295, 50811040_50811176, 50841862_50841992, 50820552_50820649, 50816006_50816279, 50843988_50844122, 50836074_50836312, 50806729_50806880}",8,"[11, 11, 11, 11, 11, 11, 11, 11]",1
5,A4GALT,"[NM_001318038, NM_017436]","[[42692890_42693951], [42692890_42693951]]",{42692890_42693951},2,"[1, 1]",1
13,AADAT,"[NM_001286682, NM_001286683, NM_016228, NM_182662]","[[170060928_170060969, 170061892_170061993, 170064719_170064825, 170066414_170066478, 170067327_170067388, 170068591_170068687, 170069148_170069230, 170070587_170070652, 170073136_170073345, 17007...","{170087116_170087248, 170068591_170068687, 170064719_170064825, 170078509_170078583, 170060928_170060969, 170067327_170067388, 170088396_170088564, 170061892_170061993, 170073136_170073345, 170070...",4,"[13, 13, 13, 13]",1
17,AAMP,"[NM_001087, NM_001302545]","[[218264533_218264608, 218265020_218265174, 218265371_218265461, 218265579_218265682, 218265831_218265946, 218266064_218266147, 218266443_218266587, 218266847_218266986, 218267494_218267613, 21826...","{218264533_218264608, 218269966_218270086, 218266443_218266587, 218265831_218265946, 218266847_218266986, 218266064_218266147, 218265020_218265174, 218265371_218265461, 218265579_218265682, 218267...",2,"[11, 11]",1
19,AAR2,"[NM_001271874, NM_015511]","[[36239869_36240625, 36244697_36244926, 36255578_36255745], [36239869_36240625, 36244697_36244926, 36255578_36255745]]","{36255578_36255745, 36244697_36244926, 36239869_36240625}",2,"[3, 3]",1
...,...,...,...,...,...,...,...
19320,ZSCAN5B,"[NM_001080456, NM_001385638]","[[56189827_56190575, 56190837_56190987, 56191850_56192053, 56192669_56193052], [56189827_56190575, 56190837_56190987, 56191850_56192053, 56192669_56193052]]","{56190837_56190987, 56191850_56192053, 56192669_56193052, 56189827_56190575}",2,"[4, 4]",1
19329,ZSWIM7,"[NM_001042697, NM_001042698]","[[15978047_15978163, 15981040_15981144, 15987266_15987368, 15993757_15993778, 15999519_15999594], [15978047_15978163, 15981040_15981144, 15987266_15987368, 15993757_15993778, 15999519_15999594]]","{15978047_15978163, 15993757_15993778, 15981040_15981144, 15987266_15987368, 15999519_15999594}",2,"[5, 5]",1
19330,ZSWIM8,"[NM_001242487, NM_001242488, NM_001367799, NM_015037]","[[73785879_73786086, 73788670_73788823, 73789096_73789190, 73789367_73789539, 73789717_73789824, 73789956_73790037, 73790172_73790292, 73790975_73791176, 73791324_73791499, 73791859_73792852, 7379...","{73793588_73793719, 73793865_73794044, 73797781_73798070, 73797113_73797271, 73789096_73789190, 73794147_73794330, 73796774_73797014, 73800640_73800759, 73800296_73800472, 73798230_73798453, 73791...",4,"[26, 26, 26, 26]",1
19335,ZWINT,"[NM_001005413, NM_007057, NM_032997]","[[56358418_56358459, 56358556_56358687, 56358909_56358947, 56359476_56359532, 56359687_56359853, 56360018_56360141, 56360293_56360383, 56361196_56361236], [56358418_56358459, 56358556_56358724, 56...","{56359687_56359853, 56360293_56360383, 56361196_56361236, 56358418_56358459, 56359476_56359532, 56360018_56360141}",3,"[8, 8, 8]",1


In [92]:
concat_exons_gene_level = concat_exons_gene_level.reset_index()
concat_exons_gene_level.loc[(concat_exons_gene_level['mRNA_nb'] > 1) & (concat_exons_gene_level['Exon_list_set'] == 1) & (concat_exons_gene_level.Gene.isin(exotic.symbol.unique().tolist()))]


,Gene,mRNA,Exon_list,Intersection,mRNA_nb,Exon_list_nb,Exon_list_set
17,AAMP,"[NM_001087, NM_001302545]","[[218264533_218264608, 218265020_218265174, 218265371_218265461, 218265579_218265682, 218265831_218265946, 218266064_218266147, 218266443_218266587, 218266847_218266986, 218267494_218267613, 21826...","{218264533_218264608, 218269966_218270086, 218266443_218266587, 218265831_218265946, 218266847_218266986, 218266064_218266147, 218265020_218265174, 218265371_218265461, 218265579_218265682, 218267...",2,"[11, 11]",1
34,ABCA2,"[NM_001606, NM_212533]","[[137007929_137007964, 137008416_137008622, 137008731_137008868, 137008951_137009053, 137009370_137009462, 137009541_137009644, 137009769_137009903, 137009983_137010124, 137010193_137010371, 13701...","{137021392_137021610, 137010620_137010737, 137009769_137009903, 137016920_137017124, 137017787_137017901, 137015675_137015871, 137012265_137012376, 137012485_137012590, 137016291_137016471, 137023...",2,"[49, 49]",1
60,ABCC8,"[NM_000352, NM_001287174, NM_001351295, NM_001351296, NM_001351297]","[[17392991_17393128, 17393697_17393759, 17394266_17394399, 17395172_17395275, 17395610_17395718, 17395852_17395930, 17396916_17397046, 17397193_17397313, 17397684_17397797, 17398339_17398441, 1740...","{17407354_17407453, 17392991_17393128, 17394266_17394399, 17406888_17407129, 17398339_17398441, 17428565_17428670, 17427049_17427154, 17404512_17404669, 17413394_17413478, 17414512_17414610, 17395...",5,"[39, 39, 39, 39, 39]",1
72,ABCG2,"[NM_001257386, NM_001348985, NM_001348986, NM_001348987, NM_001348988, NM_001348989, NM_004827]","[[88092356_88092381, 88094577_88094659, 88095520_88095599, 88097453_88097607, 88099324_88099448, 88101230_88101319, 88107184_88107266, 88113303_88113553, 88114957_88115058, 88118109_88118260, 8812...","{88097453_88097607, 88107184_88107266, 88101230_88101319, 88114957_88115058, 88131803_88131917, 88094577_88094659, 88132576_88132635, 88139793_88139995, 88099324_88099448, 88121635_88121792, 88131...",7,"[15, 15, 15, 15, 15, 15, 15]",1
79,ABHD12,"[NM_001042472, NM_015600]","[[25300845_25300884, 25302219_25302346, 25303550_25303628, 25306833_25306915, 25307966_25308045, 25308457_25308494, 25309446_25309575, 25314925_25314970, 25317048_25317078, 25320199_25320318, 2532...","{25390513_25390703, 25309446_25309575, 25302219_25302346, 25306833_25306915, 25303550_25303628, 25320199_25320318, 25308457_25308494, 25339227_25339351, 25323325_25323430, 25307966_25308045, 25317...",2,"[13, 13]",1
...,...,...,...,...,...,...,...
19299,ZRANB2,"[NM_005455, NM_203350]","[[71065719_71065752, 71066776_71066934, 71069276_71069362, 71070827_71070996, 71072121_71072255, 71072472_71072548, 71076795_71076877, 71078457_71078565, 71078656_71078708, 71080940_71080995], [71...","{71076795_71076877, 71078457_71078565, 71072472_71072548, 71080940_71080995, 71072121_71072255, 71066776_71066934, 71069276_71069362, 71078656_71078708, 71070827_71070996}",2,"[10, 10]",1
19307,ZSCAN2,"[NM_001007072, NM_017894, NM_181877]","[[84603928_84604333, 84616309_84616343], [84603928_84604333, 84606529_84606575], [84603928_84604333, 84620602_84622040]]",{84603928_84604333},3,"[2, 2, 2]",1
19319,ZSCAN5A,"[NM_001322061, NM_001322062, NM_001322064, NM_001322065, NM_001322066, NM_001322067, NM_001322068, NM_001322070, NM_001322072, NM_001322073, NM_001322074, NM_001322075, NM_001322076, NM_001322077,...","[[56221575_56222323, 56222591_56222741, 56223631_56223834, 56224663_56225046], [56221575_56222323, 56222591_56222741, 56223631_56223834, 56224663_56225046], [56221575_56222326, 56222591_56222741, ...",{56222591_56222741},16,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]",1
19330,ZSWIM8,"[NM_001242487, NM_001242488, NM_001367799, NM_015037]","[[73785879_73786086, 73788670_73788823, 73789096_73789190, 73789367_73789539, 73789717_73789824, 73789956_73790037, 73790172_73790292, 73790975_73791176, 73791324_73791499, 73791859_73792852, 7379...","{73793588_73793719, 73793865_73794044, 73797781_73798070, 73797113_73797

In [86]:
refseq_corrected

,Gene,ranges,new_mRNA_exons,new_mRNA_nb,MAP
0,A2M,9220419-9220435,"['NM_001347423', 'NM_001347425', 'NM_000014', 'NM_001347424']",4,A2M_9220419-9220435
1,A2M,9220779-9220820,"['NM_001347423', 'NM_001347425', 'NM_000014', 'NM_001347424']",4,A2M_9220779-9220820
2,A2M,9221336-9221438,"['NM_001347423', 'NM_001347425', 'NM_000014', 'NM_001347424']",4,A2M_9221336-9221438
3,A2M,9222341-9222409,"['NM_001347423', 'NM_001347425', 'NM_000014', 'NM_001347424']",4,A2M_9222341-9222409
4,A2M,9223084-9223174,"['NM_001347423', 'NM_001347425', 'NM_000014', 'NM_001347424']",4,A2M_9223084-9223174
...,...,...,...,...,...
176794,ZZZ3,78047461-78047576,"['NM_001308237', 'NM_015534']",2,ZZZ3_78047461-78047576
176795,ZZZ3,78047664-78047811,"['NM_001308237', 'NM_015534']",2,ZZZ3_78047664-78047811
176796,ZZZ3,78050202-78050340,"['NM_001308237', 'NM_015534']",2,ZZZ3_78050202-78050340
176797,ZZZ3,78097535-78099039,['NM_015534'],1,ZZZ3_78097535-78099039


In [90]:
exotic.symbol.unique().tolist()

6446

In [59]:
concat_exons_gene_level.loc['A1BG'].to_dict()

{'mRNA': ['NM_130786'],
 'Exon_list': [['58347022_58347029',
   '58347353_58347640',
   '58350370_58350651',
   '58351391_58351687',
   '58352283_58352555',
   '58352928_58353197',
   '58353292_58353327',
   '58353404_58353437']],
 'Intersection': {'58347022_58347029',
  '58347353_58347640',
  '58350370_58350651',
  '58351391_58351687',
  '58352283_58352555',
  '58352928_58353197',
  '58353292_58353327',
  '58353404_58353437'}}

In [10]:
df.loc[df['Gene'] == 'AADACL2']

,Gene,Gene_start,Gene_stop,Exon_start,Exon_stop,mRNA,mRNA_start,mRNA_stop,Length
647167,AADACL2,155100,180671,155209,155346,NM_207365,151733927,151761339,137
647198,AADACL2,155100,180671,161819,162041,NM_207365,151733927,151761339,222
647228,AADACL2,155100,180671,165266,165335,NM_207365,151733927,151761339,69
647284,AADACL2,155100,180671,166682,166853,NM_207365,151733927,151761339,171
647342,AADACL2,155100,180671,178165,178767,NM_207365,151733927,151761339,602
131623,AADACL2,151733927,151761339,151734036,151734173,NM_207365,151733927,151761339,137
131706,AADACL2,151733927,151761339,151740646,151740868,NM_207365,151733927,151761339,222
131746,AADACL2,151733927,151761339,151744093,151744162,NM_207365,151733927,151761339,69
131815,AADACL2,151733927,151761339,151745509,151745680,NM_207365,151733927,151761339,171
131883,AADACL2,151733927,151761339,151756992,151757594,NM_207365,151733927,151761339,602


In [148]:
from tqdm import tqdm
l = list()
for j, index in tqdm(enumerate(list(refseq_38_df_pc_genes.index))):
#     if j == 10:
#         break

    if j < len(list(refseq_38_df_pc_genes.index)) - 1 :
        start_index, stop_index = index, list(refseq_38_df_pc_genes.index)[j+1]

        current_gene = [e.replace('Name=', '') for e in refseq_38_df_pc_genes.loc[start_index]['Attributes'].split(';') if 'Name' in e][0]
        for cds in list(refseq_38_df_cds.index):
            if cds > start_index and cds < stop_index:

                current_gene_cds = [e.replace('gene=', '') for e in refseq_38_df_cds.loc[cds]['Attributes'].split(';') if 'gene' in e][0]
                parent_mrna = [e.replace('Parent=rna-', '') for e in refseq_38_df_cds.loc[cds]['Attributes'].split(';') if 'Parent' in e][0].split('.')[0]
                mrna_index = refseq_38_df_mrna.loc[refseq_38_df_mrna['Attributes'].str.contains(parent_mrna)].index[0]
                
                

#                 print(j, start_index, stop_index, cds, current_gene, current_gene_cds)

                if current_gene == current_gene_cds:
                    l.append(
                        {
                            'Gene' : current_gene,
                            'Gene_start' : refseq_38_df_pc_genes.loc[start_index]['Start'],
                            'Gene_stop' : refseq_38_df_pc_genes.loc[start_index]['End'],
                            'Exon_start' : refseq_38_df_cds.loc[cds]['Start'],
                            'Exon_stop' : refseq_38_df_cds.loc[cds]['End'],
                            'mRNA' : parent_mrna,
                            'mRNA_start' : refseq_38_df_mrna.loc[mrna_index]['Start'],
                            'mRNA_stop' : refseq_38_df_mrna.loc[mrna_index]['End'],
                        }
                    )
        
            
pd.options.display.max_rows =200
df = pd.DataFrame(l).sort_values(by=['Gene', 'Exon_start', 'Exon_stop']).drop_duplicates()
df['Length'] = df['Exon_stop'] - df['Exon_start']
df = df.sort_values(by=['Gene', 'Exon_start', 'Exon_stop', 'Length'], ascending=[True, True, True, False])
df = df.drop_duplicates(subset=['Gene', 'Exon_start'], keep='last').drop_duplicates(subset=['Gene', 'Exon_stop'], keep='first')

df


319it [05:43,  1.08s/it]


KeyboardInterrupt: 

In [16]:

output_json = dict()

for ig, gene in tqdm(enumerate(df.Gene.unique())):
    if ig == 10:
        break

    output_json[gene] = dict()

    tmp_gene_df = df.loc[df['Gene'] == gene]
    for chrom in chroms:
        if chrom[0] <= tmp_gene_df.index[0] < chrom[1]:
            
            output_json[gene]['CHROM'] = [c.replace('chromosome=', '') for c in refseq_38_df_chroms.loc[chrom[0]]['Attributes'].split(';') if 'chromosome=' in c][0]
# tmp_gene_df
    
    tmp_gene_df = tmp_gene_df.reset_index(drop=True)
    output_json[gene]['Gene_start_end'] = str(tmp_gene_df.Gene_start.unique()[0]) + '_' + str(tmp_gene_df.Gene_stop.unique()[0])
    for j, row in tmp_gene_df.iterrows():
        if 'Exon_list' not in output_json[gene]:
            output_json[gene]['Exon_list'] = list()
        if j == 0:
            output_json[gene]['5_prime_UTR'] = str(tmp_gene_df.Gene_start.unique()[0]) + '_' + str(row.Exon_start - 1)
            output_json[gene]['Exon_list'].append(str(row.Exon_start) + '_' + str(row.Exon_stop))
            if j == (tmp_gene_df.shape[0] - 1):
                output_json[gene]['3_prime_UTR'] = str(row.Exon_stop + 1) + '_' + str(tmp_gene_df.Gene_stop.unique()[0])
        elif j > 0 and j < (tmp_gene_df.shape[0] - 1):
            output_json[gene]['Exon_list'].append(str(row.Exon_start) + '_' + str(row.Exon_stop))
        elif j == (tmp_gene_df.shape[0] - 1):
            output_json[gene]['Exon_list'].append(str(row.Exon_start) + '_' + str(row.Exon_stop))
            output_json[gene]['3_prime_UTR'] = str(row.Exon_stop + 1) + '_' + str(tmp_gene_df.Gene_stop.unique()[0])
            
for gene in output_json:
    
    output_json[gene]['Exons_length'] = [int(e.split('_')[1]) - int(e.split('_')[0]) for e in output_json[gene]['Exon_list']]
    output_json[gene]['Intron_list'] = list()
    output_json[gene]['Exon_size_total'] = sum([int(e.split('_')[1]) - int(e.split('_')[0]) for e in output_json[gene]['Exon_list']])
    output_json[gene]['Bin_size_exon'] = round(output_json[gene]['Exon_size_total'] / 20)
    for j, exon in enumerate(output_json[gene]['Exon_list']):
        if j < len(output_json[gene]['Exon_list'])-1:
            intron_start = int(output_json[gene]['Exon_list'][j].split('_')[1]) + 1
            intron_end = int(output_json[gene]['Exon_list'][j+1].split('_')[0]) - 1
            output_json[gene]['Intron_list'].append('{}_{}'.format(str(intron_start), str(intron_end)))
    output_json[gene]['Intron_length'] = [int(e.split('_')[1]) - int(e.split('_')[0]) for e in output_json[gene]['Intron_list']]
    output_json[gene]['Intron_size_total'] = sum([int(e.split('_')[1]) - int(e.split('_')[0]) for e in output_json[gene]['Intron_list']])
    output_json[gene]['Bin_size_intron'] = round(output_json[gene]['Intron_size_total'] / 20)
    output_json[gene]['Gene_body_size'] = int(output_json[gene]['Gene_start_end'].split('_')[1]) - int(output_json[gene]['Gene_start_end'].split('_')[0])
    output_json[gene]['Bin_size_GB'] = round(output_json[gene]['Gene_body_size'] / 20)
    
    init_gene_body_bin = int(output_json[gene]['Gene_start_end'].split('_')[0])
    output_json[gene]['Bin_size_UpDown'] = round(5000 / 20)
    output_json[gene]['5_prime_UTR_size'] = int(output_json[gene]['5_prime_UTR'].split('_')[1]) - int(output_json[gene]['5_prime_UTR'].split('_')[0])
    output_json[gene]['3_prime_UTR_size'] = int(output_json[gene]['3_prime_UTR'].split('_')[1]) - int(output_json[gene]['3_prime_UTR'].split('_')[0])

    output_json[gene]['Bin_size_5_prime_UTR'] = round(int(output_json[gene]['5_prime_UTR_size']) / 20)
    output_json[gene]['Bin_size_3_prime_UTR'] = round(int(output_json[gene]['3_prime_UTR_size']) / 20)
            
refseq_38_df_transformed = pd.DataFrame.from_dict(output_json).T
refseq_38_df_transformed

10it [00:00, 433.05it/s]


,Gene_start_end,Exon_list,5_prime_UTR,3_prime_UTR,Exons_length,Intron_list,Exon_size_total,Bin_size_exon,Intron_length,Intron_size_total,Bin_size_intron,Gene_body_size,Bin_size_GB,Bin_size_UpDown,5_prime_UTR_size,3_prime_UTR_size,Bin_size_5_prime_UTR,Bin_size_3_prime_UTR
ACAP3,1292384_1307930,"[1293564_1293708, 1293823_1293933, 1294090_1294199, 1294402_1294628, 1294718_1294816, 1295447_1295554, 1295736_1295938, 1296015_1296109, 1296211_1296280, 1296425_1296633, 1297822_1297933, 1298013_...",1292384_1293563,1307816_1307930,"[144, 110, 109, 226, 98, 107, 202, 94, 69, 208, 111, 100, 51, 112, 11, 74, 95, 44, 183, 58, 53, 119, 57, 46]","[1293709_1293822, 1293934_1294089, 1294200_1294401, 1294629_1294717, 1294817_1295446, 1295555_1295735, 1295939_1296014, 1296110_1296210, 1296281_1296424, 1296634_1297821, 1297934_1298012, 1298114_...",2481,124,"[113, 155, 201, 88, 629, 180, 75, 100, 143, 1187, 78, 255, 144, 664, 473, 66, 88, 305, 1294, 874, 185, 803, 3624]",11724,586,15546,777,250,1179,114,59,6
AGRN,1020102_1056119,"[1020173_1020373, 1022201_1022462, 1034556_1034703, 1035277_1035324, 1040665_1040880, 1041173_1041397, 1041478_1041702, 1041956_1042162, 1043239_1043457, 1043538_1043732, 1043823_1044023, 1044109_...",1020102_1020172,1054982_1056119,"[200, 261, 147, 47, 215, 224, 224, 206, 218, 194, 200, 148, 105, 116, 164, 143, 124, 105, 338, 137, 127, 114, 119, 353, 192, 215, 229, 134, 96, 164, 111, 11, 116, 192, 87, 23, 32, 224, 103, 157]","[1020374_1022200, 1022463_1034555, 1034704_1035276, 1035325_1040664, 1040881_1041172, 1041398_1041477, 1041703_1041955, 1042163_1043238, 1043458_1043537, 1043733_1043822, 1044024_1044108, 1044258_...",6315,316,"[1826, 12092, 572, 5339, 291, 79, 252, 1075, 79, 89, 84, 75, 720, 80, 208, 86, 70, 130, 83, 368, 117, 87, 115, 500, 175, 113, 106, 194, 96, 133, 193, 208, 82, 81, 176, 1443, 258, 469, 271]",28415,1421,36017,1801,250,70,1137,4,57
ANKRD65,1418420_1422471,"[1419100_1419549, 1420052_1420592, 1420797_1421005]",1418420_1419099,1421006_1422471,"[449, 540, 208]","[1419550_1420051, 1420593_1420796]",1197,60,"[501, 203]",704,35,4051,203,250,679,1465,34,73
ATAD3A,1512143_1534686,"[1512269_1512473, 1516012_1516088, 1517167_1517412, 1517716_1517775, 1518921_1518990, 1520141_1520306, 1520548_1520617, 1522744_1522899, 1523511_1523567, 1523839_1523964, 1524273_1524397, 1525240_...",1512143_1512268,1534073_1534686,"[204, 76, 245, 59, 69, 165, 69, 155, 56, 125, 124, 51, 70, 167, 108, 146]","[1512474_1516011, 1516089_1517166, 1517413_1517715, 1517776_1518920, 1518991_1520140, 1520307_1520547, 1520618_1522743, 1522900_1523510, 1523568_1523838, 1523965_1524272, 1524398_1525239, 1525292_...",1889,94,"[3537, 1077, 302, 1144, 1149, 240, 2125, 610, 270, 307, 841, 1168, 1162, 1359, 4593]",19884,994,22543,1127,250,125,613,6,31
ATAD3B,1471732_1509466,"[1471885_1472089, 1477274_1477350, 1478500_1478745, 1479049_1479108, 1480867_1480936, 1482138_1482303, 1482545_1482614, 1485016_1485171, 1485782_1485838, 1486110_1486235, 1486544_1486668, 1487863_...",1471732_1471884,1495818_1509466,"[204, 76, 245, 59, 69, 165, 69, 155, 56, 125, 124, 51, 70, 167, 108, 332]","[1472090_1477273, 1477351_1478499, 1478746_1479048, 1479109_1480866, 1480937_1482137, 1482304_1482544, 1482615_1485015, 1485172_1485781, 1485839_1486109, 1486236_1486543, 1486669_1487862, 1487915_...",2075,104,"[5183, 1148, 302, 1757, 1200, 240, 2400, 609, 270, 307, 1193, 1288, 981, 137, 4812]",21827,1091,37734,1887,250,152,13648,8,682
ATAD3C,1449689_1470163,"[1450684_1450758, 1452046_1452122, 1452365_1452434, 1454345_1454500, 1455460_1455519, 1455791_1455916, 1456225_1456349, 1457129_1457180, 1459161_1459231, 1460750_1460917, 1462600_1462708, 1468384_...",1449689_1450683,1468531_1470163,"[74, 76, 69, 155, 59, 125, 124, 51, 70, 167, 108, 146]","[1450759_1452045, 1452123_1452364, 1452435_1454344, 1454501_1455459, 1455520_1455790, 1455917_1456224, 1456350_1457128, 1457181_1459160, 1459232_1460749, 1460918_1462599, 1462709_1468383]",1224,61,"[1286, 241, 1909, 958, 270, 3

In [ ]:
l = list()
for j, index in tqdm(enumerate(list(refseq_38_df_pc_genes.index))):
#     if j == 10:
#         break

    if j < len(list(refseq_38_df_pc_genes.index)) - 1 :
        start_index, stop_index = index, list(refseq_38_df_pc_genes.index)[j+1]

        current_gene = [e.replace('Name=', '') for e in refseq_38_df_pc_genes.loc[start_index]['Attributes'].split(';') if 'Name' in e][0]
        for cds in list(refseq_38_df_cds.index):
            if cds > start_index and cds < stop_index:

                current_gene_cds = [e.replace('gene=', '') for e in refseq_38_df_cds.loc[cds]['Attributes'].split(';') if 'gene' in e][0]

#                 print(j, start_index, stop_index, cds, current_gene, current_gene_cds)

                if current_gene == current_gene_cds:
                    l.append(
                        {
                            'Gene' : current_gene,
                            'Gene_start' : refseq_38_df_pc_genes.loc[start_index]['Start'],
                            'Gene_stop' : refseq_38_df_pc_genes.loc[start_index]['End'],
                            'Exon_start' : refseq_38_df_cds.loc[cds]['Start'],
                            'Exon_stop' : refseq_38_df_cds.loc[cds]['End'],
                        }
                    )
        
            
pd.options.display.max_rows =200
df = pd.DataFrame(l).sort_values(by=['Gene', 'Exon_start', 'Exon_stop']).drop_duplicates()
df['Length'] = df['Exon_stop'] - df['Exon_start']
df = df.sort_values(by=['Gene', 'Exon_start', 'Exon_stop', 'Length'], ascending=[True, True, True, False])
df = df.drop_duplicates(subset=['Gene', 'Exon_start'], keep='last').drop_duplicates(subset=['Gene', 'Exon_stop'], keep='first')

df

output_json = dict()

for ig, gene in tqdm(enumerate(df.Gene.unique())):
    if ig == 10:
        break

    output_json[gene] = dict()

    tmp_gene_df = df.loc[df['Gene'] == gene]
    for chrom in chroms:
        if chrom[0] <= tmp_gene_df.index[0] < chrom[1]:
            
            output_json[gene]['CHROM'] = [c.replace('chromosome=', '') for c in refseq_38_df_chroms.loc[chrom[0]]['Attributes'].split(';') if 'chromosome=' in c][0]
# tmp_gene_df
    
    tmp_gene_df = tmp_gene_df.reset_index(drop=True)
    output_json[gene]['Gene_start_end'] = str(tmp_gene_df.Gene_start.unique()[0]) + '_' + str(tmp_gene_df.Gene_stop.unique()[0])
    for j, row in tmp_gene_df.iterrows():
        if 'Exon_list' not in output_json[gene]:
            output_json[gene]['Exon_list'] = list()
        if j == 0:
            output_json[gene]['5_prime_UTR'] = str(tmp_gene_df.Gene_start.unique()[0]) + '_' + str(row.Exon_start - 1)
            output_json[gene]['Exon_list'].append(str(row.Exon_start) + '_' + str(row.Exon_stop))
            if j == (tmp_gene_df.shape[0] - 1):
                output_json[gene]['3_prime_UTR'] = str(row.Exon_stop + 1) + '_' + str(tmp_gene_df.Gene_stop.unique()[0])
        elif j > 0 and j < (tmp_gene_df.shape[0] - 1):
            output_json[gene]['Exon_list'].append(str(row.Exon_start) + '_' + str(row.Exon_stop))
        elif j == (tmp_gene_df.shape[0] - 1):
            output_json[gene]['Exon_list'].append(str(row.Exon_start) + '_' + str(row.Exon_stop))
            output_json[gene]['3_prime_UTR'] = str(row.Exon_stop + 1) + '_' + str(tmp_gene_df.Gene_stop.unique()[0])
            
for gene in output_json:
    
    output_json[gene]['Intron_list'] = list()
    output_json[gene]['Exon_size_total'] = sum([int(e.split('_')[1]) - int(e.split('_')[0]) for e in output_json[gene]['Exon_list']])
    output_json[gene]['Bin_size_exon'] = round(output_json[gene]['Exon_size_total'] / 20)
    for j, exon in enumerate(output_json[gene]['Exon_list']):
        if j < len(output_json[gene]['Exon_list'])-1:
            intron_start = int(output_json[gene]['Exon_list'][j].split('_')[1]) + 1
            intron_end = int(output_json[gene]['Exon_list'][j+1].split('_')[0]) - 1
            output_json[gene]['Intron_list'].append('{}_{}'.format(str(intron_start), str(intron_end)))
    output_json[gene]['Intron_size_total'] = sum([int(e.split('_')[1]) - int(e.split('_')[0]) for e in output_json[gene]['Intron_list']])
    output_json[gene]['Bin_size_intron'] = round(output_json[gene]['Intron_size_total'] / 20)
    output_json[gene]['Gene_body_size'] = int(output_json[gene]['Gene_start_end'].split('_')[1]) - int(output_json[gene]['Gene_start_end'].split('_')[0])
    output_json[gene]['Bin_size_GB'] = round(output_json[gene]['Gene_body_size'] / 20)
    
    init_gene_body_bin = int(output_json[gene]['Gene_start_end'].split('_')[0])
    output_json[gene]['Bin_size_UpDown'] = round(5000 / 20)
    output_json[gene]['5_prime_UTR_size'] = int(output_json[gene]['5_prime_UTR'].split('_')[1]) - int(output_json[gene]['5_prime_UTR'].split('_')[0])
    output_json[gene]['3_prime_UTR_size'] = int(output_json[gene]['3_prime_UTR'].split('_')[1]) - int(output_json[gene]['3_prime_UTR'].split('_')[0])

    output_json[gene]['Bin_size_5_prime_UTR'] = round(int(output_json[gene]['5_prime_UTR_size']) / 20)
    output_json[gene]['Bin_size_3_prime_UTR'] = round(int(output_json[gene]['3_prime_UTR_size']) / 20)
    
    output_json[gene]['Gene_body_bins'] = [round((b* output_json[gene]['Bin_size_GB']) + init_gene_body_bin) for b in range(20)]
    output_json[gene]['Gene_body_bins'] = [(e, output_json[gene]['Gene_body_bins'][i+1]) for i, e in enumerate(output_json[gene]['Gene_body_bins']) if i < len(output_json[gene]['Gene_body_bins'])-1]

    output_json[gene]['Upstream_bins'] = sorted([int(output_json[gene]['Gene_start_end'].split('_')[0]) - 1  - round((b* output_json[gene]['Bin_size_UpDown'])) for b in range(20)])
    output_json[gene]['Upstream_bins'] = [(e, output_json[gene]['Upstream_bins'][i+1]) for i, e in enumerate(output_json[gene]['Upstream_bins']) if i < len(output_json[gene]['Upstream_bins'])-1]

    output_json[gene]['Downstream_bins'] = [int(output_json[gene]['Gene_start_end'].split('_')[1]) + 1 + round((b* output_json[gene]['Bin_size_UpDown'])) for b in range(20)]
    output_json[gene]['Downstream_bins'] = [(e, output_json[gene]['Downstream_bins'][i+1]) for i, e in enumerate(output_json[gene]['Downstream_bins']) if i < len(output_json[gene]['Downstream_bins'])-1]


    output_json[gene]['Exon_bins'] = list()
    tmp_bins_list = [sub_pos for exon in output_json[gene]['Exon_list'] for sub_pos in range(int(exon.split('_')[0]), int(exon.split('_')[1]) + 1) ]
    output_json[gene]['Exon_bins'] = [pos for i, pos in enumerate(tmp_bins_list) if (i+1) % output_json[gene]['Bin_size_exon'] == 0]
    output_json[gene]['Exon_bins'] = [output_json[gene]['Exon_list'][0].split('_')[0]] + output_json[gene]['Exon_bins']
    output_json[gene]['Exon_bins'] = [(e, output_json[gene]['Exon_bins'][i+1]) for i, e in enumerate(output_json[gene]['Exon_bins']) if i < len(output_json[gene]['Exon_bins'])-1]
    
    current_size = 0
    previous_size = 0
    exon_bin_counter = 0
    
    for j, exon in enumerate(output_json[gene]['Exon_list']):
                
        if j < len(output_json[gene]['Exon_list'])-1:
            intron_start = int(output_json[gene]['Exon_list'][j].split('_')[1]) + 1
            intron_end = int(output_json[gene]['Exon_list'][j+1].split('_')[0]) - 1
            output_json[gene]['Intron_list'].append('{}_{}'.format(str(intron_start), str(intron_end)))
    
    
    
    output_json[gene]['Intron_bins'] = list()
    if output_json[gene]['Intron_list']:
        tmp_bins_list = [sub_pos for intron in output_json[gene]['Intron_list'] for sub_pos in range(int(intron.split('_')[0]), int(intron.split('_')[1]) + 1) ]
        output_json[gene]['Intron_bins'] = [pos for i, pos in enumerate(tmp_bins_list) if (i+1) % output_json[gene]['Bin_size_intron'] == 0]
        output_json[gene]['Intron_bins'] = [output_json[gene]['Intron_list'][0].split('_')[0]] + output_json[gene]['Intron_bins']
        output_json[gene]['Intron_bins'] = [(e, output_json[gene]['Intron_bins'][i+1]) for i, e in enumerate(output_json[gene]['Intron_bins']) if i < len(output_json[gene]['Intron_bins'])-1]
    
    
    output_json[gene]['BED_boundaries_for_hail_10kbp_added'] = "chr" +  str(output_json[gene]['CHROM']) + ':' + str((int(output_json[gene]['Gene_start_end'].split('_')[0]) - 5000)) + '-' + str((int(output_json[gene]['Gene_start_end'].split('_')[1]) + 5000))

                
                
            

In [14]:
def test(genes, l):

    output_json = dict()
    bins_position = list()
    l = list()

    for ig, gene in enumerate(genes):
#         print(gene)
    #     if ig % 10 == 0:
    #         tmp_df = pd.DataFrame.from_dict(output_json).T
    #         print(tmp_df)
    #         l.append(tmp_df)
    #         output_json = dict()
    #         bins_position = list()

        output_json[gene] = dict()

        tmp_gene_df = df.loc[df['Gene'] == gene]
        for chrom in chroms:
            if chrom[0] <= tmp_gene_df.index[0] < chrom[1]:

                output_json[gene]['CHROM'] = [c.replace('chromosome=', '') for c in refseq_38_df_chroms.loc[chrom[0]]['Attributes'].split(';') if 'chromosome=' in c][0]
    # tmp_gene_df

        tmp_gene_df = tmp_gene_df.reset_index(drop=True)
        output_json[gene]['Gene_start_end'] = str(tmp_gene_df.Gene_start.unique()[0]) + '_' + str(tmp_gene_df.Gene_stop.unique()[0])
        for j, row in tmp_gene_df.iterrows():
            if 'Exon_list' not in output_json[gene]:
                output_json[gene]['Exon_list'] = list()
            if j == 0:
                output_json[gene]['5_prime_UTR'] = str(tmp_gene_df.Gene_start.unique()[0]) + '_' + str(row.Exon_start - 1)
                output_json[gene]['Exon_list'].append(str(row.Exon_start) + '_' + str(row.Exon_stop))
                if j == (tmp_gene_df.shape[0] - 1):
                    output_json[gene]['3_prime_UTR'] = str(row.Exon_stop + 1) + '_' + str(tmp_gene_df.Gene_stop.unique()[0])
            elif j > 0 and j < (tmp_gene_df.shape[0] - 1):
                output_json[gene]['Exon_list'].append(str(row.Exon_start) + '_' + str(row.Exon_stop))
            elif j == (tmp_gene_df.shape[0] - 1):
                output_json[gene]['Exon_list'].append(str(row.Exon_start) + '_' + str(row.Exon_stop))
                output_json[gene]['3_prime_UTR'] = str(row.Exon_stop + 1) + '_' + str(tmp_gene_df.Gene_stop.unique()[0])
    for gene in output_json:
        output_json[gene]['Intron_list'] = list()
        output_json[gene]['Exon_size_total'] = sum([int(e.split('_')[1]) - int(e.split('_')[0]) for e in output_json[gene]['Exon_list']])
        output_json[gene]['Bin_size_exon'] = round(output_json[gene]['Exon_size_total'] / 20)
        for j, exon in enumerate(output_json[gene]['Exon_list']):
            
            if j < len(output_json[gene]['Exon_list'])-1:
                intron_start = int(output_json[gene]['Exon_list'][j].split('_')[1]) + 1
                intron_end = int(output_json[gene]['Exon_list'][j+1].split('_')[0]) - 1
#                 print(j, output_json[gene]['Exon_list'][j].split('_')[1], intron_start, intron_end, output_json[gene]['Exon_list'][j+1].split('_')[0])
                
                output_json[gene]['Intron_list'].append('{}_{}'.format(str(intron_start), str(intron_end)))
        output_json[gene]['Intron_size_total'] = sum([int(e.split('_')[1]) - int(e.split('_')[0]) for e in output_json[gene]['Intron_list']])
        output_json[gene]['Bin_size_intron'] = round(output_json[gene]['Intron_size_total'] / 20)
        output_json[gene]['Gene_body_size'] = int(output_json[gene]['Gene_start_end'].split('_')[1]) - int(output_json[gene]['Gene_start_end'].split('_')[0])
        output_json[gene]['Bin_size_GB'] = round(output_json[gene]['Gene_body_size'] / 20)

        init_gene_body_bin = int(output_json[gene]['Gene_start_end'].split('_')[0])
        output_json[gene]['Bin_size_UpDown'] = round(5000 / 20)
        output_json[gene]['5_prime_UTR_size'] = int(output_json[gene]['5_prime_UTR'].split('_')[1]) - int(output_json[gene]['5_prime_UTR'].split('_')[0])
        output_json[gene]['3_prime_UTR_size'] = int(output_json[gene]['3_prime_UTR'].split('_')[1]) - int(output_json[gene]['3_prime_UTR'].split('_')[0])

        output_json[gene]['Bin_size_5_prime_UTR'] = round(int(output_json[gene]['5_prime_UTR_size']) / 20)
        output_json[gene]['Bin_size_3_prime_UTR'] = round(int(output_json[gene]['3_prime_UTR_size']) / 20)

        output_json[gene]['Gene_body_bins'] = [round((b* output_json[gene]['Bin_size_GB']) + init_gene_body_bin) for b in range(20)]
        output_json[gene]['Gene_body_bins'] = ["{}_{}".format(str(e), str(output_json[gene]['Gene_body_bins'][i+1])) for i, e in enumerate(output_json[gene]['Gene_body_bins']) if i < len(output_json[gene]['Gene_body_bins'])-1]

        output_json[gene]['Upstream_bins'] = sorted([int(output_json[gene]['Gene_start_end'].split('_')[0]) - 1  - round((b* output_json[gene]['Bin_size_UpDown'])) for b in range(20)])
        output_json[gene]['Upstream_bins'] = ["{}_{}".format(str(e), str(output_json[gene]['Upstream_bins'][i+1])) for i, e in enumerate(output_json[gene]['Upstream_bins']) if i < len(output_json[gene]['Upstream_bins'])-1]

        output_json[gene]['Downstream_bins'] = [int(output_json[gene]['Gene_start_end'].split('_')[1]) + 1 + round((b* output_json[gene]['Bin_size_UpDown'])) for b in range(20)]
        output_json[gene]['Downstream_bins'] = ["{}_{}".format(str(e), str(output_json[gene]['Downstream_bins'][i+1])) for i, e in enumerate(output_json[gene]['Downstream_bins']) if i < len(output_json[gene]['Downstream_bins'])-1]


        output_json[gene]['Exon_bins'] = list()
        tmp_bins_list = [sub_pos for exon in output_json[gene]['Exon_list'] for sub_pos in range(int(exon.split('_')[0]), int(exon.split('_')[1]) + 1) ]
        output_json[gene]['Exon_bins'] = [pos for i, pos in enumerate(tmp_bins_list) if (i+1) % output_json[gene]['Bin_size_exon'] == 0]
        output_json[gene]['Exon_bins'] = [output_json[gene]['Exon_list'][0].split('_')[0]] + output_json[gene]['Exon_bins']
        output_json[gene]['Exon_bins'] = ["{}_{}".format(str(e), str(output_json[gene]['Exon_bins'][i+1])) for i, e in enumerate(output_json[gene]['Exon_bins']) if i < len(output_json[gene]['Exon_bins'])-1]

        current_size = 0
        previous_size = 0
        exon_bin_counter = 0

        for j, exon in enumerate(output_json[gene]['Exon_list']):

            if j < len(output_json[gene]['Exon_list'])-1:
                intron_start = int(output_json[gene]['Exon_list'][j].split('_')[1]) + 1
                intron_end = int(output_json[gene]['Exon_list'][j+1].split('_')[0]) - 1
                output_json[gene]['Intron_list'].append('{}_{}'.format(str(intron_start), str(intron_end)))

        for elem in ['Gene_body_bins', 'Upstream_bins', 'Downstream_bins', 'Exon_bins']:
            bins_position.extend([{'CHR' :  'chr' +  output_json[gene]['CHROM'], 'Start' : e.split('_')[0], 'End' : e.split('_')[1], 'Region_type' : elem.replace('_bins', ''), 'Num' : i,  'Gene': gene} for i,e in zip(list(range(1,21)), output_json[gene][elem])])

        output_json[gene]['Intron_bins'] = list()
        if output_json[gene]['Intron_list']:
            tmp_bins_list = [sub_pos for intron in output_json[gene]['Intron_list'] for sub_pos in range(int(intron.split('_')[0]), int(intron.split('_')[1]) + 1) ]
#             print(len(tmp_bins_list))
#             output_json[gene]['Intron_bins'] = [pos for i, pos in enumerate(tmp_bins_list) if (i+1) % output_json[gene]['Bin_size_intron'] == 0]
#             output_json[gene]['Intron_bins'] = [output_json[gene]['Intron_list'][0].split('_')[0]] + output_json[gene]['Intron_bins']
#             output_json[gene]['Intron_bins'] = ["{}_{}".format(str(e), str(output_json[gene]['Intron_bins'][i+1])) for i, e in enumerate(output_json[gene]['Intron_bins']) if i < len(output_json[gene]['Intron_bins'])-1]
#             bins_position.extend([{'CHR' :  'chr' +  output_json[gene]['CHROM'],  'Start' : e.split('_')[0], 'End' : e.split('_')[1], 'Region_type' : elem.replace('_bins', ''), 'Num' : i, 'Gene' :  gene} for i,e in zip(list(range(1,21)), output_json[gene]['Intron_bins'])])
#     #         bins_position.extend([{'Num' : i, 'CHR' :  'chr' +  output_json[gene]['CHROM'], 'Region' : e, 'Region_type' : elem.replace('_bins', '')} for i,e in zip(list(range(1,21)), output_json[gene][elem])])

        output_json[gene]['BED_boundaries_for_hail_10kbp_added'] = "chr" +  str(output_json[gene]['CHROM']) + ':' + str((int(output_json[gene]['Gene_start_end'].split('_')[0]) - 5000)) + '-' + str((int(output_json[gene]['Gene_start_end'].split('_')[1]) + 5000))


#     refseq_38_df_transformed = pd.DataFrame.from_dict(output_json)
    tmp_dict = output_json[gene]
    tmp_dict['Gene'] = gene
    return tmp_dict
# refseq_38_df_transformed = pd.concat(l)
# refseq_38_df_transformed

tmp_list = list()
m = multiprocessing.Manager()
final_list_concat= m.list()
test_list = df.Gene.unique().tolist()[:200]

# tmp_list.append(test(test_list[:1], []))
# pd.DataFrame(tmp_list)

chunks = [test_list[x:x+1] for x in range(0, len(test_list), 1)]
# parmap.starmap(test, list(zip(chunks)), final_list_concat, pm_pbar=True)

for gene in tqdm(test_list):
    tmp_list.append(test([gene], tmp_list))


# for i, gene in enumerate):
#     tmp_list.append(gene)
#     if i % 10 == 0:
#         final_list_concat.append(test(tmp_list))
#         tmp_list = list()
        
tmp2 = pd.DataFrame(list(tmp_list))
tmp2
    

100%|██████████| 200/200 [00:06<00:00, 29.21it/s]


,CHROM,Gene_start_end,Exon_list,5_prime_UTR,3_prime_UTR,Intron_list,Exon_size_total,Bin_size_exon,Intron_size_total,Bin_size_intron,...,3_prime_UTR_size,Bin_size_5_prime_UTR,Bin_size_3_prime_UTR,Gene_body_bins,Upstream_bins,Downstream_bins,Exon_bins,Intron_bins,BED_boundaries_for_hail_10kbp_added,Gene
0,2,58345183_58353492,"[58347022_58347029, 58347353_58347640, 5835037...",58345183_58347021,58353438_58353492,"[58347030_58347352, 58347641_58350369, 5835065...",1480,74,4921,246,...,54,92,3,"[58345183_58345598, 58345598_58346013, 5834601...","[58340432_58340682, 58340682_58340932, 5834093...","[58353493_58353743, 58353743_58353993, 5835399...","[58347022_58347418, 58347418_58347492, 5834749...",[],chr2:58340183-58358492,A1BG
1,2,50799409_50885666,"[50806729_50806880, 50809894_50810042, 5081104...",50799409_50806728,50859941_50885666,"[50806881_50809893, 50810043_50811039, 5081117...",1896,95,51293,2565,...,25725,366,1286,"[50799409_50803722, 50803722_50808035, 5080803...","[50794658_50794908, 50794908_50795158, 5079515...","[50885667_50885917, 50885917_50886167, 5088616...","[50806729_50806823, 50806823_50809931, 5080993...",[],chr2:50794409-50890666,A1CF
2,2,9067708_9116229,"[9067823_9067839, 9068183_9068224, 9068740_906...",9067708_9067822,9115850_9116229,"[9067840_9068182, 9068225_9068739, 9068843_906...",4389,219,43567,2178,...,379,6,19,"[9067708_9070134, 9070134_9072560, 9072560_907...","[9062957_9063207, 9063207_9063457, 9063457_906...","[9116230_9116480, 9116480_9116730, 9116730_911...","[9067823_9069801, 9069801_9072474, 9072474_907...",[],chr2:9062708-9121229,A2M
3,2,8822554_8887202,"[8822652_8822713, 8823182_8823365, 8823720_882...",8822554_8822651,8875012_8887202,"[8822714_8823181, 8823366_8823719, 8823883_882...",4332,217,47957,2398,...,12190,5,610,"[8822554_8825786, 8825786_8829018, 8829018_883...","[8817803_8818053, 8818053_8818303, 8818303_881...","[8887203_8887453, 8887453_8887703, 8887703_888...","[8822652_8823336, 8823336_8829751, 8829751_883...",[],chr2:8817554-8892202,A2ML1
4,1,33306766_33321098,"[33306766_33307453, 33312052_33312189, 3331250...",33306766_33306765,33321099_33321098,"[33307454_33312051, 33312190_33312500, 3331259...",1018,51,13306,665,...,-1,0,0,"[33306766_33307483, 33307483_33308200, 3330820...","[33302015_33302265, 33302265_33302515, 3330251...","[33321099_33321349, 33321349_33321599, 3332159...","[33306766_33306816, 33306816_33306867, 3330686...",[],chr1:33301766-33326098,A3GALT2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2,20536226_20576376,"[20537258_20537362, 20540654_20540773, 2054291...",20536226_20537257,20564846_20576376,"[20537363_20540653, 20540774_20542913, 2054301...",1721,86,25842,1292,...,11530,52,576,"[20536226_20538234, 20538234_20540242, 2054024...","[20531475_20531725, 20531725_20531975, 2053197...","[20576377_20576627, 20576627_20576877, 2057687...","[20537258_20537343, 20537343_20540720, 2054072...",[],chr2:20531226-20581376,ACSM2B
196,2,20742440_20797811,"[20770035_20770253, 20775839_20776049, 2077737...",20742440_20770034,20796973_20797811,"[20770254_20775838, 20776050_20777372, 2077758...",1841,92,25072,1254,...,838,1380,42,"[20742440_20745209, 20745209_20747978, 2074797...","[20737689_20737939, 20737939_20738189, 2073818...","[20797812_20798062, 20798062_20798312, 2079831...","[20770035_20770126, 20770126_20770218, 2077021...",[],chr2:20737440-20802811,ACSM3
197,1,7304332_7328373,"[7304332_7304532, 7306533_7306743, 7310539_731...",7304332_7304331,7328374_7328373,"[7304533_7306532, 7306744_7310538, 7310747_731...",1730,86,22287,1114,...,-1,0,0,"[7304332_7305534, 7305534_7306736, 7306736_730...","[7299581_7299831, 7299831_7300081, 7300081_730...","[7328374_7328624, 7328624_7328874, 7328874_732...","[7304332_7304417, 7304417_7304503, 7304503_730...",[],chr1:7299332-7333373,ACSM4
198,2,20409534_20441336,"[20411485_20411688, 20418059_20418269, 2041922...",20409534_20411484,20440453_20441336,"[20411689_20418058, 20418270_2041

In [21]:
tmp2

""


In [15]:
refseq_38_df_transformed.to_parquet("/gstock/EXOTIC/data/GENOMICS/refseq_38_df_transformed.parquet")

In [ ]:
refseq_38_df_transformed.to_csv("/gstock/EXOTIC/data/GENOMICS/refseq_38_df_transformed.csv.gz", compression='gzip', sep='\t')

In [244]:
bed = pd.DataFrame(bins_position)
bed['Start'] = bed['Start'].astype(int)
bed['End'] = bed['End'].astype(int)
bed = bed.loc[(bed['Start'] > 0 ) & (bed['End'] > 0 )]
bed.to_csv('/gstock/EXOTIC/data/GENOMICS/BED_test_h.txt', sep='\t', index=False)
bed

,CHR,Start,End,Region_type,Num,Gene
0,chr2,58345183,58345598,Gene_body,1,A1BG
1,chr2,58345598,58346013,Gene_body,2,A1BG
2,chr2,58346013,58346428,Gene_body,3,A1BG
3,chr2,58346428,58346843,Gene_body,4,A1BG
4,chr2,58346843,58347258,Gene_body,5,A1BG
...,...,...,...,...,...,...
945,chr1,151824193,151825005,Exon,16,AADAC
946,chr1,151825005,151825645,Exon,17,AADAC
947,chr1,151825645,151826285,Exon,18,AADAC
948,chr1,151826285,151826925,Exon,19,AADAC


In [46]:
refseq_38_df_transformed.to_parquet('/gstock/EXOTIC/data/GENOMICS/RefSeq_38_precomputed.parquet')

In [138]:
refseq_38_df_transformed  = pd.read_parquet('/gstock/EXOTIC/data/GENOMICS/RefSeq_38_precomputed.parquet')
refseq_38_df_transformed

,CHROM,Gene_start_end,Exon_list,5_prime_UTR,3_prime_UTR,Intron_list,Exon_size_total,Bin_size_exon,Intron_size_total,Bin_size_intron,Gene_body_size,Bin_size_GB,Bin_size_UpDown,5_prime_UTR_size,3_prime_UTR_size,Bin_size_5_prime_UTR,Bin_size_3_prime_UTR,BED_boundaries_for_hail_10kbp_added
A1BG,2,58345183_58353492,"[58347022_58347029, 58347353_58347640, 5835037...",58345183_58347021,58353438_58353492,"[58347030_58347352, 58347641_58350369, 5835065...",1480,74.00,4921,246.05,8309,415.45,250.0,1838,54,91.90,2.70,chr2:58340183-58358492
A1CF,1,50799409_50885666,"[50806729_50806880, 50809894_50810042, 5081104...",50799409_50806728,50859941_50885666,"[50806881_50809893, 50810043_50811039, 5081117...",1896,94.80,51293,2564.65,86257,4312.85,250.0,7319,25725,365.95,1286.25,chr1:50794409-50890666
A2M,2,9067708_9116229,"[9067823_9067839, 9068183_9068224, 9068740_906...",9067708_9067822,9115850_9116229,"[9067840_9068182, 9068225_9068739, 9068843_906...",4389,219.45,43567,2178.35,48521,2426.05,250.0,114,379,5.70,18.95,chr2:9062708-9121229
A2ML1,2,8822554_8887202,"[8822652_8822713, 8823182_8823365, 8823720_882...",8822554_8822651,8875012_8887202,"[8822714_8823181, 8823366_8823719, 8823883_882...",4332,216.60,47957,2397.85,64648,3232.40,250.0,97,12190,4.85,609.50,chr2:8817554-8892202
A3GALT2,1,33306766_33321098,"[33306766_33307453, 33312052_33312189, 3331250...",33306766_33306765,33321099_33321098,"[33307454_33312051, 33312190_33312500, 3331259...",1018,50.90,13306,665.30,14332,716.60,250.0,-1,-1,-0.05,-0.05,chr1:33301766-33326098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,1,52842736_52894998,"[52842884_52842973, 52854465_52854630, 5285699...",52842736_52842883,52892958_52894998,"[52842974_52854464, 52854631_52856997, 5285775...",2266,113.30,47781,2389.05,52262,2613.10,250.0,147,2040,7.35,102.00,chr1:52837736-52899998
ZYG11B,1,52726453_52827336,"[52726654_52726683, 52756458_52756623, 5277102...",52726453_52726653,52821630_52827336,"[52726684_52756457, 52756624_52771019, 5277177...",2221,111.05,92728,4636.40,100883,5044.15,250.0,200,5706,10.00,285.30,chr1:52721453-52832336
ZYX,2,709_10475,"[936_1143, 1612_1811, 1957_2049, 2165_2686, 75...",709_935,143390683_10475,"[1144_1611, 1812_1956, 2050_2164, 2687_7582, 7...",3420,171.00,143386292,7169314.60,9766,488.30,250.0,226,-143380208,11.30,-7169010.40,chr2:-4291-15475
ZZEF1,2,4004445_4143030,"[4006890_4006970, 4008883_4008954, 4009604_400...",4004445_4006889,4142896_4143030,"[4006971_4008882, 4008955_4009603, 4009758_401...",8831,441.55,127066,6353.30,138585,6929.25,250.0,2444,134,122.20,6.70,chr2:3999445-4148030


In [109]:
biomart_raw = pd.read_csv('/gstock/EXOTIC/data/OTHERS/biomart_with_strand.txt.gz', compression='gzip', sep='\t')
# biomart_raw['Gene_start_end_ENS'] = biomart_raw['Gene start (bp)'].astype(str) + '_' + biomart_raw['Gene end (bp)'].astype(str)
biomart_raw

,Gene stable ID,Transcript stable ID,Strand,Gene name,Transcript type,Gene type,Gene % GC content,APPRIS annotation,Transcript support level (TSL),Transcript length (including UTRs and CDS),Transcription start site (TSS),Transcript end (bp),Transcript start (bp),Gene end (bp),Gene start (bp),Transcript count,RefSeq mRNA ID,HGNC ID
0,ENSG00000210049,ENST00000387314,1,MT-TF,Mt_tRNA,Mt_tRNA,40.85,NaN,tslNA,71,577,647,577,647,577,1,NaN,HGNC:7481
1,ENSG00000211459,ENST00000389680,1,MT-RNR1,Mt_rRNA,Mt_rRNA,45.49,NaN,tslNA,954,648,1601,648,1601,648,1,NaN,HGNC:7470
2,ENSG00000210077,ENST00000387342,1,MT-TV,Mt_tRNA,Mt_tRNA,42.03,NaN,tslNA,69,1602,1670,1602,1670,1602,1,NaN,HGNC:7500
3,ENSG00000210082,ENST00000387347,1,MT-RNR2,Mt_rRNA,Mt_rRNA,42.81,NaN,tslNA,1559,1671,3229,1671,3229,1671,1,NaN,HGNC:7471
4,ENSG00000209082,ENST00000386347,1,MT-TL1,Mt_tRNA,Mt_tRNA,38.67,NaN,tslNA,75,3230,3304,3230,3304,3230,1,NaN,HGNC:7490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276111,ENSG00000185220,ENST00000355360,1,PGBD2,protein_coding,protein_coding,46.25,NaN,tsl1,2879,248906196,248919946,248906196,248919946,248906196,3,NM_001017434,HGNC:19399
276112,ENSG00000185220,ENST00000329291,1,PGBD2,protein_coding,protein_coding,46.25,principal1,tsl1 (assigned to previous version 5),2717,248906235,248919146,248906235,248919946,248906196,3,NM_170725,HGNC:19399
276113,ENSG00000185220,ENST00000462488,1,PGBD2,processed_transcript,protein_coding,46.25,NaN,tsl5,282,248906372,248917401,248906372,248919946,248906196,3,NaN,HGNC:19399
276114,ENSG00000200495,ENST00000363625,-1,RNU6-1205P,snRNA,snRNA,40.57,NaN,tslNA,106,248912795,248912795,248912690,248912795,248912690,1,NaN,HGNC:48168


In [111]:
biomart_raw[['RefSeq mRNA ID', 'Strand']].dropna()

,RefSeq mRNA ID,Strand
54,NM_001318854,1
55,NM_001318855,1
58,NM_005286,-1
64,NM_004535,1
69,NM_001104925,1
...,...,...
276082,NM_001193328,-1
276084,NM_001350073,-1
276088,NM_001136036,-1
276111,NM_001017434,1


In [185]:
merge_biomart_refseq_38 = pd.merge(biomart_raw.loc[biomart_raw['Gene stable ID'].isin(exotic_genes_not_sqtl), ['Gene stable ID', 'Gene name']].drop_duplicates().sort_values(by='Gene name'), refseq_38_df_transformed.reset_index(), left_on='Gene name', right_on='index')
merge_biomart_refseq_38['Gene_start'] = merge_biomart_refseq_38.Gene_start_end.apply(lambda r: r.split('_')[0])
merge_biomart_refseq_38['Gene_start'] = merge_biomart_refseq_38['Gene_start'].astype(int)
merge_biomart_refseq_38['Gene_end'] = merge_biomart_refseq_38.Gene_start_end.apply(lambda r: r.split('_')[1])
merge_biomart_refseq_38['Gene_end'] = merge_biomart_refseq_38['Gene_end'].astype(int)
merge_biomart_refseq_38

,Gene stable ID,Gene name,index,CHROM,Gene_start_end,Exon_list,5_prime_UTR,3_prime_UTR,Intron_list,Exon_size_total,...,Bin_size_3_prime_UTR,Gene_body_bins,Upstream_bins,Downstream_bins,Exon_bins,Bins_position,Intron_bins,BED_boundaries_for_hail_10kbp_added,Gene_start,Gene_end
0,ENSG00000175899,A2M,A2M,2,9067708_9116229,"[9067823_9067839, 9068183_9068224, 9068740_9068842, 9069745_9069813, 9070488_9070578, 9072359_9072486, 9072653_9072871, 9074560_9074783, 9076756_9076936, 9077346_9077420, 9077701_9077857, 9079244_...",9067708_9067822,9115850_9116229,"[9067840_9068182, 9068225_9068739, 9068843_9069744, 9069814_9070487, 9070579_9072358, 9072487_9072652, 9072872_9074559, 9074784_9076755, 9076937_9077345, 9077421_9077700, 9077858_9079243, 9079332_...",4389,...,19,"[9067708_9070134, 9070134_9072560, 9072560_9074986, 9074986_9077412, 9077412_9079838, 9079838_9082264, 9082264_9084690, 9084690_9087116, 9087116_9089542, 9089542_9091968, 9091968_9094394, 9094394_...","[9062957_9063207, 9063207_9063457, 9063457_9063707, 9063707_9063957, 9063957_9064207, 9064207_9064457, 9064457_9064707, 9064707_9064957, 9064957_9065207, 9065207_9065457, 9065457_9065707, 9065707_...","[9116230_9116480, 9116480_9116730, 9116730_9116980, 9116980_9117230, 9117230_9117480, 9117480_9117730, 9117730_9117980, 9117980_9118230, 9118230_9118480, 9118480_9118730, 9118730_9118980, 9118980_...","[9067823_9069801, 9069801_9072474, 9072474_9072859, 9072859_9074766, 9074766_9077366, 9077366_9079251, 9079251_9079777, 9079777_9089946, 9089946_9091215, 9091215_9093469, 9093469_9095081, 9095081_...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]","[9067840_9070231, 9070231_9072628, 9072628_9075249, 9075249_9077683, 9077683_9080367, 9080367_9082545, 9082545_9084723, 9084723_9086901, 9086901_9089079, 9089079_9091787, 9091787_9094080, 9094080_...",chr2:9062708-9121229,9067708,9116229
1,ENSG00000166535,A2ML1,A2ML1,2,8822554_8887202,"[8822652_8822713, 8823182_8823365, 8823720_8823882, 8829727_8829779, 8834662_8834682, 8835507_8835666, 8836255_8836339, 8837440_8837566, 8838336_8838450, 8839113_8839222, 8841369_8841536, 8843134_...",8822554_8822651,8875012_8887202,"[8822714_8823181, 8823366_8823719, 8823883_8829726, 8829780_8834661, 8834683_8835506, 8835667_8836254, 8836340_8837439, 8837567_8838335, 8838451_8839112, 8839223_8841368, 8841537_8843133, 8843362_...",4332,...,610,"[8822554_8825786, 8825786_8829018, 8829018_8832250, 8832250_8835482, 8835482_8838714, 8838714_8841946, 8841946_8845178, 8845178_8848410, 8848410_8851642, 8851642_8854874, 8854874_8858106, 8858106_...","[8817803_8818053, 8818053_8818303, 8818303_8818553, 8818553_8818803, 8818803_8819053, 8819053_8819303, 8819303_8819553, 8819553_8819803, 8819803_8820053, 8820053_8820303, 8820303_8820553, 8820553_...","[8887203_8887453, 8887453_8887703, 8887703_8887953, 8887953_8888203, 8888203_8888453, 8888453_8888703, 8888703_8888953, 8888953_8889203, 8889203_8889453, 8889453_8889703, 8889703_8889953, 8889953_...","[8822652_8823336, 8823336_8829751, 8829751_8836262, 8836262_8838348, 8838348_8841373, 8841373_8843187, 8843187_8845481, 8845481_8847598, 8847598_8848836, 8848836_8850207, 8850207_8851933, 8851933_...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]","[8822714_8825458, 8825458_8827856, 8827856_8830307, 8830307_8832705, 8832705_8835124, 8835124_8837894, 8837894_8840517, 8840517_8843083, 8843083_8845773, 8845773_8848467, 8848467_8851266, 8851266_...",chr2:8817554-8892202,8822554,8887202


In [137]:
merge_biomart_refseq_38

NameError: name 'merge_biomart_refseq_38' is not defined

# LiftOver

In [58]:
from liftover import ChainFile
converter = ChainFile("/gstock/biolo_datasets/variation/benchmark/Databases/CrossMap/GRCh38_to_GRCh37.chain.gz", "GRCh38", "GRCh37")

def apply_liftover(r, name):
#     print(r)
    try:
        output = converter[r["CHROM"]][int(r[name])]
        # print(output)
        converted_pos = output[0][1]
    except KeyError:
        converted_pos = np.nan
    except IndexError:
        converted_pos = np.nan
    return converted_pos

# EXOTIC

In [88]:
exotic = pd.read_parquet('/gstock/EXOTIC/data/EXOTIC/EXOTIC_min_max_processed.parquet')
exotic[['Exon_start', 'Exon_end']] = exotic['Exon'].str.split('-', expand=True)

# COMPUTE BINS EXOTIC-MAX
bins = np.arange(0.5,1.05,0.05)
labels = bins.copy()
labels_ratio = [str(round(labels[j], 3)) + " - " + str(round(labels[j + 1], 3)) for j in range(len(labels) - 1)]

# COMPUTE BINS COLUMNS
exotic["EXOTIC_bins_max"] = pd.cut(exotic["EXOTIC_max"], bins=bins, labels=labels_ratio, include_lowest=True)

# COMPUTE BINS EXOTIC-MIN
bins = np.arange(0,0.55,0.05)
labels = bins.copy()
labels_ratio = [str(round(labels[j], 3)) + " - " + str(round(labels[j + 1], 3)) for j in range(len(labels) - 1)]

# COMPUTE BINS COLUMNS
exotic["EXOTIC_bins_min"] = pd.cut(exotic["EXOTIC_min"], bins=bins, labels=labels_ratio, include_lowest=True)
exotic





,symbol,ensg,MAP,Exon,mean_proportion,EXOTIC_min,EXOTIC_tissues_min,EXOTIC_pext_associated_min,EXOTIC_check_pext_min,EXOTIC_tissues_corrected_min,...,EXOTIC_max,EXOTIC_tissues_max,EXOTIC_pext_associated_max,EXOTIC_check_pext_max,EXOTIC_tissues_corrected_max,EXOTIC_tissues_nb_max,Exon_start,Exon_end,EXOTIC_bins_max,EXOTIC_bins_min
0,A2ML1,ENSG00000166535,A2ML1_8975248-8975309,8975248-8975309,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17518],[True],[Brain - Cerebellar Hemisphere],...,0.731334,"[Adipose - Subcutaneous, Adipose - Visceral (Omentum), Adrenal Gland, Artery - Aorta, Artery - Coronary, Artery - Tibial, Breast - Mammary Tissue, Cells - Cultured fibroblasts, Cells - EBV-transfo...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]","[Adipose - Subcutaneous, Adipose - Visceral (Omentum), Adrenal Gland, Artery - Aorta, Artery - Coronary, Artery - Tibial, Breast - Mammary Tissue, Cells - Cultured fibroblasts, Cells - EBV-transfo...",24,8975248,8975309,0.7 - 0.75,0.15 - 0.2
1,A2ML1,ENSG00000166535,A2ML1_8975778-8975961,8975778-8975961,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17518],[True],[Brain - Cerebellar Hemisphere],...,0.731334,"[Adipose - Subcutaneous, Adipose - Visceral (Omentum), Adrenal Gland, Artery - Aorta, Artery - Coronary, Artery - Tibial, Breast - Mammary Tissue, Cells - Cultured fibroblasts, Cells - EBV-transfo...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]","[Adipose - Subcutaneous, Adipose - Visceral (Omentum), Adrenal Gland, Artery - Aorta, Artery - Coronary, Artery - Tibial, Breast - Mammary Tissue, Cells - Cultured fibroblasts, Cells - EBV-transfo...",24,8975778,8975961,0.7 - 0.75,0.15 - 0.2
2,A2ML1,ENSG00000166535,A2ML1_8976316-8976478,8976316-8976478,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17518],[True],[Brain - Cerebellar Hemisphere],...,0.731334,"[Adipose - Subcutaneous, Adipose - Visceral (Omentum), Adrenal Gland, Artery - Aorta, Artery - Coronary, Artery - Tibial, Breast - Mammary Tissue, Cells - Cultured fibroblasts, Cells - EBV-transfo...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]","[Adipose - Subcutaneous, Adipose - Visceral (Omentum), Adrenal Gland, Artery - Aorta, Artery - Coronary, Artery - Tibial, Breast - Mammary Tissue, Cells - Cultured fibroblasts, Cells - EBV-transfo...",24,8976316,8976478,0.7 - 0.75,0.15 - 0.2
3,A2ML1,ENSG00000166535,A2ML1_8982323-8982375,8982323-8982375,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17518],[True],[Brain - Cerebellar Hemisphere],...,0.731334,"[Adipose - Subcutaneous, Adipose - Visceral (Omentum), Adrenal Gland, Artery - Aorta, Artery - Coronary, Artery - Tibial, Breast - Mammary Tissue, Cells - Cultured fibroblasts, Cells - EBV-transfo...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]","[Adipose - Subcutaneous, Adipose - Visceral (Omentum), Adrenal Gland, Artery - Aorta, Artery - Coronary, Artery - Tibial, Breast - Mammary Tissue, Cells - Cultured fibroblasts, Cells - EBV-transfo...",24,8982323,8982375,0.7 - 0.75,0.15 - 0.2
4,A2ML1,ENSG00000166535,A2ML1_8987258-8987278,8987258-8987278,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17

In [96]:
biomart = pd.read_csv('/gstock/EXOTIC/data/OTHERS/biomart_refseq_ensembl_hgnc.txt.gz', compression='gzip', sep='\t')
biomart.head()

/home/weber/.conda/envs/ExoCarto/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Gene stable ID,Gene end (bp),Gene start (bp),Gene name,Transcript start (bp),Transcript end (bp),Transcription start site (TSS),Transcript stable ID,Transcript support level (TSL),APPRIS annotation,Transcript type,Gene type,HGNC ID,RefSeq mRNA ID,Chromosome/scaffold name
0,ENSG00000198888,4262,3307,MT-ND1,3307,4262,3307,ENST00000361390,tslNA,principal1,protein_coding,protein_coding,HGNC:7455,NaN,MT
1,ENSG00000198763,5511,4470,MT-ND2,4470,5511,4470,ENST00000361453,tslNA,principal1,protein_coding,protein_coding,HGNC:7456,NaN,MT
2,ENSG00000198804,7445,5904,MT-CO1,5904,7445,5904,ENST00000361624,tslNA,principal1,protein_coding,protein_coding,HGNC:7419,NaN,MT
3,ENSG00000198712,8269,7586,MT-CO2,7586,8269,7586,ENST00000361739,tslNA,principal1,protein_coding,protein_coding,HGNC:7421,NaN,MT
4,ENSG00000228253,8572,8366,MT-ATP8,8366,8572,8366,ENST00000361851,tslNA,principal1,protein_coding,protein_coding,HGNC:7415,NaN,MT


In [97]:
merge_gene_chr = pd.merge(biomart[['Chromosome/scaffold name', 'Gene name']].drop_duplicates().rename({'Gene name' : 'symbol'}, axis=1), exotic[['symbol']].drop_duplicates(), on='symbol')
merge_gene_chr['Chromosome/scaffold name'] = merge_gene_chr['Chromosome/scaffold name'].astype(str)
merge_gene_chr =  merge_gene_chr.loc[~merge_gene_chr['Chromosome/scaffold name'].str.contains('CHR')]
merge_gene_chr.columns = ['CHROM', 'symbol']
exotic = pd.merge(exotic, merge_gene_chr, on='symbol')
exotic.head()


,symbol,ensg,MAP,Exon,mean_proportion,EXOTIC_min,EXOTIC_tissues_min,EXOTIC_pext_associated_min,EXOTIC_check_pext_min,EXOTIC_tissues_corrected_min,...,EXOTIC_tissues_max,EXOTIC_pext_associated_max,EXOTIC_check_pext_max,EXOTIC_tissues_corrected_max,EXOTIC_tissues_nb_max,Exon_start,Exon_end,EXOTIC_bins_max,EXOTIC_bins_min,CHROM
0,A2ML1,ENSG00000166535,A2ML1_8975248-8975309,8975248-8975309,0.69514,0.15343,[Brain - Cerebellar Hemisphere],[0.17518],[True],[Brain - Cerebellar Hemisphere],...,"[Adipose - Subcutaneous, Adipose - Visceral (O...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[True, True, True, True, True, True, True, Tru...","[Adipose - Subcutaneous, Adipose - Visceral (O...",24,8975248,8975309,0.7 - 0.75,0.15 - 0.2,12
1,A2ML1,ENSG00000166535,A2ML1_8975778-8975961,8975778-8975961,0.69514,0.15343,[Brain - Cerebellar Hemisphere],[0.17518],[True],[Brain - Cerebellar Hemisphere],...,"[Adipose - Subcutaneous, Adipose - Visceral (O...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[True, True, True, True, True, True, True, Tru...","[Adipose - Subcutaneous, Adipose - Visceral (O...",24,8975778,8975961,0.7 - 0.75,0.15 - 0.2,12
2,A2ML1,ENSG00000166535,A2ML1_8976316-8976478,8976316-8976478,0.69514,0.15343,[Brain - Cerebellar Hemisphere],[0.17518],[True],[Brain - Cerebellar Hemisphere],...,"[Adipose - Subcutaneous, Adipose - Visceral (O...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[True, True, True, True, True, True, True, Tru...","[Adipose - Subcutaneous, Adipose - Visceral (O...",24,8976316,8976478,0.7 - 0.75,0.15 - 0.2,12
3,A2ML1,ENSG00000166535,A2ML1_8982323-8982375,8982323-8982375,0.69514,0.15343,[Brain - Cerebellar Hemisphere],[0.17518],[True],[Brain - Cerebellar Hemisphere],...,"[Adipose - Subcutaneous, Adipose - Visceral (O...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[True, True, True, True, True, True, True, Tru...","[Adipose - Subcutaneous, Adipose - Visceral (O...",24,8982323,8982375,0.7 - 0.75,0.15 - 0.2,12
4,A2ML1,ENSG00000166535,A2ML1_8987258-8987278,8987258-8987278,0.69514,0.15343,[Brain - Cerebellar Hemisphere],[0.17518],[True],[Brain - Cerebellar Hemisphere],...,"[Adipose - Subcutaneous, Adipose - Visceral (O...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[True, True, True, True, True, True, True, Tru...","[Adipose - Subcutaneous, Adipose - Visceral (O...",24,8987258,8987278,0.7 - 0.75,0.15 - 0.2,12


In [44]:
tqdm.pandas()
exotic['Exon_start_38'] = exotic[['CHROM', 'Exon_start']].progress_apply(lambda r: apply_liftover(r, 'Exon_start'), axis=1)
exotic['Exon_end_38'] = exotic[['CHROM', 'Exon_end']].progress_apply(lambda r: apply_liftover(r, 'Exon_end'), axis=1)


/home/weber/.conda/envs/ExoCarto/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 27525/27525 [00:00<00:00, 86380.17it/s]


In [94]:
exotic

,symbol,ensg,MAP,Exon,mean_proportion,EXOTIC_min,EXOTIC_tissues_min,EXOTIC_pext_associated_min,EXOTIC_check_pext_min,EXOTIC_tissues_corrected_min,...,EXOTIC_max,EXOTIC_tissues_max,EXOTIC_pext_associated_max,EXOTIC_check_pext_max,EXOTIC_tissues_corrected_max,EXOTIC_tissues_nb_max,Exon_start,Exon_end,EXOTIC_bins_max,EXOTIC_bins_min
0,A2ML1,ENSG00000166535,A2ML1_8975248-8975309,8975248-8975309,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17518],[True],[Brain - Cerebellar Hemisphere],...,0.731334,"[Adipose - Subcutaneous, Adipose - Visceral (O...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[True, True, True, True, True, True, True, Tru...","[Adipose - Subcutaneous, Adipose - Visceral (O...",24,8975248,8975309,0.7 - 0.75,0.15 - 0.2
1,A2ML1,ENSG00000166535,A2ML1_8975778-8975961,8975778-8975961,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17518],[True],[Brain - Cerebellar Hemisphere],...,0.731334,"[Adipose - Subcutaneous, Adipose - Visceral (O...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[True, True, True, True, True, True, True, Tru...","[Adipose - Subcutaneous, Adipose - Visceral (O...",24,8975778,8975961,0.7 - 0.75,0.15 - 0.2
2,A2ML1,ENSG00000166535,A2ML1_8976316-8976478,8976316-8976478,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17518],[True],[Brain - Cerebellar Hemisphere],...,0.731334,"[Adipose - Subcutaneous, Adipose - Visceral (O...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[True, True, True, True, True, True, True, Tru...","[Adipose - Subcutaneous, Adipose - Visceral (O...",24,8976316,8976478,0.7 - 0.75,0.15 - 0.2
3,A2ML1,ENSG00000166535,A2ML1_8982323-8982375,8982323-8982375,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17518],[True],[Brain - Cerebellar Hemisphere],...,0.731334,"[Adipose - Subcutaneous, Adipose - Visceral (O...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[True, True, True, True, True, True, True, Tru...","[Adipose - Subcutaneous, Adipose - Visceral (O...",24,8982323,8982375,0.7 - 0.75,0.15 - 0.2
4,A2ML1,ENSG00000166535,A2ML1_8987258-8987278,8987258-8987278,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17518],[True],[Brain - Cerebellar Hemisphere],...,0.731334,"[Adipose - Subcutaneous, Adipose - Visceral (O...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[True, True, True, True, True, True, True, Tru...","[Adipose - Subcutaneous, Adipose - Visceral (O...",24,8987258,8987278,0.7 - 0.75,0.15 - 0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27533,ZXDC,ENSG00000070476,ZXDC_126158475-126158570,126158475-126158570,0.20989,0.150855,[Brain - Nucleus accumbens (basal ganglia)],[0.11398],[True],[Brain - Nucleus accumbens (basal ganglia)],...,0.912582,[Muscle - Skeletal],[0.34009],[True],[Muscle - Skeletal],1,126158475,126158570,0.9 - 0.95,0.15 - 0.2
27534,ZXDC,ENSG00000070476,ZXDC_126160608-126160789,126160608-126160789,0.20989,0.150855,[Brain - Nucleus accumbens (basal ganglia)],[0.11398],[True],[Brain - Nucleus accumbens (basal ganglia)],...,0.912582,[Muscle - Skeletal],[0.34009],[True],[Muscle - Skeletal],1,126160608,126160789,0.9 - 0.95,0.15 - 0.2
27535,ZXDC,ENSG00000070476,ZXDC_126178496-126178580,126178496-126178580,0.20989,0.150855,[Brain - Nucleus accumbens (basal ganglia)],[0.11398],[True],[Brain - Nucleus accumbens (basal ganglia)],...,0.912582,[Muscle - Skeletal],[0.34009],[True],[Muscle - Skeletal],1,126178496,126178580,0.9 - 0.95,0.15 - 0.2
27536,ZXDC,ENSG00000070476,ZXDC_126180378-126181063,126180378-126181063,0.34803,0.139039,[Whole Blood],[0.20951999999999998],[True],[Whole Blood],...,0.910061,[Artery - Tibial],[0.52384],[True],[Artery - Tibial],1,126180378,126181063,0.9 - 0.95,0.1 - 0.15


In [95]:
exotic[['CHROM', 'Exon_start', 'Exon_end', 'MAP']].sort_values(by=['CHROM', 'Exon_start']).to_csv('/biolo/ngs/remap/EXOTIC.bed', header=False,sep='\t', index=False)


KeyError: "['CHROM'] not in index"

In [100]:
exotic[['CHROM', 'Exon_start', 'Exon_end', 'MAP']].sort_values(by=['CHROM', 'Exon_start']).to_csv('/biolo/ngs/remap/EXOTIC.bed', header=False,sep='\t', index=False)
import subprocess
subprocess.call("/usr/local/bin/perl remap_api.pl --mode asm-asm --from GCF_000001405.25 --dest GCF_000001405.26 --annotation /biolo/ngs/remap/EXOTIC.bed --annot_out /biolo/ngs/remap/EXOTIC_remap.bed --report_out report --in_format bed --out_format bed", shell=True)
exotic_remap_map = pd.read_csv('/biolo/ngs/remap/EXOTIC_remap.bed', sep='\t', names=['CHROM_38', 'Exon_start_38', 'Exon_end_38', 'MAP'])
exotic_remap_map = exotic_remap_map.loc[~exotic_remap_map['CHROM_38'].str.contains('HSCHR')]
exotic_remap_map

2

In [82]:
exotic[['CHROM', 'Exon_start', 'Exon_end', 'MAP']]

,CHROM,Exon_start,Exon_end,MAP
0,12,8975248,8975309,A2ML1_8975248-8975309
1,12,8975778,8975961,A2ML1_8975778-8975961
2,12,8976316,8976478,A2ML1_8976316-8976478
3,12,8982323,8982375,A2ML1_8982323-8982375
4,12,8987258,8987278,A2ML1_8987258-8987278
...,...,...,...,...
27520,3,126158475,126158570,ZXDC_126158475-126158570
27521,3,126160608,126160789,ZXDC_126160608-126160789
27522,3,126178496,126178580,ZXDC_126178496-126178580
27523,3,126180378,126181063,ZXDC_126180378-126181063


In [143]:
refseq_corrected

,Gene,ranges,new_mRNA_exons,new_mRNA_nb,MAP
0,A2M,9220419-9220435,"['NM_001347423', 'NM_001347425', 'NM_000014', 'NM_001347424']",4,A2M_9220419-9220435
1,A2M,9220779-9220820,"['NM_001347423', 'NM_001347425', 'NM_000014', 'NM_001347424']",4,A2M_9220779-9220820
2,A2M,9221336-9221438,"['NM_001347423', 'NM_001347425', 'NM_000014', 'NM_001347424']",4,A2M_9221336-9221438
3,A2M,9222341-9222409,"['NM_001347423', 'NM_001347425', 'NM_000014', 'NM_001347424']",4,A2M_9222341-9222409
4,A2M,9223084-9223174,"['NM_001347423', 'NM_001347425', 'NM_000014', 'NM_001347424']",4,A2M_9223084-9223174
...,...,...,...,...,...
176794,ZZZ3,78047461-78047576,"['NM_001308237', 'NM_015534']",2,ZZZ3_78047461-78047576
176795,ZZZ3,78047664-78047811,"['NM_001308237', 'NM_015534']",2,ZZZ3_78047664-78047811
176796,ZZZ3,78050202-78050340,"['NM_001308237', 'NM_015534']",2,ZZZ3_78050202-78050340
176797,ZZZ3,78097535-78099039,['NM_015534'],1,ZZZ3_78097535-78099039


In [85]:
refseq_corrected = pd.read_csv('/gstock/EXOTIC/data/GENOMICS/RefSeq_corrected_by_GTEx.csv.gz', compression="gzip", sep='\t')
refseq_corrected['MAP'] = refseq_corrected.Gene + '_' + refseq_corrected.ranges.astype(str)
tmp = pd.merge(
    refseq_corrected[['MAP']],
    refseq_37,
    on='MAP'
)
sns.boxenplot(data=tmp, x='Sharing_status', y='Length')
plt.ylim(0,500)


NameError: name 'refseq_37' is not defined

In [28]:
refseq_37.loc[refseq_37['Gene'] == 'ZNF701']

,Gene,Chrom,Start,End,ranges,Length,GeneID,HGNC,CDS_representation,Ratio,Sharing_status,Sources,Count_CDS_alternative,Count_CDS_constitutive,mRNA_IDS,mRNA_nb,MAP
108800,ZNF701,19,53075495,53075621,53075495-53075621,126,55762,25597.0,Unique,1.0/2,False,"[(53075495, 53075621, '1.0/2')]",2,3,"['NM_018260.2', 'NM_001172655.1']",2,ZNF701_53075495-53075621
108801,ZNF701,19,53077330,53077401,53077330-53077401,71,55762,25597.0,Variable_region,1.0/2,True,"[(53077330, 53077415, '1.0/2')]",2,3,"['NM_018260.2', 'NM_001172655.1']",2,ZNF701_53077330-53077401
108802,ZNF701,19,53077401,53077415,53077401-53077415,14,55762,25597.0,Variable_region,2.0/2,True,"[(53077330, 53077415, '1.0/2'), (53077401, 53077415, '1.0/2')]",2,3,"['NM_018260.2', 'NM_001172655.1']",2,ZNF701_53077401-53077415
108803,ZNF701,19,53079148,53079274,53079148-53079274,126,55762,25597.0,Unique,2.0/2,False,"[(53079148, 53079274, '2.0/2')]",2,3,"['NM_018260.2', 'NM_001172655.1']",2,ZNF701_53079148-53079274
108804,ZNF701,19,53085455,53086710,53085455-53086710,1255,55762,25597.0,Unique,2.0/2,False,"[(53085455, 53086710, '2.0/2')]",2,3,"['NM_018260.2', 'NM_001172655.1']",2,ZNF701_53085455-53086710


In [26]:
refseq_37 = pd.read_csv('/gstock/EXOTIC/data/GENOMICS/RefSeq_processed.csv.gz', compression="gzip", sep='\t')
refseq_37['MAP'] = refseq_37.Gene + '_' + refseq_37.Start.astype(str) + '-' + refseq_37.End.astype(str)
refseq_37 = refseq_37.drop_duplicates(subset='MAP')
refseq_37.Sharing_status.value_counts()

/home/weber/.conda/envs/ExoCarto/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


False    187552
True      25050
Name: Sharing_status, dtype: int64

In [ ]:
pd.merge(
    exotic_remap_map.loc[~exotic_remap_map['MAP'].isin(pd.merge(refseq_38_df_transformed[['symbol', 'CHROM', 'Exon_list']].explode('Exon_list').rename({'Exon_list' : 'Exon_38'},axis=1), exotic_remap_map, on='Exon_38').MAP.values)],
    refseq_37,
    on='MAP'
).CDS_representation.value_counts()

In [126]:
pd.merge(
    exotic[["MAP"]],
    refseq_37,
    on='MAP'
).CDS_representation.value_counts()

Unique             20454
Variable_region     7071
Name: CDS_representation, dtype: int64

In [84]:
exotic_remap_map = pd.read_csv('/biolo/ngs/remap/EXOTIC_remap.bed', sep='\t', names=['CHROM_38', 'Exon_start_38', 'Exon_end_38', 'MAP'])
exotic_remap_map = exotic_remap_map.loc[~exotic_remap_map['CHROM_38'].str.contains('HSCHR')]
exotic_remap_map['Exon_38'] = exotic_remap_map['Exon_start_38'].astype(str) + '_' + exotic_remap_map['Exon_end_38'].astype(str)
exotic_remap_map

,CHROM_38,Exon_start_38,Exon_end_38,MAP,Exon_38
0,1,36140069,36140070,TRAPPC3_36605670-36605671,36140069_36140070
1,1,43451310,43451311,HYI_43916981-43916982,43451310_43451311
2,1,46665647,46665648,ATPAF1_47131319-47131320,46665647_46665648
3,1,69568640,69568641,LRRC7_70034323-70034324,69568640_69568641
4,1,161118177,161118178,NIT1_161087967-161087968,161118177_161118178
...,...,...,...,...,...
27457,Y,14622120,14622591,NLGN4Y_16734000-16734471,14622120_14622591
27458,Y,14723149,14723269,NLGN4Y_16835029-16835149,14723149_14723269
27459,Y,19723341,19723433,KDM5D_21885227-21885319,19723341_19723433
27460,Y,19741318,19741488,KDM5D_21903204-21903374,19741318_19741488


In [110]:
exotic_remap_map.loc[~exotic_remap_map['MAP'].isin(pd.merge(refseq_38_df_transformed[['symbol', 'CHROM', 'Exon_list']].explode('Exon_list').rename({'Exon_list' : 'Exon_38'},axis=1), exotic_remap_map, on='Exon_38').MAP.values)]

,CHROM_38,Exon_start_38,Exon_end_38,MAP,Exon_38
0,1,36140069,36140070,TRAPPC3_36605670-36605671,36140069_36140070
1,1,43451310,43451311,HYI_43916981-43916982,43451310_43451311
5,1,971113,971208,PLEKHN1_906493-906588,971113_971208
7,1,999692,999787,HES4_935072-935167,999692_999787
8,1,999866,999973,HES4_935246-935353,999866_999973
...,...,...,...,...,...
27452,Y,13297707,13297848,UTY_15409587-15409728,13297707_13297848
27453,Y,13355375,13355398,UTY_15467255-15467278,13355375_13355398
27454,Y,13358464,13358553,UTY_15470344-15470433,13358464_13358553
27456,Y,13360430,13360480,UTY_15472310-15472360,13360430_13360480


In [134]:
tmp

,symbol,ensg,MAP,Exon,mean_proportion,EXOTIC_min,EXOTIC_tissues_min,EXOTIC_pext_associated_min,EXOTIC_check_pext_min,EXOTIC_tissues_corrected_min,...,EXOTIC_tissues_nb_max,Exon_start,Exon_end,EXOTIC_bins_max,EXOTIC_bins_min,CHROM,CHROM_38,Exon_start_38,Exon_end_38,Exon_38
0,A2ML1,ENSG00000166535,A2ML1_8975248-8975309,8975248-8975309,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17518],[ True],[Brain - Cerebellar Hemisphere],...,24,8975248,8975309,0.7 - 0.75,0.15 - 0.2,12,12,8822652,8822713,8822652_8822713
1,A2ML1,ENSG00000166535,A2ML1_8975778-8975961,8975778-8975961,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17518],[ True],[Brain - Cerebellar Hemisphere],...,24,8975778,8975961,0.7 - 0.75,0.15 - 0.2,12,12,8823182,8823365,8823182_8823365
2,A2ML1,ENSG00000166535,A2ML1_8976316-8976478,8976316-8976478,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17518],[ True],[Brain - Cerebellar Hemisphere],...,24,8976316,8976478,0.7 - 0.75,0.15 - 0.2,12,12,8823720,8823882,8823720_8823882
3,A2ML1,ENSG00000166535,A2ML1_8982323-8982375,8982323-8982375,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17518],[ True],[Brain - Cerebellar Hemisphere],...,24,8982323,8982375,0.7 - 0.75,0.15 - 0.2,12,12,8829727,8829779,8829727_8829779
4,A2ML1,ENSG00000166535,A2ML1_8987258-8987278,8987258-8987278,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17518],[ True],[Brain - Cerebellar Hemisphere],...,24,8987258,8987278,0.7 - 0.75,0.15 - 0.2,12,12,8834662,8834682,8834662_8834682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27493,ZXDC,ENSG00000070476,ZXDC_126158475-126158570,126158475-126158570,0.20989,0.150855,[Brain - Nucleus accumbens (basal ganglia)],[0.11398],[ True],[Brain - Nucleus accumbens (basal ganglia)],...,1,126158475,126158570,0.9 - 0.95,0.15 - 0.2,3,3,126439632,126439727,126439632_126439727
27494,ZXDC,ENSG00000070476,ZXDC_126160608-126160789,126160608-126160789,0.20989,0.150855,[Brain - Nucleus accumbens (basal ganglia)],[0.11398],[ True],[Brain - Nucleus accumbens (basal ganglia)],...,1,126160608,126160789,0.9 - 0.95,0.15 - 0.2,3,3,126441765,126441946,126441765_126441946
27495,ZXDC,ENSG00000070476,ZXDC_126178496-126178580,126178496-126178580,0.20989,0.150855,[Brain - Nucleus accumbens (basal ganglia)],[0.11398],[ True],[Brain - Nucleus accumbens (basal ganglia)],...,1,126178496,126178580,0.9 - 0.95,0.15 - 0.2,3,3,126459653,126459737,126459653_126459737
27496,ZXDC,ENSG00000070476,ZXDC_126180378-126181063,126180378-126181063,0.34803,0.139039,[Whole Blood],[0.20951999999999998],[ True],[Whole Blood],...,1,126180378,126181063,0.9 - 0.95,0.1 - 0.15,3,3,126461535,126462220,126461535_126462220


In [136]:
tmp = pd.merge(exotic, exotic_remap_map, on='MAP')
tmp['EXOTIC_check_pext_min'] = tmp['EXOTIC_check_pext_min'].astype(str)
tmp['EXOTIC_check_pext_max'] = tmp['EXOTIC_check_pext_max'].astype(str)
tmp.to_parquet('/gstock/EXOTIC/data/EXOTIC/EXOTIC_38.parquet')

# DROPNA

In [33]:
exotic['Exon_start_38'].isna().sum()

11275

In [34]:
exotic['Exon_end_38'].isna().sum()

11272

In [41]:
exotic['Exon_end_38'].isna().sum() / exotic_test.shape[0]


0.40951861943687556

In [39]:
t = pd.concat([exotic['Exon_start_38'].isna(), exotic['EXOTIC_bins_max']], axis=1).groupby('EXOTIC_bins_max')['Exon_start_38'].value_counts()
t = pd.DataFrame(t) 
t.columns = ['Count']
t = t.reset_index()
t = t.pivot(index='EXOTIC_bins_max', columns='Exon_start_38', values='Count')
t['Total'] = t[True] + t[False]
t['Diff%'] = 100 * (((t["Total"] - t[False]) / t["Total"]))
t


Exon_start_38,False,True,Total,Diff%
EXOTIC_bins_max,,,,
0.5 - 0.55,140,69,209,33.014354
0.55 - 0.6,200,112,312,35.897436
0.6 - 0.65,215,216,431,50.116009
0.65 - 0.7,356,292,648,45.061728
0.7 - 0.75,419,468,887,52.762120
0.75 - 0.8,963,605,1568,38.584184
0.8 - 0.85,2301,1291,3592,35.940980
0.85 - 0.9,3515,2578,6093,42.310849
0.9 - 0.95,4094,2893,6987,41.405467


In [40]:
t = pd.concat([exotic['Exon_start_38'].isna(), exotic['EXOTIC_bins_min']], axis=1).groupby('EXOTIC_bins_min')['Exon_start_38'].value_counts()
t = pd.DataFrame(t) 
t.columns = ['Count']
t = t.reset_index()
t = t.pivot(index='EXOTIC_bins_min', columns='Exon_start_38', values='Count')
t['Total'] = t[True] + t[False]
t['Diff%'] = 100 * (((t["Total"] - t[False]) / t["Total"]))
t


Exon_start_38,False,True,Total,Diff%
EXOTIC_bins_min,,,,
0.0 - 0.05,4079,3036,7115,42.670415
0.05 - 0.1,4093,2841,6934,40.972022
0.1 - 0.15,3562,2355,5917,39.800575
0.15 - 0.2,2057,1377,3434,40.099010
0.2 - 0.25,893,644,1537,41.899805
0.25 - 0.3,523,362,885,40.903955
0.3 - 0.35,363,223,586,38.054608
0.35 - 0.4,312,224,536,41.791045
0.4 - 0.45,153,96,249,38.554217


In [48]:
refseq_38_df_transformed = refseq_38_df_transformed.reset_index().rename({'index' : 'symbol'}, axis=1)
refseq_38_df_transformed

,symbol,CHROM,Gene_start_end,Exon_list,5_prime_UTR,3_prime_UTR,Intron_list,Exon_size_total,Bin_size_exon,Intron_size_total,Bin_size_intron,Gene_body_size,Bin_size_GB,Bin_size_UpDown,5_prime_UTR_size,3_prime_UTR_size,Bin_size_5_prime_UTR,Bin_size_3_prime_UTR,BED_boundaries_for_hail_10kbp_added
0,A1BG,2,58345183_58353492,"[58347022_58347029, 58347353_58347640, 5835037...",58345183_58347021,58353438_58353492,"[58347030_58347352, 58347641_58350369, 5835065...",1480,74.00,4921,246.05,8309,415.45,250.0,1838,54,91.90,2.70,chr2:58340183-58358492
1,A1CF,1,50799409_50885666,"[50806729_50806880, 50809894_50810042, 5081104...",50799409_50806728,50859941_50885666,"[50806881_50809893, 50810043_50811039, 5081117...",1896,94.80,51293,2564.65,86257,4312.85,250.0,7319,25725,365.95,1286.25,chr1:50794409-50890666
2,A2M,2,9067708_9116229,"[9067823_9067839, 9068183_9068224, 9068740_906...",9067708_9067822,9115850_9116229,"[9067840_9068182, 9068225_9068739, 9068843_906...",4389,219.45,43567,2178.35,48521,2426.05,250.0,114,379,5.70,18.95,chr2:9062708-9121229
3,A2ML1,2,8822554_8887202,"[8822652_8822713, 8823182_8823365, 8823720_882...",8822554_8822651,8875012_8887202,"[8822714_8823181, 8823366_8823719, 8823883_882...",4332,216.60,47957,2397.85,64648,3232.40,250.0,97,12190,4.85,609.50,chr2:8817554-8892202
4,A3GALT2,1,33306766_33321098,"[33306766_33307453, 33312052_33312189, 3331250...",33306766_33306765,33321099_33321098,"[33307454_33312051, 33312190_33312500, 3331259...",1018,50.90,13306,665.30,14332,716.60,250.0,-1,-1,-0.05,-0.05,chr1:33301766-33326098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19382,ZYG11A,1,52842736_52894998,"[52842884_52842973, 52854465_52854630, 5285699...",52842736_52842883,52892958_52894998,"[52842974_52854464, 52854631_52856997, 5285775...",2266,113.30,47781,2389.05,52262,2613.10,250.0,147,2040,7.35,102.00,chr1:52837736-52899998
19383,ZYG11B,1,52726453_52827336,"[52726654_52726683, 52756458_52756623, 5277102...",52726453_52726653,52821630_52827336,"[52726684_52756457, 52756624_52771019, 5277177...",2221,111.05,92728,4636.40,100883,5044.15,250.0,200,5706,10.00,285.30,chr1:52721453-52832336
19384,ZYX,2,709_10475,"[936_1143, 1612_1811, 1957_2049, 2165_2686, 75...",709_935,143390683_10475,"[1144_1611, 1812_1956, 2050_2164, 2687_7582, 7...",3420,171.00,143386292,7169314.60,9766,488.30,250.0,226,-143380208,11.30,-7169010.40,chr2:-4291-15475
19385,ZZEF1,2,4004445_4143030,"[4006890_4006970, 4008883_4008954, 4009604_400...",4004445_4006889,4142896_4143030,"[4006971_4008882, 4008955_4009603, 4009758_401...",8831,441.55,127066,6353.30,138585,6929.25,250.0,2444,134,122.20,6.70,chr2:3999445-4148030


In [51]:
exotic

,symbol,ensg,MAP,Exon,mean_proportion,EXOTIC_min,EXOTIC_tissues_min,EXOTIC_pext_associated_min,EXOTIC_check_pext_min,EXOTIC_tissues_corrected_min,...,EXOTIC_tissues_corrected_max,EXOTIC_tissues_nb_max,Exon_start,Exon_end,EXOTIC_bins_max,EXOTIC_bins_min,CHROM,Exon_start_38,Exon_end_38,Exon_38
0,A2ML1,ENSG00000166535,A2ML1_8975248-8975309,8975248-8975309,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17518],[True],[Brain - Cerebellar Hemisphere],...,"[Adipose - Subcutaneous, Adipose - Visceral (O...",24,8975248,8975309,0.7 - 0.75,0.15 - 0.2,12,NaN,NaN,nan_nan
1,A2ML1,ENSG00000166535,A2ML1_8975778-8975961,8975778-8975961,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17518],[True],[Brain - Cerebellar Hemisphere],...,"[Adipose - Subcutaneous, Adipose - Visceral (O...",24,8975778,8975961,0.7 - 0.75,0.15 - 0.2,12,NaN,NaN,nan_nan
2,A2ML1,ENSG00000166535,A2ML1_8976316-8976478,8976316-8976478,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17518],[True],[Brain - Cerebellar Hemisphere],...,"[Adipose - Subcutaneous, Adipose - Visceral (O...",24,8976316,8976478,0.7 - 0.75,0.15 - 0.2,12,NaN,NaN,nan_nan
3,A2ML1,ENSG00000166535,A2ML1_8982323-8982375,8982323-8982375,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17518],[True],[Brain - Cerebellar Hemisphere],...,"[Adipose - Subcutaneous, Adipose - Visceral (O...",24,8982323,8982375,0.7 - 0.75,0.15 - 0.2,12,NaN,NaN,nan_nan
4,A2ML1,ENSG00000166535,A2ML1_8987258-8987278,8987258-8987278,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17518],[True],[Brain - Cerebellar Hemisphere],...,"[Adipose - Subcutaneous, Adipose - Visceral (O...",24,8987258,8987278,0.7 - 0.75,0.15 - 0.2,12,NaN,NaN,nan_nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27520,ZXDC,ENSG00000070476,ZXDC_126158475-126158570,126158475-126158570,0.20989,0.150855,[Brain - Nucleus accumbens (basal ganglia)],[0.11398],[True],[Brain - Nucleus accumbens (basal ganglia)],...,[Muscle - Skeletal],1,126158475,126158570,0.9 - 0.95,0.15 - 0.2,3,126116797.0,126116892.0,126116797.0_126116892.0
27521,ZXDC,ENSG00000070476,ZXDC_126160608-126160789,126160608-126160789,0.20989,0.150855,[Brain - Nucleus accumbens (basal ganglia)],[0.11398],[True],[Brain - Nucleus accumbens (basal ganglia)],...,[Muscle - Skeletal],1,126160608,126160789,0.9 - 0.95,0.15 - 0.2,3,126118930.0,126119111.0,126118930.0_126119111.0
27522,ZXDC,ENSG00000070476,ZXDC_126178496-126178580,126178496-126178580,0.20989,0.150855,[Brain - Nucleus accumbens (basal ganglia)],[0.11398],[True],[Brain - Nucleus accumbens (basal ganglia)],...,[Muscle - Skeletal],1,126178496,126178580,0.9 - 0.95,0.15 - 0.2,3,126136818.0,126136902.0,126136818.0_126136902.0
27523,ZXDC,ENSG00000070476,ZXDC_126180378-126181063,126180378-126181063,0.34803,0.139039,[Whole Blood],[0.20951999999999998],[True],[Whole Blood],...,[Artery - Tibial],1,126180378,126181063,0.9 - 0.95,0.1 - 0.15,3,126138700.0,126139385.0,126138700.0_126139385.0


In [54]:
exotic_wt_nas

,symbol,ensg,MAP,Exon,mean_proportion,EXOTIC_min,EXOTIC_tissues_min,EXOTIC_pext_associated_min,EXOTIC_check_pext_min,EXOTIC_tissues_corrected_min,...,EXOTIC_tissues_corrected_max,EXOTIC_tissues_nb_max,Exon_start,Exon_end,EXOTIC_bins_max,EXOTIC_bins_min,CHROM,Exon_start_38,Exon_end_38,Exon_38
17,AACS,ENSG00000081760,AACS_125609448-125609570,125609448-125609570,0.761190,0.114371,[Brain - Hippocampus],[0.60343],[True],[Brain - Hippocampus],...,[Muscle - Skeletal],1,125609448,125609570,0.9 - 0.95,0.1 - 0.15,12,125500282.0,125500404.0,125500282_125500404
18,AACS,ENSG00000081760,AACS_125621208-125621410,125621208-125621410,0.780410,0.097720,[Brain - Hippocampus],[0.61934],[True],[Brain - Hippocampus],...,[Adrenal Gland],1,125621208,125621410,0.85 - 0.9,0.05 - 0.1,12,125512042.0,125512244.0,125512042_125512244
19,AACS,ENSG00000081760,AACS_125626638-125626759,125626638-125626759,0.933440,0.054758,[Fallopian Tube],[0.85136],[True],[Fallopian Tube],...,[Muscle - Skeletal],1,125626638,125626759,0.8 - 0.85,0.05 - 0.1,12,125517472.0,125517593.0,125517472_125517593
33,AAMDC,ENSG00000087884,AAMDC_77552232-77552270,77552232-77552270,0.008957,0.269705,"[Brain - Amygdala, Brain - Anterior cingulate ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[True, True, True, True, True, True, True, Tru...","[Brain - Amygdala, Brain - Anterior cingulate ...",...,[],0,77552232,77552270,0.95 - 1.0,0.25 - 0.3,11,77552232.0,77552270.0,77552232_77552270
34,AAMDC,ENSG00000087884,AAMDC_77553543-77553674,77553543-77553674,0.971930,0.009409,[Brain - Cerebellar Hemisphere],[0.8974700000000001],[True],[Brain - Cerebellar Hemisphere],...,"[Liver, Whole Blood]",2,77553543,77553674,0.85 - 0.9,0.0 - 0.05,11,77553543.0,77553674.0,77553543_77553674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27520,ZXDC,ENSG00000070476,ZXDC_126158475-126158570,126158475-126158570,0.209890,0.150855,[Brain - Nucleus accumbens (basal ganglia)],[0.11398],[True],[Brain - Nucleus accumbens (basal ganglia)],...,[Muscle - Skeletal],1,126158475,126158570,0.9 - 0.95,0.15 - 0.2,3,126116797.0,126116892.0,126116797_126116892
27521,ZXDC,ENSG00000070476,ZXDC_126160608-126160789,126160608-126160789,0.209890,0.150855,[Brain - Nucleus accumbens (basal ganglia)],[0.11398],[True],[Brain - Nucleus accumbens (basal ganglia)],...,[Muscle - Skeletal],1,126160608,126160789,0.9 - 0.95,0.15 - 0.2,3,126118930.0,126119111.0,126118930_126119111
27522,ZXDC,ENSG00000070476,ZXDC_126178496-126178580,126178496-126178580,0.209890,0.150855,[Brain - Nucleus accumbens (basal ganglia)],[0.11398],[True],[Brain - Nucleus accumbens (basal ganglia)],...,[Muscle - Skeletal],1,126178496,126178580,0.9 - 0.95,0.15 - 0.2,3,126136818.0,126136902.0,126136818_126136902
27523,ZXDC,ENSG00000070476,ZXDC_126180378-126181063,126180378-126181063,0.348030,0.139039,[Whole Blood],[0.20951999999999998],[True],[Whole Blood],...,[Artery - Tibial],1,126180378,126181063,0.9 - 0.95,0.1 - 0.15,3,126138700.0,126139385.0,126138700_126139385


In [53]:
exotic_wt_nas = exotic.dropna(subset=['Exon_start_38', 'Exon_end_38'])
exotic_wt_nas['Exon_38'] = exotic_wt_nas['Exon_start_38'].astype(int).astype(str) + '_' + exotic_wt_nas['Exon_end_38'].astype(int).astype(str)
pd.merge(refseq_38_df_transformed[['symbol', 'CHROM', 'Exon_list']].explode('Exon_list'), exotic_wt_nas, left_on='Exon_list', right_on='Exon_38')

,symbol_x,CHROM_x,Exon_list,symbol_y,ensg,MAP,Exon,mean_proportion,EXOTIC_min,EXOTIC_tissues_min,...,EXOTIC_tissues_corrected_max,EXOTIC_tissues_nb_max,Exon_start,Exon_end,EXOTIC_bins_max,EXOTIC_bins_min,CHROM_y,Exon_start_38,Exon_end_38,Exon_38
0,ABCB7,2,75098942_75099061,ABCB7,ENSG00000131269,ABCB7_74318777-74318896,74318777-74318896,0.88542,0.026348,[Liver],...,[Muscle - Skeletal],1,74318777,74318896,0.8 - 0.85,0.0 - 0.05,X,75098942.0,75099061.0,75098942_75099061
1,ABHD17B,1,71862581_71862607,ABHD17B,ENSG00000107362,ABHD17B_74477497-74477523,74477497-74477523,0.19793,0.166596,[Whole Blood],...,[Cells - Cultured fibroblasts],1,74477497,74477523,0.9 - 0.95,0.15 - 0.2,9,71862581.0,71862607.0,71862581_71862607
2,ACAT2,1,159762088_159762142,ACAT2,ENSG00000120437,ACAT2_160183120-160183174,160183120-160183174,0.12641,0.107266,[Liver],...,[Testis],1,160183120,160183174,0.95 - 1.0,0.1 - 0.15,6,159762088.0,159762142.0,159762088_159762142
3,ACAT2,1,159762638_159762779,ACAT2,ENSG00000120437,ACAT2_160183670-160183811,160183670-160183811,0.79909,0.083261,[Pituitary],...,[Esophagus - Mucosa],1,160183670,160183811,0.8 - 0.85,0.05 - 0.1,6,159762638.0,159762779.0,159762638_159762779
4,ACP1,1,275140_275201,ACP1,ENSG00000143727,ACP1_275140-275201,275140-275201,0.74595,0.016588,[Testis],...,[Brain - Anterior cingulate cortex (BA24)],1,275140,275201,0.8 - 0.85,0.0 - 0.05,2,275140.0,275201.0,275140_275201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,ZNF680,1,64526399_64526517,ZNF680,ENSG00000173041,ZNF680_63986777-63986895,63986777-63986895,0.15807,0.261880,[Spleen],...,[Brain - Frontal Cortex (BA9)],1,63986777,63986895,0.85 - 0.9,0.25 - 0.3,7,64526399.0,64526517.0,64526399_64526517
406,ZNF736,1,64348090_64349147,ZNF736,ENSG00000234444,ZNF736_63808468-63809525,63808468-63809525,0.48901,0.089022,[Brain - Spinal cord (cervical c-1)],...,[Whole Blood],1,63808468,63809525,0.95 - 1.0,0.05 - 0.1,7,64348090.0,64349147.0,64348090_64349147
407,ZNF75A,2,3316912_3317022,ZNF75A,ENSG00000162086,ZNF75A_3366912-3367022,3366912-3367022,0.27652,0.183680,[Cervix - Endocervix],...,[Brain - Putamen (basal ganglia)],1,3366912,3367022,0.9 - 0.95,0.15 - 0.2,16,3316912.0,3317022.0,3316912_3317022
408,ZSCAN32,2,3390423_3390517,ZSCAN32,ENSG00000140987,ZSCAN32_3440423-3440517,3440423-3440517,0.22787,0.045012,[Cervix - Endocervix],...,[Cells - Cultured fibroblasts],1,3440423,3440517,0.9 - 0.95,0.0 - 0.05,16,3390423.0,3390517.0,3390423_3390517


In [57]:
refseq_38_df_transformed.loc[refseq_38_df_transformed['symbol'] == 'AACS']['Exon_list'].values

array([array(['125065585_125065717', '125073876_125073979',
       '125076491_125076611', '125086330_125086443',
       '125091426_125091523', '125102679_125102793',
       '125103000_125103081', '125107121_125107268',
       '125114477_125114557', '125118641_125118765',
       '125124705_125124769', '125124902_125125024',
       '125128161_125128274', '125129335_125129460',
       '125134003_125134072', '125134794_125134852',
       '125136662_125136864', '125142092_125142229'], dtype=object)],
      dtype=object)

In [62]:
tmp = refseq_38_df_transformed[['symbol', 'CHROM', 'Exon_list']].explode('Exon_list')
tmp[['Exon_start', 'Exon_end']] = tmp['Exon_list'].str.split('_', expand=True)
tmp['Exon_start_37'] = tmp[['CHROM', 'Exon_start']].progress_apply(lambda r: apply_liftover(r, 'Exon_start'), axis=1)
tmp['Exon_end_37'] = tmp[['CHROM', 'Exon_end']].progress_apply(lambda r: apply_liftover(r, 'Exon_end'), axis=1)
tmp = tmp.dropna()
tmp['Exon_start_37'] = tmp['Exon_start_37'].astype(int)
tmp['Exon_end_37'] = tmp['Exon_end_37'].astype(int)
tmp


100%|██████████| 223771/223771 [00:02<00:00, 85415.18it/s] 


,symbol,CHROM,Exon_list,Exon_start,Exon_end,Exon_start_37,Exon_end_37
0,A1BG,2,58347022_58347029,58347022,58347029,NaN,NaN
0,A1BG,2,58347353_58347640,58347353,58347640,NaN,NaN
0,A1BG,2,58350370_58350651,58350370,58350651,NaN,NaN
0,A1BG,2,58351391_58351687,58351391,58351687,NaN,NaN
0,A1BG,2,58352283_58352555,58352283,58352555,NaN,NaN
...,...,...,...,...,...,...,...
19386,ZZZ3,1,77581776_77581891,77581776,77581891,77908160.0,77908275.0
19386,ZZZ3,1,77581979_77582126,77581979,77582126,77908363.0,77908510.0
19386,ZZZ3,1,77584517_77584655,77584517,77584655,77910901.0,77911039.0
19386,ZZZ3,1,77631850_77633354,77631850,77633354,77958234.0,77959738.0


In [71]:
tmp = tmp.dropna()
tmp['Exon_start_37'] = tmp['Exon_start_37'].astype(int).astype(str)
tmp['Exon_end_37'] = tmp['Exon_end_37'].astype(int).astype(str)
tmp['Exon_37'] = tmp['Exon_start_37'].astype(int).astype(str) + '-' + tmp['Exon_end_37'].astype(int).astype(str)

tmp

,symbol,CHROM,Exon_list,Exon_start,Exon_end,Exon_start_37,Exon_end_37,Exon_37
1,A1CF,1,50806729_50806880,50806729,50806880,51133113,51133264,51133113-51133264
1,A1CF,1,50809894_50810042,50809894,50810042,51136278,51136426,51136278-51136426
1,A1CF,1,50811040_50811176,50811040,50811176,51137424,51137560,51137424-51137560
1,A1CF,1,50813857_50814062,50813857,50814062,51140241,51140446,51140241-51140446
1,A1CF,1,50816006_50816279,50816006,50816279,51142390,51142663,51142390-51142663
...,...,...,...,...,...,...,...,...
19386,ZZZ3,1,77581776_77581891,77581776,77581891,77908160,77908275,77908160-77908275
19386,ZZZ3,1,77581979_77582126,77581979,77582126,77908363,77908510,77908363-77908510
19386,ZZZ3,1,77584517_77584655,77584517,77584655,77910901,77911039,77910901-77911039
19386,ZZZ3,1,77631850_77633354,77631850,77633354,77958234,77959738,77958234-77959738


In [67]:
exotic

,symbol,ensg,MAP,Exon,mean_proportion,EXOTIC_min,EXOTIC_tissues_min,EXOTIC_pext_associated_min,EXOTIC_check_pext_min,EXOTIC_tissues_corrected_min,...,EXOTIC_tissues_corrected_max,EXOTIC_tissues_nb_max,Exon_start,Exon_end,EXOTIC_bins_max,EXOTIC_bins_min,CHROM,Exon_start_38,Exon_end_38,Exon_38
0,A2ML1,ENSG00000166535,A2ML1_8975248-8975309,8975248-8975309,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17518],[True],[Brain - Cerebellar Hemisphere],...,"[Adipose - Subcutaneous, Adipose - Visceral (O...",24,8975248,8975309,0.7 - 0.75,0.15 - 0.2,12,NaN,NaN,nan_nan
1,A2ML1,ENSG00000166535,A2ML1_8975778-8975961,8975778-8975961,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17518],[True],[Brain - Cerebellar Hemisphere],...,"[Adipose - Subcutaneous, Adipose - Visceral (O...",24,8975778,8975961,0.7 - 0.75,0.15 - 0.2,12,NaN,NaN,nan_nan
2,A2ML1,ENSG00000166535,A2ML1_8976316-8976478,8976316-8976478,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17518],[True],[Brain - Cerebellar Hemisphere],...,"[Adipose - Subcutaneous, Adipose - Visceral (O...",24,8976316,8976478,0.7 - 0.75,0.15 - 0.2,12,NaN,NaN,nan_nan
3,A2ML1,ENSG00000166535,A2ML1_8982323-8982375,8982323-8982375,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17518],[True],[Brain - Cerebellar Hemisphere],...,"[Adipose - Subcutaneous, Adipose - Visceral (O...",24,8982323,8982375,0.7 - 0.75,0.15 - 0.2,12,NaN,NaN,nan_nan
4,A2ML1,ENSG00000166535,A2ML1_8987258-8987278,8987258-8987278,0.69514,0.153430,[Brain - Cerebellar Hemisphere],[0.17518],[True],[Brain - Cerebellar Hemisphere],...,"[Adipose - Subcutaneous, Adipose - Visceral (O...",24,8987258,8987278,0.7 - 0.75,0.15 - 0.2,12,NaN,NaN,nan_nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27520,ZXDC,ENSG00000070476,ZXDC_126158475-126158570,126158475-126158570,0.20989,0.150855,[Brain - Nucleus accumbens (basal ganglia)],[0.11398],[True],[Brain - Nucleus accumbens (basal ganglia)],...,[Muscle - Skeletal],1,126158475,126158570,0.9 - 0.95,0.15 - 0.2,3,126116797.0,126116892.0,126116797.0_126116892.0
27521,ZXDC,ENSG00000070476,ZXDC_126160608-126160789,126160608-126160789,0.20989,0.150855,[Brain - Nucleus accumbens (basal ganglia)],[0.11398],[True],[Brain - Nucleus accumbens (basal ganglia)],...,[Muscle - Skeletal],1,126160608,126160789,0.9 - 0.95,0.15 - 0.2,3,126118930.0,126119111.0,126118930.0_126119111.0
27522,ZXDC,ENSG00000070476,ZXDC_126178496-126178580,126178496-126178580,0.20989,0.150855,[Brain - Nucleus accumbens (basal ganglia)],[0.11398],[True],[Brain - Nucleus accumbens (basal ganglia)],...,[Muscle - Skeletal],1,126178496,126178580,0.9 - 0.95,0.15 - 0.2,3,126136818.0,126136902.0,126136818.0_126136902.0
27523,ZXDC,ENSG00000070476,ZXDC_126180378-126181063,126180378-126181063,0.34803,0.139039,[Whole Blood],[0.20951999999999998],[True],[Whole Blood],...,[Artery - Tibial],1,126180378,126181063,0.9 - 0.95,0.1 - 0.15,3,126138700.0,126139385.0,126138700.0_126139385.0


In [73]:
pd.merge(exotic[['symbol', 'Exon_end']], tmp[['symbol', 'Exon_end_37']], left_on='Exon_end', right_on='Exon_end_37')

,symbol_x,Exon_end,symbol_y,Exon_end_37
0,ACSM5,20448689,PRMT3,20448689
1,ANKRD65,1354929,ANKRD65,1354929
2,ANKRD65,1355972,ANKRD65,1355972
3,ANKRD65,1356385,ANKRD65,1356385
4,ANO9,431906,ANO9,431906
...,...,...,...,...
118,UBE2J2,1203372,UBE2J2,1203372
119,USP14,179037,USP14,179037
120,VWA1,1372823,VWA1,1372823
121,VWA1,1375167,VWA1,1375167
